In [1]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
from IPython.display import display
import itertools
from typing import Callable

pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", None)

data = pd.read_csv("./french-verb-conjugation.csv").replace(
    {np.nan: None}
)
data = data.drop(columns=["compound verb"])

Remove obsolete, uncommon, and misspelled words:


In [2]:
data = data[
    ~data["infinitive"].isin(
        [
            "estre",  # être
            "copier-coller",  # Complicated to analyze because both parts conjugate
            "ruisseller",  # ruisseler
            "râtisser",  # ratisser
            "râcler",  # racler
            "céler",  # celer
            "recéler",  # receler
            "guèder",  # guéder
            "assener",  # asséner
            "décolléter",  # décolleter
            "démèner",  # démener
            "œconomiser",  # économiser
            # These have weird future je conjugations
            "brusler",  # brûler
            "cuider",
            "esloigner",  # éloigner
            "esposer",  # exposer
        ]
    )
]

Words that have multiple spellings in the same cell: explode them into separate rows. Later, we will merge them again.


In [3]:
multi_value = data.applymap(lambda x: ";" in str(x)).any(axis=1)
data[multi_value]

,infinitive,present participle,past participle,present 1sg,present 2sg,present 3sg,present 1pl,present 2pl,present 3pl,imperfect 1sg,imperfect 2sg,imperfect 3sg,imperfect 1pl,imperfect 2pl,imperfect 3pl,past historic 1sg,past historic 2sg,past historic 3sg,past historic 1pl,past historic 2pl,past historic 3pl,future 1sg,future 2sg,future 3sg,future 1pl,future 2pl,future 3pl,conditional 1sg,conditional 2sg,conditional 3sg,conditional 1pl,conditional 2pl,conditional 3pl,subjunctive present 1sg,subjunctive present 2sg,subjunctive present 3sg,subjunctive present 1pl,subjunctive present 2pl,subjunctive present 3pl,subjunctive imperfect 1sg,subjunctive imperfect 2sg,subjunctive imperfect 3sg,subjunctive imperfect 1pl,subjunctive imperfect 2pl,subjunctive imperfect 3pl,imperative 2sg,imperative 1pl,imperative 2pl
5,abayer,abayant,abayé,abaye;abaie,abayes;abaies,abaye;abaie,abayons,abayez,abayent;abaient,abayais,abayais,abayait,abayions,abayiez,abayaient,abayai,abayas,abaya,abayâmes,abayâtes,abayèrent,abayerai;abaierai,abayeras;abaieras,abayera;abaiera,abayerons;abaierons,abayerez;abaierez,abayeront;abaieront,abayerais;abaierais,abayerais;abaierais,abayerait;abaierait,abayerions;abaierions,abayeriez;abaieriez,abayeraient;abaieraient,abaye;abaie,abayes;abaies,abaye;abaie,abayions,abayiez,abayent;abaient,abayasse,abayasses,abayât,abayassions,abayassiez,abayassent,abaye;abaie,abayons,abayez
6,abcéder,abcédant,abcédé,abcède,abcèdes,abcède,abcédons,abcédez,abcèdent,abcédais,abcédais,abcédait,abcédions,abcédiez,abcédaient,abcédai,abcédas,abcéda,abcédâmes,abcédâtes,abcédèrent,abcèderai;abcéderai,abcèderas;abcéderas,abcèdera;abcédera,abcèderons;abcéderons,abcèderez;abcéderez,abcèderont;abcéderont,abcèderais;abcéderais,abcèderais;abcéderais,abcèderait;abcéderait,abcèderions;abcéderions,abcèderiez;abcéderiez,abcèderaient;abcéderaient,abcède,abcèdes,abcède,abcédions,abcédiez,abcèdent,abcédasse,abcédasses,abcédât,abcédassions,abcédassiez,abcédassent,abcède,abcédons,abcédez
35,abréger,abrégeant,abrégé,abrège,abrèges,abrège,abrégeons,abrégez,abrègent,abrégeais,abrégeais,abrégeait,abrégions,abrégiez,abrégeaient,abrégeai,abrégeas,abrégea,abrégeâmes,abrégeâtes,abrégèrent,abrègerai;abrégerai,abrègeras;abrégeras,abrègera;abrégera,abrègerons;abrégerons,abrègerez;abrégerez,abrègeront;abrégeront,abrègerais;abrégerais,abrègerais;abrégerais,abrègerait;abrégerait,abrègerions;abrégerions,abrègeriez;abrégeriez,abrègeraient;abrégeraient,abrège,abrèges,abrège,abrégions,abrégiez,abrègent,abrégeasse,abrégeasses,abrégeât,abrégeassions,abrégeassiez,abrégeassent,abrège,abrégeons,abrégez
42,absoudre,absolvant,absous;absout,absous,absous,absout,absolvons,absolvez,absolvent,absolvais,absolvais,absolvait,absolvions,absolviez,absolvaient,None,None,None,None,None,None,absoudrai,absoudras,absoudra,absoudrons,absoudrez,absoudront,absoudrais,absoudrais,absoudrait,absoudrions,absoudriez,absoudraient,absolve,absolves,absolve,absolvions,absolviez,absolvent,None,None,None,None,None,None,absous,absolvons,absolvez
77,accueillir,accueillant,accueilli,accueille,accueilles,accueille,accueillons,accueillez,accueillent,accueillais,accueillais,accueillait,accueillions,accueilliez,accueillaient,accueillis,accueillis,accueillit,accueillîmes,accueillîtes,accueillirent,accueillirai;accueillerai,accueilliras;accueilleras,accueillira;accueillera,accueillirons;accueillerons,accueillirez;accueillerez,accueilliront;accueilleront,accueillirais;accueillerais,accueillirais;accueillerais,accueillirait;accueillerait,accueillirions;accueillerions,accueilliriez;accueilleriez,accueilliraient;accueilleraient,accueille,accueilles,accueille,accueillions,accueilliez,accueillent,accueillisse,accueillisses,accueillît,accueillissions,accueillissiez,accueillissent,accueille,accueillons,accueillez
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5272,épousseter,épousset

In [4]:
def clean_multi_value(item: str):
    if item and ";" in item:
        alternatives = item.split(";")
        assert len(alternatives) == 2
        return tuple(alternatives)
    return item


data.loc[multi_value] = data[multi_value].applymap(clean_multi_value)

For orthographic variations, use uppercase placeholders to indicate ambiguity.

For the "choirai/cherrai" pair, we chose the regular choirai form.


In [5]:
def drop_dup(group: pd.DataFrame):
    if len(group) > 1:
        display(group)
        raise ValueError("Different")
    return group


known_dupes = data["infinitive"].isin(
    itertools.chain(
        ("apparaitre", "apparaître"),
        ("comparaitre", "comparaître"),
        ("disparaitre", "disparaître"),
        ("réapparaitre", "réapparaître"),
        ("reparaitre", "reparaître"),
        ("paraitre", "paraître"),
        ("connaitre", "connaître"),
        ("méconnaitre", "méconnaître"),
        ("reconnaitre", "reconnaître"),
        ("naitre", "naître"),
        ("renaitre", "renaître"),
        ("repaitre", "repaître"),

        ("déchainer", "déchaîner"),
        ("enchainer", "enchaîner"),
        ("entrainer", "entraîner"),
        ("trainer", "traîner"),
    
        ("couter", "coûter"),
        ("crouter", "croûter"),
        ("dégouter", "dégoûter"),
        ("envouter", "envoûter"),
        ("souler", "soûler"),
        ("gouter", "goûter"),
        ("vouter", "voûter"),

        ("déjeuner", "déjeûner"),
        ("jeuner", "jeûner"),

        ("abimer", "abîmer"),
        ("affuter", "affûter"),
        ("argüer", "arguer"),
        ("bruler", "brûler"),
        ("cloitrer", "cloîtrer"),
        ("déboiter", "déboîter"),
        ("diner", "dîner"),
        ("embucher", "embûcher"),
        ("enamourer", "énamourer"),
        ("maitriser", "maîtriser"),
        ("murir", "mûrir"),
        ("rafraichir", "rafraîchir"),
        ("refréner", "réfréner"),
        ("règlementer", "réglementer"),

        ("designer", "désigner"),  # to design vs to designate
        ("haler", "hâler"),  # to haul, tow vs to tan
        ("repartir", "répartir"),  # to leave again vs to distribute
        ("roder", "rôder"),  # to polish vs to loiter
        ("pécher", "pêcher"),  # to sin vs to fish
        ("tâcher", "tacher"),  # to try vs to soil
        ("reformer", "réformer"),  # to reform (put back together) vs to reform (improve)
        ("resigner", "résigner"),  # to sign again vs to renounce

        ("départir",),
        # This verb has two conjugations, one is older and irregular,
        # but is in modern usage giving way to a conjugation like that of finir.
        # It is hardly used except in the infinitive, past historic, and the
        # composed tenses. The third-person singular present indicative faut is
        # also found in certain set phrases.
        ("faillir",),
        ("ressortir",),
    )
)
data1 = data[~known_dupes]
data2 = data[known_dupes]
data1 = (
    data1.groupby(
        data1["infinitive"]
        .str.normalize("NFKD")
        .str.encode("ascii", errors="ignore")
        .str.decode("utf-8"),
        as_index=False,
    )
    .apply(drop_dup)
    .reset_index(drop=True)
)
data = pd.concat([data1, data2], ignore_index=True)
data.sort_values("infinitive", inplace=True)

In [6]:
data = data.reset_index(drop=True)
data

,infinitive,present participle,past participle,present 1sg,present 2sg,present 3sg,present 1pl,present 2pl,present 3pl,imperfect 1sg,imperfect 2sg,imperfect 3sg,imperfect 1pl,imperfect 2pl,imperfect 3pl,past historic 1sg,past historic 2sg,past historic 3sg,past historic 1pl,past historic 2pl,past historic 3pl,future 1sg,future 2sg,future 3sg,future 1pl,future 2pl,future 3pl,conditional 1sg,conditional 2sg,conditional 3sg,conditional 1pl,conditional 2pl,conditional 3pl,subjunctive present 1sg,subjunctive present 2sg,subjunctive present 3sg,subjunctive present 1pl,subjunctive present 2pl,subjunctive present 3pl,subjunctive imperfect 1sg,subjunctive imperfect 2sg,subjunctive imperfect 3sg,subjunctive imperfect 1pl,subjunctive imperfect 2pl,subjunctive imperfect 3pl,imperative 2sg,imperative 1pl,imperative 2pl
0,abader,abadant,abadé,abade,abades,abade,abadons,abadez,abadent,abadais,abadais,abadait,abadions,abadiez,abadaient,abadai,abadas,abada,abadâmes,abadâtes,abadèrent,abaderai,abaderas,abadera,abaderons,abaderez,abaderont,abaderais,abaderais,abaderait,abaderions,abaderiez,abaderaient,abade,abades,abade,abadions,abadiez,abadent,abadasse,abadasses,abadât,abadassions,abadassiez,abadassent,abade,abadons,abadez
1,abaisser,abaissant,abaissé,abaisse,abaisses,abaisse,abaissons,abaissez,abaissent,abaissais,abaissais,abaissait,abaissions,abaissiez,abaissaient,abaissai,abaissas,abaissa,abaissâmes,abaissâtes,abaissèrent,abaisserai,abaisseras,abaissera,abaisserons,abaisserez,abaisseront,abaisserais,abaisserais,abaisserait,abaisserions,abaisseriez,abaisseraient,abaisse,abaisses,abaisse,abaissions,abaissiez,abaissent,abaissasse,abaissasses,abaissât,abaissassions,abaissassiez,abaissassent,abaisse,abaissons,abaissez
2,abandonner,abandonnant,abandonné,abandonne,abandonnes,abandonne,abandonnons,abandonnez,abandonnent,abandonnais,abandonnais,abandonnait,abandonnions,abandonniez,abandonnaient,abandonnai,abandonnas,abandonna,abandonnâmes,abandonnâtes,abandonnèrent,abandonnerai,abandonneras,abandonnera,abandonnerons,abandonnerez,abandonneront,abandonnerais,abandonnerais,abandonnerait,abandonnerions,abandonneriez,abandonneraient,abandonne,abandonnes,abandonne,abandonnions,abandonniez,abandonnent,abandonnasse,abandonnasses,abandonnât,abandonnassions,abandonnassiez,abandonnassent,abandonne,abandonnons,abandonnez
3,abasourdir,abasourdissant,abasourdi,abasourdis,abasourdis,abasourdit,abasourdissons,abasourdissez,abasourdissent,abasourdissais,abasourdissais,abasourdissait,abasourdissions,abasourdissiez,abasourdissaient,abasourdis,abasourdis,abasourdit,abasourdîmes,abasourdîtes,abasourdirent,abasourdirai,abasourdiras,abasourdira,abasourdirons,abasourdirez,abasourdiront,abasourdirais,abasourdirais,abasourdirait,abasourdirions,abasourdiriez,abasourdiraient,abasourdisse,abasourdisses,abasourdisse,abasourdissions,abasourdissiez,abasourdissent,abasourdisse,abasourdisses,abasourdît,abasourdissions,abasourdissiez,abasourdissent,abasourdis,abasourdissons,abasourdissez
4,abattre,abattant,abattu,abats,abats,abat,abattons,abattez,abattent,abattais,abattais,abattait,abattions,abattiez,abattaient,abattis,abattis,abattit,abattîmes,abattîtes,abattirent,abattrai,abattras,abattra,abattrons,abattrez,abattront,abattrais,abattrais,abattrait,abattrions,abattriez,abattraient,abatte,abattes,abatte,abattions,abattiez,abattent,abattisse,abattisses,abattît,abattissions,abattissiez,abattissent,abats,abattons,abattez
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5332,évoluer,évoluant,évolué,évolue,évolues,évolue,évoluons,évoluez,évoluent,évoluais,évoluais,évoluait,évoluions,évoluiez,évoluaient,évoluai,évoluas,évolua,évoluâmes,évoluâtes,évoluèrent,évoluerai,évolueras,évoluera,évoluerons,évoluerez,évolueront,évoluerais,évoluerais,évoluerait,évoluerions,évolueriez,évolueraient,évolue,évolues,évolue,évoluions,évoluiez,évoluent,évoluasse,évol

## Identification of group 1 verbs

In [7]:
def maybe_add_ending(root: str, ending: str):
    if ending.startswith("*") and not root.startswith("*"):
        raise ValueError(f"{root} must return a suffix-less ending in the pattern of {ending}")
    if root.startswith("*"):
        return root.removeprefix("*")
    return f"{root}{ending}"

all_ending_patterns = []
data["group"] = None
remaining_data = data

def is_like(example_word: str, suffix: str, modify_root: Callable[[str, str | None, str], tuple[str | tuple[str], str]] | str | tuple[Callable[[str, str | None, str], str | tuple[str]], str]):
    example_root = example_word.removesuffix(suffix)
    group_example = remaining_data[remaining_data["infinitive"] == example_word].iloc[0].rename("word")
    def get_suffix(row: pd.Series):
        word = row["word"]
        column = row["index"]
        if word is None:
            return None
        if callable(modify_root):
            modified_root, _ = modify_root(example_root, None, column)
        elif isinstance(modify_root, tuple):
            modified_root = modify_root[0](example_root, None, column)
        else:
            modified_root = example_root
        if modified_root.startswith("*"):
            return modified_root
        if isinstance(word, tuple):
            if not all(w.startswith(modified_root) for w in word):
                raise ValueError(f"Unexpected value {word}, expected root {modified_root}")
            return tuple(w.removeprefix(modified_root) for w in word)
        if not word.startswith(modified_root):
            raise ValueError(f"Unexpected value {word}, expected root {modified_root}")
        return word.removeprefix(modified_root)

    group_endings = group_example.reset_index().apply(get_suffix, axis=1)
    group_endings.index = group_example.index
    display(pd.DataFrame([group_endings]))

    def get_group_label(row: pd.Series):
        infinitive = row["infinitive"]
        if not infinitive.endswith(suffix):
            return None
        root = infinitive.removesuffix(suffix)
        expected_label = None
        for column, endings in group_endings.items():
            if endings is None:
                continue
            expected = []
            if isinstance(endings, str):
                endings = (endings,)
            for ending in endings:
                if callable(modify_root):
                    alternatives, group_label = modify_root(root, ending, column)
                elif isinstance(modify_root, tuple):
                    alternatives = modify_root[0](root, ending, column)
                    group_label = modify_root[1]
                else:
                    alternatives = root
                    group_label = modify_root
                if isinstance(alternatives, str):
                    alternatives = (alternatives,)
                expected.extend(maybe_add_ending(a, ending) for a in alternatives)
            if len(expected) == 1:
                expected = expected[0]
            else:
                expected = tuple(expected)
            if row[column] is not None and row[column] != expected:
                # print(f"{infinitive} {column} is {row[column]}, expected {expected}")
                return None
            if expected_label is not None and expected_label != group_label:
                raise ValueError(f"While processing {infinitive}, {column} {row[column]} (root: {root}, ending: {ending}) returned label {group_label}, but {expected_label} expected")
            expected_label = group_label
        return expected_label
    labels = remaining_data.apply(get_group_label, axis=1)
    labels = labels[~labels.isna()]
    data.loc[labels.index, "group"] = labels
    group = remaining_data.loc[labels.index]
    group_endings["groups"] = set(labels)
    all_ending_patterns.append(group_endings)
    return group, remaining_data[~remaining_data.index.isin(group.index)]

In [8]:
always_regular = {
    "acheter",
    "cacheter",
    "celer",
    "ciseler",
    "congeler",
    "corseter",
    "craqueter",
    "crocheter",
    "duveter",
    "débequeter",
    "déceler",
    "déchiqueter",
    "décongeler",
    "déficeler",
    "dégeler",
    "démanteler",
    "déniveler",
    "ensorceler",
    "fureter",
    "geler",
    "grommeler",
    "halener",
    "haleter",
    "marteler",
    "niveler",
    "peler",
    "racheter",
    "receler",
    "regeler",
    "remodeler",
    "ruisseler",
    "surgeler",
    "tacheter",
    "écarteler",
}

always_double = {
    "appeler",
    "rappeler",
    "jeter",
    "interjeter",
    "projeter",
    "rejeter",
}

def group1_modify_root(root: str, ending: str | None, column: str) -> tuple[str | list[str], str]:
    if ending is None:
        return root, "-er"
    ending_is_open = ending.startswith("a") or ending.startswith("o") or ending.startswith("â")
    ending_is_silent = ending in ["e", "es", "ent"]
    ending_is_future = column.startswith("future") or column.startswith("conditional")
    initial_root = root
    if root.endswith("g") and not root.endswith("gg") and ending_is_open:
        root = f"{root}e"
    if root.endswith("c") and ending_is_open:
        root = f"{root.removesuffix("c")}ç"
    if re.search(r"é([bcdfghjklmnpqrstvwxz]{1,2}|qu|gu)$", initial_root):
        group_label = "-éger" if initial_root.endswith("g") else "-écer" if initial_root.endswith("c") else "-éCer"
        if ending_is_silent:
            return re.sub(r"é(.{1,2})$", r"è\1", root), group_label
        if ending_is_future:
            return (re.sub(r"é(.{1,2})$", r"è\1", root), root), group_label
        return root, group_label
    if re.search(r"e.$", root):
        if any(root.endswith(l) for l in ["ew", "ex", "ey"]):
            return root, "-e[wxy]er"
        regular_form = re.sub(r"e(.)$", r"è\1", root)
        double_form = re.sub(r"e(.)$", r"e\1\1", root)
        if not any(root.endswith(l) for l in ["et", "el"]):
            group_label = "-eger" if initial_root.endswith("g") else "-ecer" if initial_root.endswith("c") else "-e[^tlwxy]er"
            if ending_is_silent or ending_is_future:
                return regular_form, group_label
            return root, group_label
        if f"{root}er" in always_regular:
            if ending_is_silent or ending_is_future:
                    return regular_form, "-e[tl]er (regular)"
            return root, "-e[tl]er (regular)"
        if f"{root}er" in always_double:
            if ending_is_silent or ending_is_future:
                return double_form, "-e[tl]er (double)"
            return root, "-e[tl]er (double)"
        if ending_is_silent or ending_is_future:
            return (regular_form, double_form), "-e[tl]er"
        return root, "-e[tl]er"
    if any(root.endswith(l) for l in ["oy", "uy"]):
        if ending_is_silent or ending_is_future:
            return f"{root.removesuffix("y")}i", "-[ou]yer"
        return root, "-[ou]yer"
    if root.endswith("ay"):
        if ending_is_silent or ending_is_future:
            return (root, f"{root.removesuffix("y")}i"), "-ayer"
        return root, "-ayer"
    return root, "-er"
group1, remaining_data = is_like("aimer", "er", group1_modify_root)
display(group1)

,infinitive,present participle,past participle,present 1sg,present 2sg,present 3sg,present 1pl,present 2pl,present 3pl,imperfect 1sg,imperfect 2sg,imperfect 3sg,imperfect 1pl,imperfect 2pl,imperfect 3pl,past historic 1sg,past historic 2sg,past historic 3sg,past historic 1pl,past historic 2pl,past historic 3pl,future 1sg,future 2sg,future 3sg,future 1pl,future 2pl,future 3pl,conditional 1sg,conditional 2sg,conditional 3sg,conditional 1pl,conditional 2pl,conditional 3pl,subjunctive present 1sg,subjunctive present 2sg,subjunctive present 3sg,subjunctive present 1pl,subjunctive present 2pl,subjunctive present 3pl,subjunctive imperfect 1sg,subjunctive imperfect 2sg,subjunctive imperfect 3sg,subjunctive imperfect 1pl,subjunctive imperfect 2pl,subjunctive imperfect 3pl,imperative 2sg,imperative 1pl,imperative 2pl,group
0,er,ant,é,e,es,e,ons,ez,ent,ais,ais,ait,ions,iez,aient,ai,as,a,âmes,âtes,èrent,erai,eras,era,erons,erez,eront,erais,erais,erait,erions,eriez,eraient,e,es,e,ions,iez,ent,asse,asses,ât,assions,assiez,assent,e,ons,ez,None


,infinitive,present participle,past participle,present 1sg,present 2sg,present 3sg,present 1pl,present 2pl,present 3pl,imperfect 1sg,imperfect 2sg,imperfect 3sg,imperfect 1pl,imperfect 2pl,imperfect 3pl,past historic 1sg,past historic 2sg,past historic 3sg,past historic 1pl,past historic 2pl,past historic 3pl,future 1sg,future 2sg,future 3sg,future 1pl,future 2pl,future 3pl,conditional 1sg,conditional 2sg,conditional 3sg,conditional 1pl,conditional 2pl,conditional 3pl,subjunctive present 1sg,subjunctive present 2sg,subjunctive present 3sg,subjunctive present 1pl,subjunctive present 2pl,subjunctive present 3pl,subjunctive imperfect 1sg,subjunctive imperfect 2sg,subjunctive imperfect 3sg,subjunctive imperfect 1pl,subjunctive imperfect 2pl,subjunctive imperfect 3pl,imperative 2sg,imperative 1pl,imperative 2pl,group
0,abader,abadant,abadé,abade,abades,abade,abadons,abadez,abadent,abadais,abadais,abadait,abadions,abadiez,abadaient,abadai,abadas,abada,abadâmes,abadâtes,abadèrent,abaderai,abaderas,abadera,abaderons,abaderez,abaderont,abaderais,abaderais,abaderait,abaderions,abaderiez,abaderaient,abade,abades,abade,abadions,abadiez,abadent,abadasse,abadasses,abadât,abadassions,abadassiez,abadassent,abade,abadons,abadez,-er
1,abaisser,abaissant,abaissé,abaisse,abaisses,abaisse,abaissons,abaissez,abaissent,abaissais,abaissais,abaissait,abaissions,abaissiez,abaissaient,abaissai,abaissas,abaissa,abaissâmes,abaissâtes,abaissèrent,abaisserai,abaisseras,abaissera,abaisserons,abaisserez,abaisseront,abaisserais,abaisserais,abaisserait,abaisserions,abaisseriez,abaisseraient,abaisse,abaisses,abaisse,abaissions,abaissiez,abaissent,abaissasse,abaissasses,abaissât,abaissassions,abaissassiez,abaissassent,abaisse,abaissons,abaissez,-er
2,abandonner,abandonnant,abandonné,abandonne,abandonnes,abandonne,abandonnons,abandonnez,abandonnent,abandonnais,abandonnais,abandonnait,abandonnions,abandonniez,abandonnaient,abandonnai,abandonnas,abandonna,abandonnâmes,abandonnâtes,abandonnèrent,abandonnerai,abandonneras,abandonnera,abandonnerons,abandonnerez,abandonneront,abandonnerais,abandonnerais,abandonnerait,abandonnerions,abandonneriez,abandonneraient,abandonne,abandonnes,abandonne,abandonnions,abandonniez,abandonnent,abandonnasse,abandonnasses,abandonnât,abandonnassions,abandonnassiez,abandonnassent,abandonne,abandonnons,abandonnez,-er
5,abayer,abayant,abayé,"(abaye, abaie)","(abayes, abaies)","(abaye, abaie)",abayons,abayez,"(abayent, abaient)",abayais,abayais,abayait,abayions,abayiez,abayaient,abayai,abayas,abaya,abayâmes,abayâtes,abayèrent,"(abayerai, abaierai)","(abayeras, abaieras)","(abayera, abaiera)","(abayerons, abaierons)","(abayerez, abaierez)","(abayeront, abaieront)","(abayerais, abaierais)","(abayerais, abaierais)","(abayerait, abaierait)","(abayerions, abaierions)","(abayeriez, abaieriez)","(abayeraient, abaieraient)","(abaye, abaie)","(abayes, abaies)","(abaye, abaie)",abayions,abayiez,"(abayent, abaient)",abayasse,abayasses,abayât,abayassions,abayassiez,abayassent,"(abaye, abaie)",abayons,abayez,-ayer
6,abcéder,abcédant,abcédé,abcède,abcèdes,abcède,abcédons,abcédez,abcèdent,abcédais,abcédais,abcédait,abcédions,abcédiez,abcédaient,abcédai,abcédas,abcéda,abcédâmes,abcédâtes,abcédèrent,"(abcèderai, abcéderai)","(abcèderas, abcéderas)","(abcèdera, abcédera)","(abcèderons, abcéderons)","(abcèderez, abcéderez)","(abcèderont, abcéderont)","(abcèderais, abcéderais)","(abcèderais, abcéderais)","(abcèderait, abcéderait)","(abcèderions, abcéderions)","(abcèderiez, abcéderiez)","(abcèderaient, abcéderaient)",abcède,abcèdes,abcède,abcédions,abcédiez,abcèdent,abcédasse,abcédasses,abcédât,abcédassions,abcédassiez,abcédassent,abcède,abcédons,abcédez,-éCer
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5331,éviter,évitant,évité,évite,évites,évite,évitons,évitez,évitent,évitais,évitais,évitait,évitions,évitiez

## Identification of group 2 verbs

In [9]:
group2, remaining_data = is_like("réussir", "ir", "-ir")
display(group2)

,infinitive,present participle,past participle,present 1sg,present 2sg,present 3sg,present 1pl,present 2pl,present 3pl,imperfect 1sg,imperfect 2sg,imperfect 3sg,imperfect 1pl,imperfect 2pl,imperfect 3pl,past historic 1sg,past historic 2sg,past historic 3sg,past historic 1pl,past historic 2pl,past historic 3pl,future 1sg,future 2sg,future 3sg,future 1pl,future 2pl,future 3pl,conditional 1sg,conditional 2sg,conditional 3sg,conditional 1pl,conditional 2pl,conditional 3pl,subjunctive present 1sg,subjunctive present 2sg,subjunctive present 3sg,subjunctive present 1pl,subjunctive present 2pl,subjunctive present 3pl,subjunctive imperfect 1sg,subjunctive imperfect 2sg,subjunctive imperfect 3sg,subjunctive imperfect 1pl,subjunctive imperfect 2pl,subjunctive imperfect 3pl,imperative 2sg,imperative 1pl,imperative 2pl,group
0,ir,issant,i,is,is,it,issons,issez,issent,issais,issais,issait,issions,issiez,issaient,is,is,it,îmes,îtes,irent,irai,iras,ira,irons,irez,iront,irais,irais,irait,irions,iriez,iraient,isse,isses,isse,issions,issiez,issent,isse,isses,ît,issions,issiez,issent,is,issons,issez,None


,infinitive,present participle,past participle,present 1sg,present 2sg,present 3sg,present 1pl,present 2pl,present 3pl,imperfect 1sg,imperfect 2sg,imperfect 3sg,imperfect 1pl,imperfect 2pl,imperfect 3pl,past historic 1sg,past historic 2sg,past historic 3sg,past historic 1pl,past historic 2pl,past historic 3pl,future 1sg,future 2sg,future 3sg,future 1pl,future 2pl,future 3pl,conditional 1sg,conditional 2sg,conditional 3sg,conditional 1pl,conditional 2pl,conditional 3pl,subjunctive present 1sg,subjunctive present 2sg,subjunctive present 3sg,subjunctive present 1pl,subjunctive present 2pl,subjunctive present 3pl,subjunctive imperfect 1sg,subjunctive imperfect 2sg,subjunctive imperfect 3sg,subjunctive imperfect 1pl,subjunctive imperfect 2pl,subjunctive imperfect 3pl,imperative 2sg,imperative 1pl,imperative 2pl,group
3,abasourdir,abasourdissant,abasourdi,abasourdis,abasourdis,abasourdit,abasourdissons,abasourdissez,abasourdissent,abasourdissais,abasourdissais,abasourdissait,abasourdissions,abasourdissiez,abasourdissaient,abasourdis,abasourdis,abasourdit,abasourdîmes,abasourdîtes,abasourdirent,abasourdirai,abasourdiras,abasourdira,abasourdirons,abasourdirez,abasourdiront,abasourdirais,abasourdirais,abasourdirait,abasourdirions,abasourdiriez,abasourdiraient,abasourdisse,abasourdisses,abasourdisse,abasourdissions,abasourdissiez,abasourdissent,abasourdisse,abasourdisses,abasourdît,abasourdissions,abasourdissiez,abasourdissent,abasourdis,abasourdissons,abasourdissez,None
14,abolir,abolissant,aboli,abolis,abolis,abolit,abolissons,abolissez,abolissent,abolissais,abolissais,abolissait,abolissions,abolissiez,abolissaient,abolis,abolis,abolit,abolîmes,abolîtes,abolirent,abolirai,aboliras,abolira,abolirons,abolirez,aboliront,abolirais,abolirais,abolirait,abolirions,aboliriez,aboliraient,abolisse,abolisses,abolisse,abolissions,abolissiez,abolissent,abolisse,abolisses,abolît,abolissions,abolissiez,abolissent,abolis,abolissons,abolissez,None
18,abonnir,abonnissant,abonni,abonnis,abonnis,abonnit,abonnissons,abonnissez,abonnissent,abonnissais,abonnissais,abonnissait,abonnissions,abonnissiez,abonnissaient,abonnis,abonnis,abonnit,abonnîmes,abonnîtes,abonnirent,abonnirai,abonniras,abonnira,abonnirons,abonnirez,abonniront,abonnirais,abonnirais,abonnirait,abonnirions,abonniriez,abonniraient,abonnisse,abonnisses,abonnisse,abonnissions,abonnissiez,abonnissent,abonnisse,abonnisses,abonnît,abonnissions,abonnissiez,abonnissent,abonnis,abonnissons,abonnissez,None
24,aboutir,aboutissant,abouti,aboutis,aboutis,aboutit,aboutissons,aboutissez,aboutissent,aboutissais,aboutissais,aboutissait,aboutissions,aboutissiez,aboutissaient,aboutis,aboutis,aboutit,aboutîmes,aboutîtes,aboutirent,aboutirai,aboutiras,aboutira,aboutirons,aboutirez,aboutiront,aboutirais,aboutirais,aboutirait,aboutirions,aboutiriez,aboutiraient,aboutisse,aboutisses,aboutisse,aboutissions,aboutissiez,aboutissent,aboutisse,aboutisses,aboutît,aboutissions,aboutissiez,aboutissent,aboutis,aboutissons,aboutissez,None
32,abroutir,abroutissant,abrouti,abroutis,abroutis,abroutit,abroutissons,abroutissez,abroutissent,abroutissais,abroutissais,abroutissait,abroutissions,abroutissiez,abroutissaient,abroutis,abroutis,abroutit,abroutîmes,abroutîtes,abroutirent,abroutirai,abroutiras,abroutira,abroutirons,abroutirez,abroutiront,abroutirais,abroutirais,abroutirait,abroutirions,abroutiriez,abroutiraient,abroutisse,abroutisses,abroutisse,abroutissions,abroutissiez,abroutissent,abroutisse,abroutisses,abroutît,abroutissions,abroutissiez,abroutissent,abroutis,abroutissons,abroutissez,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5267,équarrir,équarrissant,équarri,équarris,équarris,équarrit,équarrissons,équarrissez,équarrissent,équarrissais,équarrissais,équarrissait,équarrissions,équarrissiez,équarrissaient,équarris,équarris,équarrit,équarrîmes,équarrîtes,équarrir

## Identification of group 3 verbs

In [10]:
def group3_modify_root(root: str, ending: str, column: str):
    if ending is None:
        return root, "-re"
    # coudre
    if root.endswith("coud"):
        if any(ending.startswith(l) for l in "aeiouyî"):
            return f"{root.removesuffix("d")}s", "-coudre"
        return root, "-coudre"
    # battre et al.
    if root.endswith("batt"):
        if column in ["present 1sg", "present 2sg", "present 3sg", "imperative 2sg"]:
            return root.removesuffix("t"), "-battre"
        return root, "-battre"
    # mettre et al.
    if root.endswith("mett"):
        if column in ["present 1sg", "present 2sg", "present 3sg", "imperative 2sg"]:
            return root.removesuffix("t"), "-mettre"
        if column == "past participle":
            return f"*{root.removesuffix("mett")}mis", "-mettre"
        if column.startswith("past historic") or column.startswith("subjunctive imperfect"):
            return root.removesuffix("ett"), "-mettre"
        return root, "-mettre"
    # vaincre et al.
    if root.endswith("c"):
        if any(ending.startswith(l) for l in "aeioî"):
            return f"{root.removesuffix("c")}qu", "-cre"
        return root, "-cre"
    return root, "-re"

group3, remaining_data = is_like("rendre", "re", group3_modify_root)
display(group3)

,infinitive,present participle,past participle,present 1sg,present 2sg,present 3sg,present 1pl,present 2pl,present 3pl,imperfect 1sg,imperfect 2sg,imperfect 3sg,imperfect 1pl,imperfect 2pl,imperfect 3pl,past historic 1sg,past historic 2sg,past historic 3sg,past historic 1pl,past historic 2pl,past historic 3pl,future 1sg,future 2sg,future 3sg,future 1pl,future 2pl,future 3pl,conditional 1sg,conditional 2sg,conditional 3sg,conditional 1pl,conditional 2pl,conditional 3pl,subjunctive present 1sg,subjunctive present 2sg,subjunctive present 3sg,subjunctive present 1pl,subjunctive present 2pl,subjunctive present 3pl,subjunctive imperfect 1sg,subjunctive imperfect 2sg,subjunctive imperfect 3sg,subjunctive imperfect 1pl,subjunctive imperfect 2pl,subjunctive imperfect 3pl,imperative 2sg,imperative 1pl,imperative 2pl,group
0,re,ant,u,s,s,,ons,ez,ent,ais,ais,ait,ions,iez,aient,is,is,it,îmes,îtes,irent,rai,ras,ra,rons,rez,ront,rais,rais,rait,rions,riez,raient,e,es,e,ions,iez,ent,isse,isses,ît,issions,issiez,issent,s,ons,ez,None


,infinitive,present participle,past participle,present 1sg,present 2sg,present 3sg,present 1pl,present 2pl,present 3pl,imperfect 1sg,imperfect 2sg,imperfect 3sg,imperfect 1pl,imperfect 2pl,imperfect 3pl,past historic 1sg,past historic 2sg,past historic 3sg,past historic 1pl,past historic 2pl,past historic 3pl,future 1sg,future 2sg,future 3sg,future 1pl,future 2pl,future 3pl,conditional 1sg,conditional 2sg,conditional 3sg,conditional 1pl,conditional 2pl,conditional 3pl,subjunctive present 1sg,subjunctive present 2sg,subjunctive present 3sg,subjunctive present 1pl,subjunctive present 2pl,subjunctive present 3pl,subjunctive imperfect 1sg,subjunctive imperfect 2sg,subjunctive imperfect 3sg,subjunctive imperfect 1pl,subjunctive imperfect 2pl,subjunctive imperfect 3pl,imperative 2sg,imperative 1pl,imperative 2pl,group
4,abattre,abattant,abattu,abats,abats,abat,abattons,abattez,abattent,abattais,abattais,abattait,abattions,abattiez,abattaient,abattis,abattis,abattit,abattîmes,abattîtes,abattirent,abattrai,abattras,abattra,abattrons,abattrez,abattront,abattrais,abattrais,abattrait,abattrions,abattriez,abattraient,abatte,abattes,abatte,abattions,abattiez,abattent,abattisse,abattisses,abattît,abattissions,abattissiez,abattissent,abats,abattons,abattez,None
38,abscondre,abscondant,abscondu,absconds,absconds,abscond,abscondons,abscondez,abscondent,abscondais,abscondais,abscondait,abscondions,abscondiez,abscondaient,abscondis,abscondis,abscondit,abscondîmes,abscondîtes,abscondirent,abscondrai,abscondras,abscondra,abscondrons,abscondrez,abscondront,abscondrais,abscondrais,abscondrait,abscondrions,abscondriez,abscondraient,absconde,abscondes,absconde,abscondions,abscondiez,abscondent,abscondisse,abscondisses,abscondît,abscondissions,abscondissiez,abscondissent,absconds,abscondons,abscondez,None
110,admettre,admettant,admis,admets,admets,admet,admettons,admettez,admettent,admettais,admettais,admettait,admettions,admettiez,admettaient,admis,admis,admit,admîmes,admîtes,admirent,admettrai,admettras,admettra,admettrons,admettrez,admettront,admettrais,admettrais,admettrait,admettrions,admettriez,admettraient,admette,admettes,admette,admettions,admettiez,admettent,admisse,admisses,admît,admissions,admissiez,admissent,admets,admettons,admettez,None
370,attendre,attendant,attendu,attends,attends,attend,attendons,attendez,attendent,attendais,attendais,attendait,attendions,attendiez,attendaient,attendis,attendis,attendit,attendîmes,attendîtes,attendirent,attendrai,attendras,attendra,attendrons,attendrez,attendront,attendrais,attendrais,attendrait,attendrions,attendriez,attendraient,attende,attendes,attende,attendions,attendiez,attendent,attendisse,attendisses,attendît,attendissions,attendissiez,attendissent,attends,attendons,attendez,None
472,battre,battant,battu,bats,bats,bat,battons,battez,battent,battais,battais,battait,battions,battiez,battaient,battis,battis,battit,battîmes,battîtes,battirent,battrai,battras,battra,battrons,battrez,battront,battrais,battrais,battrait,battrions,battriez,battraient,batte,battes,batte,battions,battiez,battent,battisse,battisses,battît,battissions,battissiez,battissent,bats,battons,battez,None
896,combattre,combattant,combattu,combats,combats,combat,combattons,combattez,combattent,combattais,combattais,combattait,combattions,combattiez,combattaient,combattis,combattis,combattit,combattîmes,combattîtes,combattirent,combattrai,combattras,combattra,combattrons,combattrez,combattront,combattrais,combattrais,combattrait,combattrions,combattriez,combattraient,combatte,combattes,combatte,combattions,combattiez,combattent,combattisse,combattisses,combattît,combattissions,combattissiez,combattissent,combats,combattons,combattez,None
905,commettre,commettant,commis,commets,commets,commet,commettons,commettez,commettent,commettais,commettais,commettait,commettions,commettiez,commettaient,commis,commis,commit,commîmes,commîtes,commirent,commettrai,commettras,commettra,commettrons,commettrez,commettront,commettrais,commettrais,com

## Irregular verbs

### -er

#### envoyer

In [11]:
def envoyer_modify_root(root: str, ending: str, column: str):
    if column.startswith("future") or column.startswith("conditional"):
        return f"{root.removesuffix("oy")}err"
    should_use_i = column in ["present 1sg", "present 2sg", "present 3sg", "present 3pl", "subjunctive present 1sg", "subjunctive present 2sg", "subjunctive present 3sg", "subjunctive present 3pl", "imperative 2sg"]
    if ending is not None:
        ending_is_silent = ending in ["e", "es", "ent"]
        assert ending_is_silent == should_use_i
    if should_use_i:
        return f"{root.removesuffix("y")}i"
    return root

like_envoyer, remaining_data = is_like("envoyer", "er", (envoyer_modify_root, "-envoyer"))
display(like_envoyer)

,infinitive,present participle,past participle,present 1sg,present 2sg,present 3sg,present 1pl,present 2pl,present 3pl,imperfect 1sg,imperfect 2sg,imperfect 3sg,imperfect 1pl,imperfect 2pl,imperfect 3pl,past historic 1sg,past historic 2sg,past historic 3sg,past historic 1pl,past historic 2pl,past historic 3pl,future 1sg,future 2sg,future 3sg,future 1pl,future 2pl,future 3pl,conditional 1sg,conditional 2sg,conditional 3sg,conditional 1pl,conditional 2pl,conditional 3pl,subjunctive present 1sg,subjunctive present 2sg,subjunctive present 3sg,subjunctive present 1pl,subjunctive present 2pl,subjunctive present 3pl,subjunctive imperfect 1sg,subjunctive imperfect 2sg,subjunctive imperfect 3sg,subjunctive imperfect 1pl,subjunctive imperfect 2pl,subjunctive imperfect 3pl,imperative 2sg,imperative 1pl,imperative 2pl,group
0,er,ant,é,e,es,e,ons,ez,ent,ais,ais,ait,ions,iez,aient,ai,as,a,âmes,âtes,èrent,ai,as,a,ons,ez,ont,ais,ais,ait,ions,iez,aient,e,es,e,ions,iez,ent,asse,asses,ât,assions,assiez,assent,e,ons,ez,None


,infinitive,present participle,past participle,present 1sg,present 2sg,present 3sg,present 1pl,present 2pl,present 3pl,imperfect 1sg,imperfect 2sg,imperfect 3sg,imperfect 1pl,imperfect 2pl,imperfect 3pl,past historic 1sg,past historic 2sg,past historic 3sg,past historic 1pl,past historic 2pl,past historic 3pl,future 1sg,future 2sg,future 3sg,future 1pl,future 2pl,future 3pl,conditional 1sg,conditional 2sg,conditional 3sg,conditional 1pl,conditional 2pl,conditional 3pl,subjunctive present 1sg,subjunctive present 2sg,subjunctive present 3sg,subjunctive present 1pl,subjunctive present 2pl,subjunctive present 3pl,subjunctive imperfect 1sg,subjunctive imperfect 2sg,subjunctive imperfect 3sg,subjunctive imperfect 1pl,subjunctive imperfect 2pl,subjunctive imperfect 3pl,imperative 2sg,imperative 1pl,imperative 2pl,group
2123,envoyer,envoyant,envoyé,envoie,envoies,envoie,envoyons,envoyez,envoient,envoyais,envoyais,envoyait,envoyions,envoyiez,envoyaient,envoyai,envoyas,envoya,envoyâmes,envoyâtes,envoyèrent,enverrai,enverras,enverra,enverrons,enverrez,enverront,enverrais,enverrais,enverrait,enverrions,enverriez,enverraient,envoie,envoies,envoie,envoyions,envoyiez,envoient,envoyasse,envoyasses,envoyât,envoyassions,envoyassiez,envoyassent,envoie,envoyons,envoyez,None
4037,renvoyer,renvoyant,renvoyé,renvoie,renvoies,renvoie,renvoyons,renvoyez,renvoient,renvoyais,renvoyais,renvoyait,renvoyions,renvoyiez,renvoyaient,renvoyai,renvoyas,renvoya,renvoyâmes,renvoyâtes,renvoyèrent,renverrai,renverras,renverra,renverrons,renverrez,renverront,renverrais,renverrais,renverrait,renverrions,renverriez,renverraient,renvoie,renvoies,renvoie,renvoyions,renvoyiez,renvoient,renvoyasse,renvoyasses,renvoyât,renvoyassions,renvoyassiez,renvoyassent,renvoie,renvoyons,renvoyez,None


### -ir

#### partir, mentir, sortir, sentir, dormir, servir...

In [12]:
def partir_modify_root(root: str, ending: str, column: str):
    if root.endswith("bouill"):
        if column in ["present 1sg", "present 2sg", "imperative 2sg"]:
            return root.removesuffix("ill")
        if column in ["present 3sg"]:
            return root.removesuffix("ill")
        return root
    if column in ["present 1sg", "present 2sg", "imperative 2sg"]:
        return root[:-1]
    if column in ["present 3sg"]:
        return root[:-1]
    return root

like_partir, remaining_data = is_like("partir", "ir", (partir_modify_root, "-ir like partir"))
display(like_partir)

,infinitive,present participle,past participle,present 1sg,present 2sg,present 3sg,present 1pl,present 2pl,present 3pl,imperfect 1sg,imperfect 2sg,imperfect 3sg,imperfect 1pl,imperfect 2pl,imperfect 3pl,past historic 1sg,past historic 2sg,past historic 3sg,past historic 1pl,past historic 2pl,past historic 3pl,future 1sg,future 2sg,future 3sg,future 1pl,future 2pl,future 3pl,conditional 1sg,conditional 2sg,conditional 3sg,conditional 1pl,conditional 2pl,conditional 3pl,subjunctive present 1sg,subjunctive present 2sg,subjunctive present 3sg,subjunctive present 1pl,subjunctive present 2pl,subjunctive present 3pl,subjunctive imperfect 1sg,subjunctive imperfect 2sg,subjunctive imperfect 3sg,subjunctive imperfect 1pl,subjunctive imperfect 2pl,subjunctive imperfect 3pl,imperative 2sg,imperative 1pl,imperative 2pl,group
0,ir,ant,i,s,s,t,ons,ez,ent,ais,ais,ait,ions,iez,aient,is,is,it,îmes,îtes,irent,irai,iras,ira,irons,irez,iront,irais,irais,irait,irions,iriez,iraient,e,es,e,ions,iez,ent,isse,isses,ît,issions,issiez,issent,s,ons,ez,None


,infinitive,present participle,past participle,present 1sg,present 2sg,present 3sg,present 1pl,present 2pl,present 3pl,imperfect 1sg,imperfect 2sg,imperfect 3sg,imperfect 1pl,imperfect 2pl,imperfect 3pl,past historic 1sg,past historic 2sg,past historic 3sg,past historic 1pl,past historic 2pl,past historic 3pl,future 1sg,future 2sg,future 3sg,future 1pl,future 2pl,future 3pl,conditional 1sg,conditional 2sg,conditional 3sg,conditional 1pl,conditional 2pl,conditional 3pl,subjunctive present 1sg,subjunctive present 2sg,subjunctive present 3sg,subjunctive present 1pl,subjunctive present 2pl,subjunctive present 3pl,subjunctive imperfect 1sg,subjunctive imperfect 2sg,subjunctive imperfect 3sg,subjunctive imperfect 1pl,subjunctive imperfect 2pl,subjunctive imperfect 3pl,imperative 2sg,imperative 1pl,imperative 2pl,group
546,bouillir,bouillant,bouilli,bous,bous,bout,bouillons,bouillez,bouillent,bouillais,bouillais,bouillait,bouillions,bouilliez,bouillaient,bouillis,bouillis,bouillit,bouillîmes,bouillîtes,bouillirent,bouillirai,bouilliras,bouillira,bouillirons,bouillirez,bouilliront,bouillirais,bouillirais,bouillirait,bouillirions,bouilliriez,bouilliraient,bouille,bouilles,bouille,bouillions,bouilliez,bouillent,bouillisse,bouillisses,bouillît,bouillissions,bouillissiez,bouillissent,bous,bouillons,bouillez,None
989,consentir,consentant,consenti,consens,consens,consent,consentons,consentez,consentent,consentais,consentais,consentait,consentions,consentiez,consentaient,consentis,consentis,consentit,consentîmes,consentîtes,consentirent,consentirai,consentiras,consentira,consentirons,consentirez,consentiront,consentirais,consentirais,consentirait,consentirions,consentiriez,consentiraient,consente,consentes,consente,consentions,consentiez,consentent,consentisse,consentisses,consentît,consentissions,consentissiez,consentissent,consens,consentons,consentez,None
1194,desservir,desservant,desservi,dessers,dessers,dessert,desservons,desservez,desservent,desservais,desservais,desservait,desservions,desserviez,desservaient,desservis,desservis,desservit,desservîmes,desservîtes,desservirent,desservirai,desserviras,desservira,desservirons,desservirez,desserviront,desservirais,desservirais,desservirait,desservirions,desserviriez,desserviraient,desserve,desserves,desserve,desservions,desserviez,desservent,desservisse,desservisses,desservît,desservissions,desservissiez,desservissent,dessers,desservons,desservez,None
1289,dormir,dormant,dormi,dors,dors,dort,dormons,dormez,dorment,dormais,dormais,dormait,dormions,dormiez,dormaient,dormis,dormis,dormit,dormîmes,dormîtes,dormirent,dormirai,dormiras,dormira,dormirons,dormirez,dormiront,dormirais,dormirais,dormirait,dormirions,dormiriez,dormiraient,dorme,dormes,dorme,dormions,dormiez,dorment,dormisse,dormisses,dormît,dormissions,dormissiez,dormissent,dors,dormons,dormez,None
1598,démentir,démentant,démenti,démens,démens,dément,démentons,démentez,démentent,démentais,démentais,démentait,démentions,démentiez,démentaient,démentis,démentis,démentit,démentîmes,démentîtes,démentirent,démentirai,démentiras,démentira,démentirons,démentirez,démentiront,démentirais,démentirais,démentirait,démentirions,démentiriez,démentiraient,démente,démentes,démente,démentions,démentiez,démentent,démentisse,démentisses,démentît,démentissions,démentissiez,démentissent,démens,démentons,démentez,None
1653,départir,départant,départi,dépars,dépars,départ,départons,départez,départent,départais,départais,départait,départions,départiez,départaient,départis,départis,départit,départîmes,départîtes,départirent,départirai,départiras,départira,départirons,départirez,départiront,départirais,départirais,départirait,départirions,départiriez,départiraient,départe,départes,départe,départions,départiez,départent,départisse,départisses,départît,départissions,départissiez,départissent,dépars,départons,départez,None
1987,endormir,endormant,endormi,endors,endors,endort,endormons,endormez,endorment,endormais,endormais,endormait,endormions,endormiez,endormaient

#### courir

In [13]:
like_courir, remaining_data = is_like("courir", "ir", "-courir")
display(like_courir)

,infinitive,present participle,past participle,present 1sg,present 2sg,present 3sg,present 1pl,present 2pl,present 3pl,imperfect 1sg,imperfect 2sg,imperfect 3sg,imperfect 1pl,imperfect 2pl,imperfect 3pl,past historic 1sg,past historic 2sg,past historic 3sg,past historic 1pl,past historic 2pl,past historic 3pl,future 1sg,future 2sg,future 3sg,future 1pl,future 2pl,future 3pl,conditional 1sg,conditional 2sg,conditional 3sg,conditional 1pl,conditional 2pl,conditional 3pl,subjunctive present 1sg,subjunctive present 2sg,subjunctive present 3sg,subjunctive present 1pl,subjunctive present 2pl,subjunctive present 3pl,subjunctive imperfect 1sg,subjunctive imperfect 2sg,subjunctive imperfect 3sg,subjunctive imperfect 1pl,subjunctive imperfect 2pl,subjunctive imperfect 3pl,imperative 2sg,imperative 1pl,imperative 2pl,group
0,ir,ant,u,s,s,t,ons,ez,ent,ais,ais,ait,ions,iez,aient,us,us,ut,ûmes,ûtes,urent,rai,ras,ra,rons,rez,ront,rais,rais,rait,rions,riez,raient,e,es,e,ions,iez,ent,usse,usses,ût,ussions,ussiez,ussent,s,ons,ez,None


,infinitive,present participle,past participle,present 1sg,present 2sg,present 3sg,present 1pl,present 2pl,present 3pl,imperfect 1sg,imperfect 2sg,imperfect 3sg,imperfect 1pl,imperfect 2pl,imperfect 3pl,past historic 1sg,past historic 2sg,past historic 3sg,past historic 1pl,past historic 2pl,past historic 3pl,future 1sg,future 2sg,future 3sg,future 1pl,future 2pl,future 3pl,conditional 1sg,conditional 2sg,conditional 3sg,conditional 1pl,conditional 2pl,conditional 3pl,subjunctive present 1sg,subjunctive present 2sg,subjunctive present 3sg,subjunctive present 1pl,subjunctive present 2pl,subjunctive present 3pl,subjunctive imperfect 1sg,subjunctive imperfect 2sg,subjunctive imperfect 3sg,subjunctive imperfect 1pl,subjunctive imperfect 2pl,subjunctive imperfect 3pl,imperative 2sg,imperative 1pl,imperative 2pl,group
70,accourir,accourant,accouru,accours,accours,accourt,accourons,accourez,accourent,accourais,accourais,accourait,accourions,accouriez,accouraient,accourus,accourus,accourut,accourûmes,accourûtes,accoururent,accourrai,accourras,accourra,accourrons,accourrez,accourront,accourrais,accourrais,accourrait,accourrions,accourriez,accourraient,accoure,accoures,accoure,accourions,accouriez,accourent,accourusse,accourusses,accourût,accourussions,accourussiez,accourussent,accours,accourons,accourez,None
952,concourir,concourant,concouru,concours,concours,concourt,concourons,concourez,concourent,concourais,concourais,concourait,concourions,concouriez,concouraient,concourus,concourus,concourut,concourûmes,concourûtes,concoururent,concourrai,concourras,concourra,concourrons,concourrez,concourront,concourrais,concourrais,concourrait,concourrions,concourriez,concourraient,concoure,concoures,concoure,concourions,concouriez,concourent,concourusse,concourusses,concourût,concourussions,concourussiez,concourussent,concours,concourons,concourez,None
1079,courir,courant,couru,cours,cours,court,courons,courez,courent,courais,courais,courait,courions,couriez,couraient,courus,courus,courut,courûmes,courûtes,coururent,courrai,courras,courra,courrons,courrez,courront,courrais,courrais,courrait,courrions,courriez,courraient,coure,coures,coure,courions,couriez,courent,courusse,courusses,courût,courussions,courussiez,courussent,cours,courons,courez,None
1234,discourir,discourant,discouru,discours,discours,discourt,discourons,discourez,discourent,discourais,discourais,discourait,discourions,discouriez,discouraient,discourus,discourus,discourut,discourûmes,discourûtes,discoururent,discourrai,discourras,discourra,discourrons,discourrez,discourront,discourrais,discourrais,discourrait,discourrions,discourriez,discourraient,discoure,discoures,discoure,discourions,discouriez,discourent,discourusse,discourusses,discourût,discourussions,discourussiez,discourussent,discours,discourons,discourez,None
1974,encourir,encourant,encouru,encours,encours,encourt,encourons,encourez,encourent,encourais,encourais,encourait,encourions,encouriez,encouraient,encourus,encourus,encourut,encourûmes,encourûtes,encoururent,encourrai,encourras,encourra,encourrons,encourrez,encourront,encourrais,encourrais,encourrait,encourrions,encourriez,encourraient,encoure,encoures,encoure,encourions,encouriez,encourent,encourusse,encourusses,encourût,encourussions,encourussiez,encourussent,encours,encourons,encourez,None
3358,parcourir,parcourant,parcouru,parcours,parcours,parcourt,parcourons,parcourez,parcourent,parcourais,parcourais,parcourait,parcourions,parcouriez,parcouraient,parcourus,parcourus,parcourut,parcourûmes,parcourûtes,parcoururent,parcourrai,parcourras,parcourra,parcourrons,parcourrez,parcourront,parcourrais,parcourrais,parcourrait,parcourrions,parcourriez,parcourraient,parcoure,parcoures,parcoure,parcourions,parcouriez,parcourent,parcourusse,parcourusses,parcourût,parcourussions,parcourussiez,parcourussent,parcours,parcourons,parcourez,None
3883,recourir,recourant,recouru,recours,recours,recourt,recourons,recourez,recourent,recourais,recourais,recourait,recourions,recouriez,r

#### vêtir

In [14]:
like_vetir, remaining_data = is_like("vêtir", "ir", "-vêtir")
display(like_vetir)

,infinitive,present participle,past participle,present 1sg,present 2sg,present 3sg,present 1pl,present 2pl,present 3pl,imperfect 1sg,imperfect 2sg,imperfect 3sg,imperfect 1pl,imperfect 2pl,imperfect 3pl,past historic 1sg,past historic 2sg,past historic 3sg,past historic 1pl,past historic 2pl,past historic 3pl,future 1sg,future 2sg,future 3sg,future 1pl,future 2pl,future 3pl,conditional 1sg,conditional 2sg,conditional 3sg,conditional 1pl,conditional 2pl,conditional 3pl,subjunctive present 1sg,subjunctive present 2sg,subjunctive present 3sg,subjunctive present 1pl,subjunctive present 2pl,subjunctive present 3pl,subjunctive imperfect 1sg,subjunctive imperfect 2sg,subjunctive imperfect 3sg,subjunctive imperfect 1pl,subjunctive imperfect 2pl,subjunctive imperfect 3pl,imperative 2sg,imperative 1pl,imperative 2pl,group
0,ir,ant,u,s,s,,ons,ez,ent,ais,ais,ait,ions,iez,aient,is,is,it,îmes,îtes,irent,irai,iras,ira,irons,irez,iront,irais,irais,irait,irions,iriez,iraient,e,es,e,ions,iez,ent,isse,isses,ît,issions,issiez,issent,s,ons,ez,None


,infinitive,present participle,past participle,present 1sg,present 2sg,present 3sg,present 1pl,present 2pl,present 3pl,imperfect 1sg,imperfect 2sg,imperfect 3sg,imperfect 1pl,imperfect 2pl,imperfect 3pl,past historic 1sg,past historic 2sg,past historic 3sg,past historic 1pl,past historic 2pl,past historic 3pl,future 1sg,future 2sg,future 3sg,future 1pl,future 2pl,future 3pl,conditional 1sg,conditional 2sg,conditional 3sg,conditional 1pl,conditional 2pl,conditional 3pl,subjunctive present 1sg,subjunctive present 2sg,subjunctive present 3sg,subjunctive present 1pl,subjunctive present 2pl,subjunctive present 3pl,subjunctive imperfect 1sg,subjunctive imperfect 2sg,subjunctive imperfect 3sg,subjunctive imperfect 1pl,subjunctive imperfect 2pl,subjunctive imperfect 3pl,imperative 2sg,imperative 1pl,imperative 2pl,group
1842,dévêtir,dévêtant,dévêtu,dévêts,dévêts,dévêt,dévêtons,dévêtez,dévêtent,dévêtais,dévêtais,dévêtait,dévêtions,dévêtiez,dévêtaient,dévêtis,dévêtis,dévêtit,dévêtîmes,dévêtîtes,dévêtirent,dévêtirai,dévêtiras,dévêtira,dévêtirons,dévêtirez,dévêtiront,dévêtirais,dévêtirais,dévêtirait,dévêtirions,dévêtiriez,dévêtiraient,dévête,dévêtes,dévête,dévêtions,dévêtiez,dévêtent,dévêtisse,dévêtisses,dévêtît,dévêtissions,dévêtissiez,dévêtissent,dévêts,dévêtons,dévêtez,None
4160,revêtir,revêtant,revêtu,revêts,revêts,revêt,revêtons,revêtez,revêtent,revêtais,revêtais,revêtait,revêtions,revêtiez,revêtaient,revêtis,revêtis,revêtit,revêtîmes,revêtîtes,revêtirent,revêtirai,revêtiras,revêtira,revêtirons,revêtirez,revêtiront,revêtirais,revêtirais,revêtirait,revêtirions,revêtiriez,revêtiraient,revête,revêtes,revête,revêtions,revêtiez,revêtent,revêtisse,revêtisses,revêtît,revêtissions,revêtissiez,revêtissent,revêts,revêtons,revêtez,None
4700,survêtir,survêtant,survêtu,survêts,survêts,survêt,survêtons,survêtez,survêtent,survêtais,survêtais,survêtait,survêtions,survêtiez,survêtaient,survêtis,survêtis,survêtit,survêtîmes,survêtîtes,survêtirent,survêtirai,survêtiras,survêtira,survêtirons,survêtirez,survêtiront,survêtirais,survêtirais,survêtirait,survêtirions,survêtiriez,survêtiraient,survête,survêtes,survête,survêtions,survêtiez,survêtent,survêtisse,survêtisses,survêtît,survêtissions,survêtissiez,survêtissent,survêts,survêtons,survêtez,None
5083,vêtir,vêtant,vêtu,vêts,vêts,vêt,vêtons,vêtez,vêtent,vêtais,vêtais,vêtait,vêtions,vêtiez,vêtaient,vêtis,vêtis,vêtit,vêtîmes,vêtîtes,vêtirent,vêtirai,vêtiras,vêtira,vêtirons,vêtirez,vêtiront,vêtirais,vêtirais,vêtirait,vêtirions,vêtiriez,vêtiraient,vête,vêtes,vête,vêtions,vêtiez,vêtent,vêtisse,vêtisses,vêtît,vêtissions,vêtissiez,vêtissent,vêts,vêtons,vêtez,None


#### cueillir

In [15]:
like_cueillir, remaining_data = is_like("cueillir", "ir", "-cueillir/-saillir")
display(like_cueillir)

,infinitive,present participle,past participle,present 1sg,present 2sg,present 3sg,present 1pl,present 2pl,present 3pl,imperfect 1sg,imperfect 2sg,imperfect 3sg,imperfect 1pl,imperfect 2pl,imperfect 3pl,past historic 1sg,past historic 2sg,past historic 3sg,past historic 1pl,past historic 2pl,past historic 3pl,future 1sg,future 2sg,future 3sg,future 1pl,future 2pl,future 3pl,conditional 1sg,conditional 2sg,conditional 3sg,conditional 1pl,conditional 2pl,conditional 3pl,subjunctive present 1sg,subjunctive present 2sg,subjunctive present 3sg,subjunctive present 1pl,subjunctive present 2pl,subjunctive present 3pl,subjunctive imperfect 1sg,subjunctive imperfect 2sg,subjunctive imperfect 3sg,subjunctive imperfect 1pl,subjunctive imperfect 2pl,subjunctive imperfect 3pl,imperative 2sg,imperative 1pl,imperative 2pl,group
0,ir,ant,i,e,es,e,ons,ez,ent,ais,ais,ait,ions,iez,aient,is,is,it,îmes,îtes,irent,"(irai, erai)","(iras, eras)","(ira, era)","(irons, erons)","(irez, erez)","(iront, eront)","(irais, erais)","(irais, erais)","(irait, erait)","(irions, erions)","(iriez, eriez)","(iraient, eraient)",e,es,e,ions,iez,ent,isse,isses,ît,issions,issiez,issent,e,ons,ez,None


,infinitive,present participle,past participle,present 1sg,present 2sg,present 3sg,present 1pl,present 2pl,present 3pl,imperfect 1sg,imperfect 2sg,imperfect 3sg,imperfect 1pl,imperfect 2pl,imperfect 3pl,past historic 1sg,past historic 2sg,past historic 3sg,past historic 1pl,past historic 2pl,past historic 3pl,future 1sg,future 2sg,future 3sg,future 1pl,future 2pl,future 3pl,conditional 1sg,conditional 2sg,conditional 3sg,conditional 1pl,conditional 2pl,conditional 3pl,subjunctive present 1sg,subjunctive present 2sg,subjunctive present 3sg,subjunctive present 1pl,subjunctive present 2pl,subjunctive present 3pl,subjunctive imperfect 1sg,subjunctive imperfect 2sg,subjunctive imperfect 3sg,subjunctive imperfect 1pl,subjunctive imperfect 2pl,subjunctive imperfect 3pl,imperative 2sg,imperative 1pl,imperative 2pl,group
77,accueillir,accueillant,accueilli,accueille,accueilles,accueille,accueillons,accueillez,accueillent,accueillais,accueillais,accueillait,accueillions,accueilliez,accueillaient,accueillis,accueillis,accueillit,accueillîmes,accueillîtes,accueillirent,"(accueillirai, accueillerai)","(accueilliras, accueilleras)","(accueillira, accueillera)","(accueillirons, accueillerons)","(accueillirez, accueillerez)","(accueilliront, accueilleront)","(accueillirais, accueillerais)","(accueillirais, accueillerais)","(accueillirait, accueillerait)","(accueillirions, accueillerions)","(accueilliriez, accueilleriez)","(accueilliraient, accueilleraient)",accueille,accueilles,accueille,accueillions,accueilliez,accueillent,accueillisse,accueillisses,accueillît,accueillissions,accueillissiez,accueillissent,accueille,accueillons,accueillez,None
335,assaillir,assaillant,assailli,assaille,assailles,assaille,assaillons,assaillez,assaillent,assaillais,assaillais,assaillait,assaillions,assailliez,assaillaient,assaillis,assaillis,assaillit,assaillîmes,assaillîtes,assaillirent,"(assaillirai, assaillerai)","(assailliras, assailleras)","(assaillira, assaillera)","(assaillirons, assaillerons)","(assaillirez, assaillerez)","(assailliront, assailleront)","(assaillirais, assaillerais)","(assaillirais, assaillerais)","(assaillirait, assaillerait)","(assaillirions, assaillerions)","(assailliriez, assailleriez)","(assailliraient, assailleraient)",assaille,assailles,assaille,assaillions,assailliez,assaillent,assaillisse,assaillisses,assaillît,assaillissions,assaillissiez,assaillissent,assaille,assaillons,assaillez,None
1145,cueillir,cueillant,cueilli,cueille,cueilles,cueille,cueillons,cueillez,cueillent,cueillais,cueillais,cueillait,cueillions,cueilliez,cueillaient,cueillis,cueillis,cueillit,cueillîmes,cueillîtes,cueillirent,"(cueillirai, cueillerai)","(cueilliras, cueilleras)","(cueillira, cueillera)","(cueillirons, cueillerons)","(cueillirez, cueillerez)","(cueilliront, cueilleront)","(cueillirais, cueillerais)","(cueillirais, cueillerais)","(cueillirait, cueillerait)","(cueillirions, cueillerions)","(cueilliriez, cueilleriez)","(cueilliraient, cueilleraient)",cueille,cueilles,cueille,cueillions,cueilliez,cueillent,cueillisse,cueillisses,cueillît,cueillissions,cueillissiez,cueillissent,cueille,cueillons,cueillez,None
3894,recueillir,recueillant,recueilli,recueille,recueilles,recueille,recueillons,recueillez,recueillent,recueillais,recueillais,recueillait,recueillions,recueilliez,recueillaient,recueillis,recueillis,recueillit,recueillîmes,recueillîtes,recueillirent,"(recueillirai, recueillerai)","(recueilliras, recueilleras)","(recueillira, recueillera)","(recueillirons, recueillerons)","(recueillirez, recueillerez)","(recueilliront, recueilleront)","(recueillirais, recueillerais)","(recueillirais, recueillerais)","(recueillirait, recueillerait)","(recueillirions, recueillerions)","(recueilliriez, recueilleriez)","(recueilliraient, recueilleraient)",recueille,recueilles,recueille,recueillions,recueilliez,recueillent,recueillisse,recueillisses,recueillît,recueillissions,recueillissiez,recueillissent,recueille,recueillons,recueillez,None
4408,saillir,saillant,s

#### défaillir

In [16]:
like_defaillir, remaining_data = is_like("défaillir", "ir", "défaillir")
display(like_defaillir)

,infinitive,present participle,past participle,present 1sg,present 2sg,present 3sg,present 1pl,present 2pl,present 3pl,imperfect 1sg,imperfect 2sg,imperfect 3sg,imperfect 1pl,imperfect 2pl,imperfect 3pl,past historic 1sg,past historic 2sg,past historic 3sg,past historic 1pl,past historic 2pl,past historic 3pl,future 1sg,future 2sg,future 3sg,future 1pl,future 2pl,future 3pl,conditional 1sg,conditional 2sg,conditional 3sg,conditional 1pl,conditional 2pl,conditional 3pl,subjunctive present 1sg,subjunctive present 2sg,subjunctive present 3sg,subjunctive present 1pl,subjunctive present 2pl,subjunctive present 3pl,subjunctive imperfect 1sg,subjunctive imperfect 2sg,subjunctive imperfect 3sg,subjunctive imperfect 1pl,subjunctive imperfect 2pl,subjunctive imperfect 3pl,imperative 2sg,imperative 1pl,imperative 2pl,group
0,ir,ant,i,e,es,e,ons,ez,ent,ais,ais,ait,ions,iez,aient,is,is,it,îmes,îtes,irent,irai,iras,ira,irons,irez,iront,irais,irais,irait,irions,iriez,iraient,e,es,e,ions,iez,ent,isse,isses,ît,issions,issiez,issent,e,ons,ez,None


,infinitive,present participle,past participle,present 1sg,present 2sg,present 3sg,present 1pl,present 2pl,present 3pl,imperfect 1sg,imperfect 2sg,imperfect 3sg,imperfect 1pl,imperfect 2pl,imperfect 3pl,past historic 1sg,past historic 2sg,past historic 3sg,past historic 1pl,past historic 2pl,past historic 3pl,future 1sg,future 2sg,future 3sg,future 1pl,future 2pl,future 3pl,conditional 1sg,conditional 2sg,conditional 3sg,conditional 1pl,conditional 2pl,conditional 3pl,subjunctive present 1sg,subjunctive present 2sg,subjunctive present 3sg,subjunctive present 1pl,subjunctive present 2pl,subjunctive present 3pl,subjunctive imperfect 1sg,subjunctive imperfect 2sg,subjunctive imperfect 3sg,subjunctive imperfect 1pl,subjunctive imperfect 2pl,subjunctive imperfect 3pl,imperative 2sg,imperative 1pl,imperative 2pl,group
1484,défaillir,défaillant,défailli,défaille,défailles,défaille,défaillons,défaillez,défaillent,défaillais,défaillais,défaillait,défaillions,défailliez,défaillaient,défaillis,défaillis,défaillit,défaillîmes,défaillîtes,défaillirent,défaillirai,défailliras,défaillira,défaillirons,défaillirez,défailliront,défaillirais,défaillirais,défaillirait,défaillirions,défailliriez,défailliraient,défaille,défailles,défaille,défaillions,défailliez,défaillent,défaillisse,défaillisses,défaillît,défaillissions,défaillissiez,défaillissent,défaille,défaillons,défaillez,None


#### ouvrir, offrir

In [17]:
def ouvrir_modify_root(root: str, ending: str, column: str):
    if column == "past participle":
        return root.removesuffix("r")
    return root

like_ouvrir, remaining_data = is_like("ouvrir", "ir", (ouvrir_modify_root, "-ouvrir/-ouffrir"))
display(like_ouvrir)

,infinitive,present participle,past participle,present 1sg,present 2sg,present 3sg,present 1pl,present 2pl,present 3pl,imperfect 1sg,imperfect 2sg,imperfect 3sg,imperfect 1pl,imperfect 2pl,imperfect 3pl,past historic 1sg,past historic 2sg,past historic 3sg,past historic 1pl,past historic 2pl,past historic 3pl,future 1sg,future 2sg,future 3sg,future 1pl,future 2pl,future 3pl,conditional 1sg,conditional 2sg,conditional 3sg,conditional 1pl,conditional 2pl,conditional 3pl,subjunctive present 1sg,subjunctive present 2sg,subjunctive present 3sg,subjunctive present 1pl,subjunctive present 2pl,subjunctive present 3pl,subjunctive imperfect 1sg,subjunctive imperfect 2sg,subjunctive imperfect 3sg,subjunctive imperfect 1pl,subjunctive imperfect 2pl,subjunctive imperfect 3pl,imperative 2sg,imperative 1pl,imperative 2pl,group
0,ir,ant,ert,e,es,e,ons,ez,ent,ais,ais,ait,ions,iez,aient,is,is,it,îmes,îtes,irent,irai,iras,ira,irons,irez,iront,irais,irais,irait,irions,iriez,iraient,e,es,e,ions,iez,ent,isse,isses,ît,issions,issiez,issent,e,ons,ez,None


,infinitive,present participle,past participle,present 1sg,present 2sg,present 3sg,present 1pl,present 2pl,present 3pl,imperfect 1sg,imperfect 2sg,imperfect 3sg,imperfect 1pl,imperfect 2pl,imperfect 3pl,past historic 1sg,past historic 2sg,past historic 3sg,past historic 1pl,past historic 2pl,past historic 3pl,future 1sg,future 2sg,future 3sg,future 1pl,future 2pl,future 3pl,conditional 1sg,conditional 2sg,conditional 3sg,conditional 1pl,conditional 2pl,conditional 3pl,subjunctive present 1sg,subjunctive present 2sg,subjunctive present 3sg,subjunctive present 1pl,subjunctive present 2pl,subjunctive present 3pl,subjunctive imperfect 1sg,subjunctive imperfect 2sg,subjunctive imperfect 3sg,subjunctive imperfect 1pl,subjunctive imperfect 2pl,subjunctive imperfect 3pl,imperative 2sg,imperative 1pl,imperative 2pl,group
1090,couvrir,couvrant,couvert,couvre,couvres,couvre,couvrons,couvrez,couvrent,couvrais,couvrais,couvrait,couvrions,couvriez,couvraient,couvris,couvris,couvrit,couvrîmes,couvrîtes,couvrirent,couvrirai,couvriras,couvrira,couvrirons,couvrirez,couvriront,couvrirais,couvrirais,couvrirait,couvririons,couvririez,couvriraient,couvre,couvres,couvre,couvrions,couvriez,couvrent,couvrisse,couvrisses,couvrît,couvrissions,couvrissiez,couvrissent,couvre,couvrons,couvrez,None
1454,découvrir,découvrant,découvert,découvre,découvres,découvre,découvrons,découvrez,découvrent,découvrais,découvrais,découvrait,découvrions,découvriez,découvraient,découvris,découvris,découvrit,découvrîmes,découvrîtes,découvrirent,découvrirai,découvriras,découvrira,découvrirons,découvrirez,découvriront,découvrirais,découvrirais,découvrirait,découvririons,découvririez,découvriraient,découvre,découvres,découvre,découvrions,découvriez,découvrent,découvrisse,découvrisses,découvrît,découvrissions,découvrissiez,découvrissent,découvre,découvrons,découvrez,None
2108,entrouvrir,entrouvrant,entrouvert,entrouvre,entrouvres,entrouvre,entrouvrons,entrouvrez,entrouvrent,entrouvrais,entrouvrais,entrouvrait,entrouvrions,entrouvriez,entrouvraient,entrouvris,entrouvris,entrouvrit,entrouvrîmes,entrouvrîtes,entrouvrirent,entrouvrirai,entrouvriras,entrouvrira,entrouvrirons,entrouvrirez,entrouvriront,entrouvrirais,entrouvrirais,entrouvrirait,entrouvririons,entrouvririez,entrouvriraient,entrouvre,entrouvres,entrouvre,entrouvrions,entrouvriez,entrouvrent,entrouvrisse,entrouvrisses,entrouvrît,entrouvrissions,entrouvrissiez,entrouvrissent,entrouvre,entrouvrons,entrouvrez,None
3278,offrir,offrant,offert,offre,offres,offre,offrons,offrez,offrent,offrais,offrais,offrait,offrions,offriez,offraient,offris,offris,offrit,offrîmes,offrîtes,offrirent,offrirai,offriras,offrira,offrirons,offrirez,offriront,offrirais,offrirais,offrirait,offririons,offririez,offriraient,offre,offres,offre,offrions,offriez,offrent,offrisse,offrisses,offrît,offrissions,offrissiez,offrissent,offre,offrons,offrez,None
3318,ouvrir,ouvrant,ouvert,ouvre,ouvres,ouvre,ouvrons,ouvrez,ouvrent,ouvrais,ouvrais,ouvrait,ouvrions,ouvriez,ouvraient,ouvris,ouvris,ouvrit,ouvrîmes,ouvrîtes,ouvrirent,ouvrirai,ouvriras,ouvrira,ouvrirons,ouvrirez,ouvriront,ouvrirais,ouvrirais,ouvrirait,ouvririons,ouvririez,ouvriraient,ouvre,ouvres,ouvre,ouvrions,ouvriez,ouvrent,ouvrisse,ouvrisses,ouvrît,ouvrissions,ouvrissiez,ouvrissent,ouvre,ouvrons,ouvrez,None
3885,recouvrir,recouvrant,recouvert,recouvre,recouvres,recouvre,recouvrons,recouvrez,recouvrent,recouvrais,recouvrais,recouvrait,recouvrions,recouvriez,recouvraient,recouvris,recouvris,recouvrit,recouvrîmes,recouvrîtes,recouvrirent,recouvrirai,recouvriras,recouvrira,recouvrirons,recouvrirez,recouvriront,recouvrirais,recouvrirais,recouvrirait,recouvririons,recouvririez,recouvriraient,recouvre,recouvres,recouvre,recouvrions,recouvriez,recouvrent,recouvrisse,recouvrisses,recouvrît,recouvrissions,recouvrissiez,recouvrissent,recouvre,recouvrons,recouvrez,None
3919,redécouvrir,redécouvrant,redécouvert,redécouvre,redécouvres,redécouvre,redécouvrons,redécouvrez,redécouvrent,redécouvrais,redécouvrais,redéco

### -oir

#### recevoir, apercevoir...

In [18]:
def recevoir_modify_root(root: str, ending: str, column: str):
    should_use_cedille = column in ["past participle", "present 1sg", "present 2sg", "present 3sg", "present 3pl", "subjunctive present 1sg", "subjunctive present 2sg", "subjunctive present 3sg", "subjunctive present 3pl", "imperative 2sg"] or column.startswith("past historic") or column.startswith("subjunctive imperfect")
    if ending is not None:
        ending_starts_with_open_vowel = any(ending.startswith(l) for l in "aouâû")
        assert should_use_cedille == ending_starts_with_open_vowel, f"{ending} {should_use_cedille}"
    if should_use_cedille:
        return f"{root.removesuffix("c")}ç"
    return root

like_recevoir, remaining_data = is_like("recevoir", "evoir", (recevoir_modify_root, "-cevoir"))
display(like_recevoir)

,infinitive,present participle,past participle,present 1sg,present 2sg,present 3sg,present 1pl,present 2pl,present 3pl,imperfect 1sg,imperfect 2sg,imperfect 3sg,imperfect 1pl,imperfect 2pl,imperfect 3pl,past historic 1sg,past historic 2sg,past historic 3sg,past historic 1pl,past historic 2pl,past historic 3pl,future 1sg,future 2sg,future 3sg,future 1pl,future 2pl,future 3pl,conditional 1sg,conditional 2sg,conditional 3sg,conditional 1pl,conditional 2pl,conditional 3pl,subjunctive present 1sg,subjunctive present 2sg,subjunctive present 3sg,subjunctive present 1pl,subjunctive present 2pl,subjunctive present 3pl,subjunctive imperfect 1sg,subjunctive imperfect 2sg,subjunctive imperfect 3sg,subjunctive imperfect 1pl,subjunctive imperfect 2pl,subjunctive imperfect 3pl,imperative 2sg,imperative 1pl,imperative 2pl,group
0,evoir,evant,u,ois,ois,oit,evons,evez,oivent,evais,evais,evait,evions,eviez,evaient,us,us,ut,ûmes,ûtes,urent,evrai,evras,evra,evrons,evrez,evront,evrais,evrais,evrait,evrions,evriez,evraient,oive,oives,oive,evions,eviez,oivent,usse,usses,ût,ussions,ussiez,ussent,ois,evons,evez,None


,infinitive,present participle,past participle,present 1sg,present 2sg,present 3sg,present 1pl,present 2pl,present 3pl,imperfect 1sg,imperfect 2sg,imperfect 3sg,imperfect 1pl,imperfect 2pl,imperfect 3pl,past historic 1sg,past historic 2sg,past historic 3sg,past historic 1pl,past historic 2pl,past historic 3pl,future 1sg,future 2sg,future 3sg,future 1pl,future 2pl,future 3pl,conditional 1sg,conditional 2sg,conditional 3sg,conditional 1pl,conditional 2pl,conditional 3pl,subjunctive present 1sg,subjunctive present 2sg,subjunctive present 3sg,subjunctive present 1pl,subjunctive present 2pl,subjunctive present 3pl,subjunctive imperfect 1sg,subjunctive imperfect 2sg,subjunctive imperfect 3sg,subjunctive imperfect 1pl,subjunctive imperfect 2pl,subjunctive imperfect 3pl,imperative 2sg,imperative 1pl,imperative 2pl,group
265,apercevoir,apercevant,aperçu,aperçois,aperçois,aperçoit,apercevons,apercevez,aperçoivent,apercevais,apercevais,apercevait,apercevions,aperceviez,apercevaient,aperçus,aperçus,aperçut,aperçûmes,aperçûtes,aperçurent,apercevrai,apercevras,apercevra,apercevrons,apercevrez,apercevront,apercevrais,apercevrais,apercevrait,apercevrions,apercevriez,apercevraient,aperçoive,aperçoives,aperçoive,apercevions,aperceviez,aperçoivent,aperçusse,aperçusses,aperçût,aperçussions,aperçussiez,aperçussent,aperçois,apercevons,apercevez,None
947,concevoir,concevant,conçu,conçois,conçois,conçoit,concevons,concevez,conçoivent,concevais,concevais,concevait,concevions,conceviez,concevaient,conçus,conçus,conçut,conçûmes,conçûtes,conçurent,concevrai,concevras,concevra,concevrons,concevrez,concevront,concevrais,concevrais,concevrait,concevrions,concevriez,concevraient,conçoive,conçoives,conçoive,concevions,conceviez,conçoivent,conçusse,conçusses,conçût,conçussions,conçussiez,conçussent,conçois,concevons,concevez,None
1391,décevoir,décevant,déçu,déçois,déçois,déçoit,décevons,décevez,déçoivent,décevais,décevais,décevait,décevions,déceviez,décevaient,déçus,déçus,déçut,déçûmes,déçûtes,déçurent,décevrai,décevras,décevra,décevrons,décevrez,décevront,décevrais,décevrais,décevrait,décevrions,décevriez,décevraient,déçoive,déçoives,déçoive,décevions,déceviez,déçoivent,déçusse,déçusses,déçût,déçussions,déçussiez,déçussent,déçois,décevons,décevez,None
2091,entrapercevoir,entrapercevant,entraperçu,entraperçois,entraperçois,entraperçoit,entrapercevons,entrapercevez,entraperçoivent,entrapercevais,entrapercevais,entrapercevait,entrapercevions,entraperceviez,entrapercevaient,entraperçus,entraperçus,entraperçut,entraperçûmes,entraperçûtes,entraperçurent,entrapercevrai,entrapercevras,entrapercevra,entrapercevrons,entrapercevrez,entrapercevront,entrapercevrais,entrapercevrais,entrapercevrait,entrapercevrions,entrapercevriez,entrapercevraient,entraperçoive,entraperçoives,entraperçoive,entrapercevions,entraperceviez,entraperçoivent,entraperçusse,entraperçusses,entraperçût,entraperçussions,entraperçussiez,entraperçussent,entraperçois,entrapercevons,entrapercevez,None
3414,percevoir,percevant,perçu,perçois,perçois,perçoit,percevons,percevez,perçoivent,percevais,percevais,percevait,percevions,perceviez,percevaient,perçus,perçus,perçut,perçûmes,perçûtes,perçurent,percevrai,percevras,percevra,percevrons,percevrez,percevront,percevrais,percevrais,percevrait,percevrions,percevriez,percevraient,perçoive,perçoives,perçoive,percevions,perceviez,perçoivent,perçusse,perçusses,perçût,perçussions,perçussiez,perçussent,perçois,percevons,percevez,None
3852,recevoir,recevant,reçu,reçois,reçois,reçoit,recevons,recevez,reçoivent,recevais,recevais,recevait,recevions,receviez,recevaient,reçus,reçus,reçut,reçûmes,reçûtes,reçurent,recevrai,recevras,recevra,recevrons,recevrez,recevront,recevrais,recevrais,recevrait,recevrions,recevriez,recevraient,reçoive,reçoives,reçoive,recevions,receviez,reçoivent,reçusse,reçusses,reçût,reçussions,reçussiez,reçussent,reçois,recevons,recevez,None
4055,repercevoir,repercevant,reperçu,reperçois,reperçois,reperçoit,repercevons,repercevez,reperçoivent,reperceva

### choir/déchoir/échoir

In [19]:
like_choir, remaining_data = is_like("choir", "oir", "choir")
display(like_choir)

,infinitive,present participle,past participle,present 1sg,present 2sg,present 3sg,present 1pl,present 2pl,present 3pl,imperfect 1sg,imperfect 2sg,imperfect 3sg,imperfect 1pl,imperfect 2pl,imperfect 3pl,past historic 1sg,past historic 2sg,past historic 3sg,past historic 1pl,past historic 2pl,past historic 3pl,future 1sg,future 2sg,future 3sg,future 1pl,future 2pl,future 3pl,conditional 1sg,conditional 2sg,conditional 3sg,conditional 1pl,conditional 2pl,conditional 3pl,subjunctive present 1sg,subjunctive present 2sg,subjunctive present 3sg,subjunctive present 1pl,subjunctive present 2pl,subjunctive present 3pl,subjunctive imperfect 1sg,subjunctive imperfect 2sg,subjunctive imperfect 3sg,subjunctive imperfect 1pl,subjunctive imperfect 2pl,subjunctive imperfect 3pl,imperative 2sg,imperative 1pl,imperative 2pl,group
0,oir,None,u,ois,ois,oit,oyons,oyez,oient,None,None,None,None,None,None,us,us,ut,ûmes,ûtes,urent,"(oirai, errai)","(oiras, erras)","(oira, erra)","(oirons, errons)","(oirez, errez)","(oiront, erront)",oirais,oirais,oirait,oirions,oiriez,oiraient,None,None,None,None,None,None,None,None,ût,None,None,None,None,None,None,None


,infinitive,present participle,past participle,present 1sg,present 2sg,present 3sg,present 1pl,present 2pl,present 3pl,imperfect 1sg,imperfect 2sg,imperfect 3sg,imperfect 1pl,imperfect 2pl,imperfect 3pl,past historic 1sg,past historic 2sg,past historic 3sg,past historic 1pl,past historic 2pl,past historic 3pl,future 1sg,future 2sg,future 3sg,future 1pl,future 2pl,future 3pl,conditional 1sg,conditional 2sg,conditional 3sg,conditional 1pl,conditional 2pl,conditional 3pl,subjunctive present 1sg,subjunctive present 2sg,subjunctive present 3sg,subjunctive present 1pl,subjunctive present 2pl,subjunctive present 3pl,subjunctive imperfect 1sg,subjunctive imperfect 2sg,subjunctive imperfect 3sg,subjunctive imperfect 1pl,subjunctive imperfect 2pl,subjunctive imperfect 3pl,imperative 2sg,imperative 1pl,imperative 2pl,group
790,choir,None,chu,chois,chois,choit,choyons,choyez,choient,None,None,None,None,None,None,chus,chus,chut,chûmes,chûtes,churent,"(choirai, cherrai)","(choiras, cherras)","(choira, cherra)","(choirons, cherrons)","(choirez, cherrez)","(choiront, cherront)",choirais,choirais,choirait,choirions,choiriez,choiraient,None,None,None,None,None,None,None,None,chût,None,None,None,None,None,None,None


In [20]:
like_echoir, remaining_data = is_like("échoir", "oir", "échoir")
display(like_echoir)

,infinitive,present participle,past participle,present 1sg,present 2sg,present 3sg,present 1pl,present 2pl,present 3pl,imperfect 1sg,imperfect 2sg,imperfect 3sg,imperfect 1pl,imperfect 2pl,imperfect 3pl,past historic 1sg,past historic 2sg,past historic 3sg,past historic 1pl,past historic 2pl,past historic 3pl,future 1sg,future 2sg,future 3sg,future 1pl,future 2pl,future 3pl,conditional 1sg,conditional 2sg,conditional 3sg,conditional 1pl,conditional 2pl,conditional 3pl,subjunctive present 1sg,subjunctive present 2sg,subjunctive present 3sg,subjunctive present 1pl,subjunctive present 2pl,subjunctive present 3pl,subjunctive imperfect 1sg,subjunctive imperfect 2sg,subjunctive imperfect 3sg,subjunctive imperfect 1pl,subjunctive imperfect 2pl,subjunctive imperfect 3pl,imperative 2sg,imperative 1pl,imperative 2pl,group
0,oir,éant,u,None,None,"(oit, et)",None,None,"(oient, ettent)",None,None,oyait,None,None,oyaient,None,None,ut,None,None,urent,None,None,oira,None,None,oiront,None,None,oirait,None,None,oiraient,None,None,oie,None,None,oient,None,None,ût,None,None,ussent,None,None,None,None


,infinitive,present participle,past participle,present 1sg,present 2sg,present 3sg,present 1pl,present 2pl,present 3pl,imperfect 1sg,imperfect 2sg,imperfect 3sg,imperfect 1pl,imperfect 2pl,imperfect 3pl,past historic 1sg,past historic 2sg,past historic 3sg,past historic 1pl,past historic 2pl,past historic 3pl,future 1sg,future 2sg,future 3sg,future 1pl,future 2pl,future 3pl,conditional 1sg,conditional 2sg,conditional 3sg,conditional 1pl,conditional 2pl,conditional 3pl,subjunctive present 1sg,subjunctive present 2sg,subjunctive present 3sg,subjunctive present 1pl,subjunctive present 2pl,subjunctive present 3pl,subjunctive imperfect 1sg,subjunctive imperfect 2sg,subjunctive imperfect 3sg,subjunctive imperfect 1pl,subjunctive imperfect 2pl,subjunctive imperfect 3pl,imperative 2sg,imperative 1pl,imperative 2pl,group
5142,échoir,échéant,échu,None,None,"(échoit, échet)",None,None,"(échoient, échettent)",None,None,échoyait,None,None,échoyaient,None,None,échut,None,None,échurent,None,None,échoira,None,None,échoiront,None,None,échoirait,None,None,échoiraient,None,None,échoie,None,None,échoient,None,None,échût,None,None,échussent,None,None,None,None


In [21]:
like_dechoir, remaining_data = is_like("déchoir", "oir", "déchoir")
display(like_dechoir)

,infinitive,present participle,past participle,present 1sg,present 2sg,present 3sg,present 1pl,present 2pl,present 3pl,imperfect 1sg,imperfect 2sg,imperfect 3sg,imperfect 1pl,imperfect 2pl,imperfect 3pl,past historic 1sg,past historic 2sg,past historic 3sg,past historic 1pl,past historic 2pl,past historic 3pl,future 1sg,future 2sg,future 3sg,future 1pl,future 2pl,future 3pl,conditional 1sg,conditional 2sg,conditional 3sg,conditional 1pl,conditional 2pl,conditional 3pl,subjunctive present 1sg,subjunctive present 2sg,subjunctive present 3sg,subjunctive present 1pl,subjunctive present 2pl,subjunctive present 3pl,subjunctive imperfect 1sg,subjunctive imperfect 2sg,subjunctive imperfect 3sg,subjunctive imperfect 1pl,subjunctive imperfect 2pl,subjunctive imperfect 3pl,imperative 2sg,imperative 1pl,imperative 2pl,group
0,oir,None,u,ois,ois,"(oit, et)",oyons,oyez,oient,None,None,None,None,None,None,us,us,ut,ûmes,ûtes,urent,"(oirai, errai)","(oiras, erras)","(oira, erra)","(oirons, errons)","(oirez, errez)","(oiront, erront)","(oirais, errais)","(oirais, errais)","(oirait, errait)","(oirions, errions)","(oiriez, erriez)","(oiraient, erraient)",oie,oies,oie,oyions,oyiez,oient,usse,usses,ût,ussions,ussiez,ussent,None,None,None,None


,infinitive,present participle,past participle,present 1sg,present 2sg,present 3sg,present 1pl,present 2pl,present 3pl,imperfect 1sg,imperfect 2sg,imperfect 3sg,imperfect 1pl,imperfect 2pl,imperfect 3pl,past historic 1sg,past historic 2sg,past historic 3sg,past historic 1pl,past historic 2pl,past historic 3pl,future 1sg,future 2sg,future 3sg,future 1pl,future 2pl,future 3pl,conditional 1sg,conditional 2sg,conditional 3sg,conditional 1pl,conditional 2pl,conditional 3pl,subjunctive present 1sg,subjunctive present 2sg,subjunctive present 3sg,subjunctive present 1pl,subjunctive present 2pl,subjunctive present 3pl,subjunctive imperfect 1sg,subjunctive imperfect 2sg,subjunctive imperfect 3sg,subjunctive imperfect 1pl,subjunctive imperfect 2pl,subjunctive imperfect 3pl,imperative 2sg,imperative 1pl,imperative 2pl,group
1401,déchoir,None,déchu,déchois,déchois,"(déchoit, déchet)",déchoyons,déchoyez,déchoient,None,None,None,None,None,None,déchus,déchus,déchut,déchûmes,déchûtes,déchurent,"(déchoirai, décherrai)","(déchoiras, décherras)","(déchoira, décherra)","(déchoirons, décherrons)","(déchoirez, décherrez)","(déchoiront, décherront)","(déchoirais, décherrais)","(déchoirais, décherrais)","(déchoirait, décherrait)","(déchoirions, décherrions)","(déchoiriez, décherriez)","(déchoiraient, décherraient)",déchoie,déchoies,déchoie,déchoyions,déchoyiez,déchoient,déchusse,déchusses,déchût,déchussions,déchussiez,déchussent,None,None,None,None


### -re

#### conduire, introduire...

In [22]:
like_conduire, remaining_data = is_like("conduire", "ire", "-uire")
display(like_conduire)

,infinitive,present participle,past participle,present 1sg,present 2sg,present 3sg,present 1pl,present 2pl,present 3pl,imperfect 1sg,imperfect 2sg,imperfect 3sg,imperfect 1pl,imperfect 2pl,imperfect 3pl,past historic 1sg,past historic 2sg,past historic 3sg,past historic 1pl,past historic 2pl,past historic 3pl,future 1sg,future 2sg,future 3sg,future 1pl,future 2pl,future 3pl,conditional 1sg,conditional 2sg,conditional 3sg,conditional 1pl,conditional 2pl,conditional 3pl,subjunctive present 1sg,subjunctive present 2sg,subjunctive present 3sg,subjunctive present 1pl,subjunctive present 2pl,subjunctive present 3pl,subjunctive imperfect 1sg,subjunctive imperfect 2sg,subjunctive imperfect 3sg,subjunctive imperfect 1pl,subjunctive imperfect 2pl,subjunctive imperfect 3pl,imperative 2sg,imperative 1pl,imperative 2pl,group
0,ire,isant,it,is,is,it,isons,isez,isent,isais,isais,isait,isions,isiez,isaient,isis,isis,isit,isîmes,isîtes,isirent,irai,iras,ira,irons,irez,iront,irais,irais,irait,irions,iriez,iraient,ise,ises,ise,isions,isiez,isent,isisse,isisses,isît,isissions,isissiez,isissent,is,isons,isez,None


,infinitive,present participle,past participle,present 1sg,present 2sg,present 3sg,present 1pl,present 2pl,present 3pl,imperfect 1sg,imperfect 2sg,imperfect 3sg,imperfect 1pl,imperfect 2pl,imperfect 3pl,past historic 1sg,past historic 2sg,past historic 3sg,past historic 1pl,past historic 2pl,past historic 3pl,future 1sg,future 2sg,future 3sg,future 1pl,future 2pl,future 3pl,conditional 1sg,conditional 2sg,conditional 3sg,conditional 1pl,conditional 2pl,conditional 3pl,subjunctive present 1sg,subjunctive present 2sg,subjunctive present 3sg,subjunctive present 1pl,subjunctive present 2pl,subjunctive present 3pl,subjunctive imperfect 1sg,subjunctive imperfect 2sg,subjunctive imperfect 3sg,subjunctive imperfect 1pl,subjunctive imperfect 2pl,subjunctive imperfect 3pl,imperative 2sg,imperative 1pl,imperative 2pl,group
960,conduire,conduisant,conduit,conduis,conduis,conduit,conduisons,conduisez,conduisent,conduisais,conduisais,conduisait,conduisions,conduisiez,conduisaient,conduisis,conduisis,conduisit,conduisîmes,conduisîtes,conduisirent,conduirai,conduiras,conduira,conduirons,conduirez,conduiront,conduirais,conduirais,conduirait,conduirions,conduiriez,conduiraient,conduise,conduises,conduise,conduisions,conduisiez,conduisent,conduisisse,conduisisses,conduisît,conduisissions,conduisissiez,conduisissent,conduis,conduisons,conduisez,None
1003,construire,construisant,construit,construis,construis,construit,construisons,construisez,construisent,construisais,construisais,construisait,construisions,construisiez,construisaient,construisis,construisis,construisit,construisîmes,construisîtes,construisirent,construirai,construiras,construira,construirons,construirez,construiront,construirais,construirais,construirait,construirions,construiriez,construiraient,construise,construises,construise,construisions,construisiez,construisent,construisisse,construisisses,construisît,construisissions,construisissiez,construisissent,construis,construisons,construisez,None
1147,cuire,cuisant,cuit,cuis,cuis,cuit,cuisons,cuisez,cuisent,cuisais,cuisais,cuisait,cuisions,cuisiez,cuisaient,cuisis,cuisis,cuisit,cuisîmes,cuisîtes,cuisirent,cuirai,cuiras,cuira,cuirons,cuirez,cuiront,cuirais,cuirais,cuirait,cuirions,cuiriez,cuiraient,cuise,cuises,cuise,cuisions,cuisiez,cuisent,cuisisse,cuisisses,cuisît,cuisissions,cuisissiez,cuisissent,cuis,cuisons,cuisez,None
1440,déconstruire,déconstruisant,déconstruit,déconstruis,déconstruis,déconstruit,déconstruisons,déconstruisez,déconstruisent,déconstruisais,déconstruisais,déconstruisait,déconstruisions,déconstruisiez,déconstruisaient,déconstruisis,déconstruisis,déconstruisit,déconstruisîmes,déconstruisîtes,déconstruisirent,déconstruirai,déconstruiras,déconstruira,déconstruirons,déconstruirez,déconstruiront,déconstruirais,déconstruirais,déconstruirait,déconstruirions,déconstruiriez,déconstruiraient,déconstruise,déconstruises,déconstruise,déconstruisions,déconstruisiez,déconstruisent,déconstruisisse,déconstruisisses,déconstruisît,déconstruisissions,déconstruisissiez,déconstruisissent,déconstruis,déconstruisons,déconstruisez,None
1467,décuire,décuisant,décuit,décuis,décuis,décuit,décuisons,décuisez,décuisent,décuisais,décuisais,décuisait,décuisions,décuisiez,décuisaient,décuisis,décuisis,décuisit,décuisîmes,décuisîtes,décuisirent,décuirai,décuiras,décuira,décuirons,décuirez,décuiront,décuirais,décuirais,décuirait,décuirions,décuiriez,décuiraient,décuise,décuises,décuise,décuisions,décuisiez,décuisent,décuisisse,décuisisses,décuisît,décuisissions,décuisissiez,décuisissent,décuis,décuisons,décuisez,None
1483,déduire,déduisant,déduit,déduis,déduis,déduit,déduisons,déduisez,déduisent,déduisais,déduisais,déduisait,déduisions,déduisiez,déduisaient,déduisis,déduisis,déduisit,déduisîmes,déduisîtes,déduisirent,déduirai,déduiras,déduira,déduirons,déduirez,déduiront,déduirais,déduirais,déduirait,déduirions,déduiriez,déduiraient,déduise,déduises,déduise,déduisions,déduisiez,déduisent,déduisisse,déduisisses,déduisît,déduisissions,déduisissiez

#### écrire

In [23]:
like_ecrire, remaining_data = is_like("écrire", "re", "-crire")
display(like_ecrire)

,infinitive,present participle,past participle,present 1sg,present 2sg,present 3sg,present 1pl,present 2pl,present 3pl,imperfect 1sg,imperfect 2sg,imperfect 3sg,imperfect 1pl,imperfect 2pl,imperfect 3pl,past historic 1sg,past historic 2sg,past historic 3sg,past historic 1pl,past historic 2pl,past historic 3pl,future 1sg,future 2sg,future 3sg,future 1pl,future 2pl,future 3pl,conditional 1sg,conditional 2sg,conditional 3sg,conditional 1pl,conditional 2pl,conditional 3pl,subjunctive present 1sg,subjunctive present 2sg,subjunctive present 3sg,subjunctive present 1pl,subjunctive present 2pl,subjunctive present 3pl,subjunctive imperfect 1sg,subjunctive imperfect 2sg,subjunctive imperfect 3sg,subjunctive imperfect 1pl,subjunctive imperfect 2pl,subjunctive imperfect 3pl,imperative 2sg,imperative 1pl,imperative 2pl,group
0,re,vant,t,s,s,t,vons,vez,vent,vais,vais,vait,vions,viez,vaient,vis,vis,vit,vîmes,vîtes,virent,rai,ras,ra,rons,rez,ront,rais,rais,rait,rions,riez,raient,ve,ves,ve,vions,viez,vent,visse,visses,vît,vissions,vissiez,vissent,s,vons,vez,None


,infinitive,present participle,past participle,present 1sg,present 2sg,present 3sg,present 1pl,present 2pl,present 3pl,imperfect 1sg,imperfect 2sg,imperfect 3sg,imperfect 1pl,imperfect 2pl,imperfect 3pl,past historic 1sg,past historic 2sg,past historic 3sg,past historic 1pl,past historic 2pl,past historic 3pl,future 1sg,future 2sg,future 3sg,future 1pl,future 2pl,future 3pl,conditional 1sg,conditional 2sg,conditional 3sg,conditional 1pl,conditional 2pl,conditional 3pl,subjunctive present 1sg,subjunctive present 2sg,subjunctive present 3sg,subjunctive present 1pl,subjunctive present 2pl,subjunctive present 3pl,subjunctive imperfect 1sg,subjunctive imperfect 2sg,subjunctive imperfect 3sg,subjunctive imperfect 1pl,subjunctive imperfect 2pl,subjunctive imperfect 3pl,imperative 2sg,imperative 1pl,imperative 2pl,group
818,circonscrire,circonscrivant,circonscrit,circonscris,circonscris,circonscrit,circonscrivons,circonscrivez,circonscrivent,circonscrivais,circonscrivais,circonscrivait,circonscrivions,circonscriviez,circonscrivaient,circonscrivis,circonscrivis,circonscrivit,circonscrivîmes,circonscrivîtes,circonscrivirent,circonscrirai,circonscriras,circonscrira,circonscrirons,circonscrirez,circonscriront,circonscrirais,circonscrirais,circonscrirait,circonscririons,circonscririez,circonscriraient,circonscrive,circonscrives,circonscrive,circonscrivions,circonscriviez,circonscrivent,circonscrivisse,circonscrivisses,circonscrivît,circonscrivissions,circonscrivissiez,circonscrivissent,circonscris,circonscrivons,circonscrivez,None
1092,coécrire,coécrivant,coécrit,coécris,coécris,coécrit,coécrivons,coécrivez,coécrivent,coécrivais,coécrivais,coécrivait,coécrivions,coécriviez,coécrivaient,coécrivis,coécrivis,coécrivit,coécrivîmes,coécrivîtes,coécrivirent,coécrirai,coécriras,coécrira,coécrirons,coécrirez,coécriront,coécrirais,coécrirais,coécrirait,coécririons,coécririez,coécriraient,coécrive,coécrives,coécrive,coécrivions,coécriviez,coécrivent,coécrivisse,coécrivisses,coécrivît,coécrivissions,coécrivissiez,coécrivissent,coécris,coécrivons,coécrivez,None
1457,décrire,décrivant,décrit,décris,décris,décrit,décrivons,décrivez,décrivent,décrivais,décrivais,décrivait,décrivions,décriviez,décrivaient,décrivis,décrivis,décrivit,décrivîmes,décrivîtes,décrivirent,décrirai,décriras,décrira,décrirons,décrirez,décriront,décrirais,décrirais,décrirait,décririons,décririez,décriraient,décrive,décrives,décrive,décrivions,décriviez,décrivent,décrivisse,décrivisses,décrivît,décrivissions,décrivissiez,décrivissent,décris,décrivons,décrivez,None
1768,désinscrire,désinscrivant,désinscrit,désinscris,désinscris,désinscrit,désinscrivons,désinscrivez,désinscrivent,désinscrivais,désinscrivais,désinscrivait,désinscrivions,désinscriviez,désinscrivaient,désinscrivis,désinscrivis,désinscrivit,désinscrivîmes,désinscrivîtes,désinscrivirent,désinscrirai,désinscriras,désinscrira,désinscrirons,désinscrirez,désinscriront,désinscrirais,désinscrirais,désinscrirait,désinscririons,désinscririez,désinscriraient,désinscrive,désinscrives,désinscrive,désinscrivions,désinscriviez,désinscrivent,désinscrivisse,désinscrivisses,désinscrivît,désinscrivissions,désinscrivissiez,désinscrivissent,désinscris,désinscrivons,désinscrivez,None
2770,inscrire,inscrivant,inscrit,inscris,inscris,inscrit,inscrivons,inscrivez,inscrivent,inscrivais,inscrivais,inscrivait,inscrivions,inscriviez,inscrivaient,inscrivis,inscrivis,inscrivit,inscrivîmes,inscrivîtes,inscrivirent,inscrirai,inscriras,inscrira,inscrirons,inscrirez,inscriront,inscrirais,inscrirais,inscrirait,inscririons,inscririez,inscriraient,inscrive,inscrives,inscrive,inscrivions,inscriviez,inscrivent,inscrivisse,inscrivisses,inscrivît,inscrivissions,inscrivissiez,inscrivissent,inscris,inscrivons,inscrivez,None
3583,prescrire,prescrivant,prescrit,prescris,prescris,prescrit,prescrivons,prescrivez,prescrivent,prescrivais,prescrivais,prescrivait,prescrivions,prescriviez,prescrivaient,prescrivis,prescrivis,prescrivit,prescrivîmes,prescrivîtes,prescriviren

#### traire

In [24]:
def traire_modify_root(root: str, ending: str, column: str):
    should_use_y = column in ["present participle", "present 1pl", "present 2pl", "subjunctive present 1pl", "subjunctive present 2pl", "imperative 1pl", "imperative 2pl"] or column.startswith("imperfect") or column.startswith("past historic") or column.startswith("subjunctive imperfect")
    if ending is not None:
        ending_starts_with_vowel = any(ending.startswith(l) for l in "aeiouyâîûè") and ending not in ["e", "es", "ent"]
        assert should_use_y == ending_starts_with_vowel, f"{ending} {should_use_y}"
    if should_use_y:
        return f"{root.removesuffix("i")}y"
    return root

like_traire, remaining_data = is_like("traire", "re", (traire_modify_root, "-traire"))
display(like_traire)

,infinitive,present participle,past participle,present 1sg,present 2sg,present 3sg,present 1pl,present 2pl,present 3pl,imperfect 1sg,imperfect 2sg,imperfect 3sg,imperfect 1pl,imperfect 2pl,imperfect 3pl,past historic 1sg,past historic 2sg,past historic 3sg,past historic 1pl,past historic 2pl,past historic 3pl,future 1sg,future 2sg,future 3sg,future 1pl,future 2pl,future 3pl,conditional 1sg,conditional 2sg,conditional 3sg,conditional 1pl,conditional 2pl,conditional 3pl,subjunctive present 1sg,subjunctive present 2sg,subjunctive present 3sg,subjunctive present 1pl,subjunctive present 2pl,subjunctive present 3pl,subjunctive imperfect 1sg,subjunctive imperfect 2sg,subjunctive imperfect 3sg,subjunctive imperfect 1pl,subjunctive imperfect 2pl,subjunctive imperfect 3pl,imperative 2sg,imperative 1pl,imperative 2pl,group
0,re,ant,t,s,s,t,ons,ez,ent,ais,ais,ait,ions,iez,aient,ai,as,a,âmes,âtes,èrent,rai,ras,ra,rons,rez,ront,rais,rais,rait,rions,riez,raient,e,es,e,ions,iez,ent,asse,asses,ât,assions,assiez,assent,e,ons,ez,None


,infinitive,present participle,past participle,present 1sg,present 2sg,present 3sg,present 1pl,present 2pl,present 3pl,imperfect 1sg,imperfect 2sg,imperfect 3sg,imperfect 1pl,imperfect 2pl,imperfect 3pl,past historic 1sg,past historic 2sg,past historic 3sg,past historic 1pl,past historic 2pl,past historic 3pl,future 1sg,future 2sg,future 3sg,future 1pl,future 2pl,future 3pl,conditional 1sg,conditional 2sg,conditional 3sg,conditional 1pl,conditional 2pl,conditional 3pl,subjunctive present 1sg,subjunctive present 2sg,subjunctive present 3sg,subjunctive present 1pl,subjunctive present 2pl,subjunctive present 3pl,subjunctive imperfect 1sg,subjunctive imperfect 2sg,subjunctive imperfect 3sg,subjunctive imperfect 1pl,subjunctive imperfect 2pl,subjunctive imperfect 3pl,imperative 2sg,imperative 1pl,imperative 2pl,group
44,abstraire,abstrayant,abstrait,abstrais,abstrais,abstrait,abstrayons,abstrayez,abstraient,abstrayais,abstrayais,abstrayait,abstrayions,abstrayiez,abstrayaient,abstrayai,abstrayas,abstraya,abstrayâmes,abstrayâtes,abstrayèrent,abstrairai,abstrairas,abstraira,abstrairons,abstrairez,abstrairont,abstrairais,abstrairais,abstrairait,abstrairions,abstrairiez,abstrairaient,abstraie,abstraies,abstraie,abstrayions,abstrayiez,abstraient,abstrayasse,abstrayasses,abstrayât,abstrayassions,abstrayassiez,abstrayassent,abstraie,abstrayons,abstrayez,None
1266,distraire,distrayant,distrait,distrais,distrais,distrait,distrayons,distrayez,distraient,distrayais,distrayais,distrayait,distrayions,distrayiez,distrayaient,distrayai,distrayas,distraya,distrayâmes,distrayâtes,distrayèrent,distrairai,distrairas,distraira,distrairons,distrairez,distrairont,distrairais,distrairais,distrairait,distrairions,distrairiez,distrairaient,distraie,distraies,distraie,distrayions,distrayiez,distraient,distrayasse,distrayasses,distrayât,distrayassions,distrayassiez,distrayassent,distraie,distrayons,distrayez,None
2224,extraire,extrayant,extrait,extrais,extrais,extrait,extrayons,extrayez,extraient,extrayais,extrayais,extrayait,extrayions,extrayiez,extrayaient,extrayai,extrayas,extraya,extrayâmes,extrayâtes,extrayèrent,extrairai,extrairas,extraira,extrairons,extrairez,extrairont,extrairais,extrairais,extrairait,extrairions,extrairiez,extrairaient,extraie,extraies,extraie,extrayions,extrayiez,extraient,extrayasse,extrayasses,extrayât,extrayassions,extrayassiez,extrayassent,extraie,extrayons,extrayez,None
4133,retraire,retrayant,retrait,retrais,retrais,retrait,retrayons,retrayez,retraient,retrayais,retrayais,retrayait,retrayions,retrayiez,retrayaient,retrayai,retrayas,retraya,retrayâmes,retrayâtes,retrayèrent,retrairai,retrairas,retraira,retrairons,retrairez,retrairont,retrairais,retrairais,retrairait,retrairions,retrairiez,retrairaient,retraie,retraies,retraie,retrayions,retrayiez,retraient,retrayasse,retrayasses,retrayât,retrayassions,retrayassiez,retrayassent,retraie,retrayons,retrayez,None
4569,soustraire,soustrayant,soustrait,soustrais,soustrais,soustrait,soustrayons,soustrayez,soustraient,soustrayais,soustrayais,soustrayait,soustrayions,soustrayiez,soustrayaient,soustrayai,soustrayas,soustraya,soustrayâmes,soustrayâtes,soustrayèrent,soustrairai,soustrairas,soustraira,soustrairons,soustrairez,soustrairont,soustrairais,soustrairais,soustrairait,soustrairions,soustrairiez,soustrairaient,soustraie,soustraies,soustraie,soustrayions,soustrayiez,soustraient,soustrayasse,soustrayasses,soustrayât,soustrayassions,soustrayassiez,soustrayassent,soustraie,soustrayons,soustrayez,None
4855,traire,trayant,trait,trais,trais,trait,trayons,trayez,traient,trayais,trayais,trayait,trayions,trayiez,trayaient,trayai,trayas,traya,trayâmes,trayâtes,trayèrent,trairai,trairas,traira,trairons,trairez,trairont,trairais,trairais,trairait,trairions,trairiez,trairaient,traie,traies,traie,trayions,trayiez,traient,trayasse,trayasses,trayât,trayassions,trayassiez,trayassent,traie,trayons,trayez,None


#### rompre

In [25]:
def foutre_modify_root(root: str, ending: str, column: str):
    if root.endswith("fout"):
        if column in ["present 1sg", "present 2sg", "present 3sg", "imperative 2sg"]:
            return root.removesuffix("t"), "foutre"
        return root, "foutre"
    return root, "-rompre"

like_rompre, remaining_data = is_like("rompre", "re", foutre_modify_root)
display(like_rompre)

,infinitive,present participle,past participle,present 1sg,present 2sg,present 3sg,present 1pl,present 2pl,present 3pl,imperfect 1sg,imperfect 2sg,imperfect 3sg,imperfect 1pl,imperfect 2pl,imperfect 3pl,past historic 1sg,past historic 2sg,past historic 3sg,past historic 1pl,past historic 2pl,past historic 3pl,future 1sg,future 2sg,future 3sg,future 1pl,future 2pl,future 3pl,conditional 1sg,conditional 2sg,conditional 3sg,conditional 1pl,conditional 2pl,conditional 3pl,subjunctive present 1sg,subjunctive present 2sg,subjunctive present 3sg,subjunctive present 1pl,subjunctive present 2pl,subjunctive present 3pl,subjunctive imperfect 1sg,subjunctive imperfect 2sg,subjunctive imperfect 3sg,subjunctive imperfect 1pl,subjunctive imperfect 2pl,subjunctive imperfect 3pl,imperative 2sg,imperative 1pl,imperative 2pl,group
0,re,ant,u,s,s,t,ons,ez,ent,ais,ais,ait,ions,iez,aient,is,is,it,îmes,îtes,irent,rai,ras,ra,rons,rez,ront,rais,rais,rait,rions,riez,raient,e,es,e,ions,iez,ent,isse,isses,ît,issions,issiez,issent,s,ons,ez,None


,infinitive,present participle,past participle,present 1sg,present 2sg,present 3sg,present 1pl,present 2pl,present 3pl,imperfect 1sg,imperfect 2sg,imperfect 3sg,imperfect 1pl,imperfect 2pl,imperfect 3pl,past historic 1sg,past historic 2sg,past historic 3sg,past historic 1pl,past historic 2pl,past historic 3pl,future 1sg,future 2sg,future 3sg,future 1pl,future 2pl,future 3pl,conditional 1sg,conditional 2sg,conditional 3sg,conditional 1pl,conditional 2pl,conditional 3pl,subjunctive present 1sg,subjunctive present 2sg,subjunctive present 3sg,subjunctive present 1pl,subjunctive present 2pl,subjunctive present 3pl,subjunctive imperfect 1sg,subjunctive imperfect 2sg,subjunctive imperfect 3sg,subjunctive imperfect 1pl,subjunctive imperfect 2pl,subjunctive imperfect 3pl,imperative 2sg,imperative 1pl,imperative 2pl,group
1061,corrompre,corrompant,corrompu,corromps,corromps,corrompt,corrompons,corrompez,corrompent,corrompais,corrompais,corrompait,corrompions,corrompiez,corrompaient,corrompis,corrompis,corrompit,corrompîmes,corrompîtes,corrompirent,corromprai,corrompras,corrompra,corromprons,corromprez,corrompront,corromprais,corromprais,corromprait,corromprions,corrompriez,corrompraient,corrompe,corrompes,corrompe,corrompions,corrompiez,corrompent,corrompisse,corrompisses,corrompît,corrompissions,corrompissiez,corrompissent,corromps,corrompons,corrompez,None
2382,foutre,foutant,foutu,fous,fous,fout,foutons,foutez,foutent,foutais,foutais,foutait,foutions,foutiez,foutaient,foutis,foutis,foutit,foutîmes,foutîtes,foutirent,foutrai,foutras,foutra,foutrons,foutrez,foutront,foutrais,foutrais,foutrait,foutrions,foutriez,foutraient,foute,foutes,foute,foutions,foutiez,foutent,foutisse,foutisses,foutît,foutissions,foutissiez,foutissent,fous,foutons,foutez,None
2811,interrompre,interrompant,interrompu,interromps,interromps,interrompt,interrompons,interrompez,interrompent,interrompais,interrompais,interrompait,interrompions,interrompiez,interrompaient,interrompis,interrompis,interrompit,interrompîmes,interrompîtes,interrompirent,interromprai,interrompras,interrompra,interromprons,interromprez,interrompront,interromprais,interromprais,interromprait,interromprions,interrompriez,interrompraient,interrompe,interrompes,interrompe,interrompions,interrompiez,interrompent,interrompisse,interrompisses,interrompît,interrompissions,interrompissiez,interrompissent,interromps,interrompons,interrompez,None
4190,rompre,rompant,rompu,romps,romps,rompt,rompons,rompez,rompent,rompais,rompais,rompait,rompions,rompiez,rompaient,rompis,rompis,rompit,rompîmes,rompîtes,rompirent,romprai,rompras,rompra,romprons,romprez,rompront,romprais,romprais,romprait,romprions,rompriez,rompraient,rompe,rompes,rompe,rompions,rompiez,rompent,rompisse,rompisses,rompît,rompissions,rompissiez,rompissent,romps,rompons,rompez,None


#### suivre

In [26]:
def suivre_modify_root(root: str, ending: str, column: str):
    if column in ["present 1sg", "present 2sg", "present 3sg", "imperative 2sg"]:
        return root.removesuffix("v")
    return root

like_suivre, remaining_data = is_like("suivre", "re", (suivre_modify_root, "-suivre"))
display(like_suivre)

,infinitive,present participle,past participle,present 1sg,present 2sg,present 3sg,present 1pl,present 2pl,present 3pl,imperfect 1sg,imperfect 2sg,imperfect 3sg,imperfect 1pl,imperfect 2pl,imperfect 3pl,past historic 1sg,past historic 2sg,past historic 3sg,past historic 1pl,past historic 2pl,past historic 3pl,future 1sg,future 2sg,future 3sg,future 1pl,future 2pl,future 3pl,conditional 1sg,conditional 2sg,conditional 3sg,conditional 1pl,conditional 2pl,conditional 3pl,subjunctive present 1sg,subjunctive present 2sg,subjunctive present 3sg,subjunctive present 1pl,subjunctive present 2pl,subjunctive present 3pl,subjunctive imperfect 1sg,subjunctive imperfect 2sg,subjunctive imperfect 3sg,subjunctive imperfect 1pl,subjunctive imperfect 2pl,subjunctive imperfect 3pl,imperative 2sg,imperative 1pl,imperative 2pl,group
0,re,ant,i,s,s,t,ons,ez,ent,ais,ais,ait,ions,iez,aient,is,is,it,îmes,îtes,irent,rai,ras,ra,rons,rez,ront,rais,rais,rait,rions,riez,raient,e,es,e,ions,iez,ent,isse,isses,ît,issions,issiez,issent,s,ons,ez,None


,infinitive,present participle,past participle,present 1sg,present 2sg,present 3sg,present 1pl,present 2pl,present 3pl,imperfect 1sg,imperfect 2sg,imperfect 3sg,imperfect 1pl,imperfect 2pl,imperfect 3pl,past historic 1sg,past historic 2sg,past historic 3sg,past historic 1pl,past historic 2pl,past historic 3pl,future 1sg,future 2sg,future 3sg,future 1pl,future 2pl,future 3pl,conditional 1sg,conditional 2sg,conditional 3sg,conditional 1pl,conditional 2pl,conditional 3pl,subjunctive present 1sg,subjunctive present 2sg,subjunctive present 3sg,subjunctive present 1pl,subjunctive present 2pl,subjunctive present 3pl,subjunctive imperfect 1sg,subjunctive imperfect 2sg,subjunctive imperfect 3sg,subjunctive imperfect 1pl,subjunctive imperfect 2pl,subjunctive imperfect 3pl,imperative 2sg,imperative 1pl,imperative 2pl,group
2074,ensuivre,ensuivant,ensuivi,None,None,ensuit,None,None,ensuivent,None,None,ensuivait,None,None,ensuivaient,None,None,ensuivit,None,None,ensuivirent,None,None,ensuivra,None,None,ensuivront,None,None,ensuivrait,None,None,ensuivraient,None,None,ensuive,None,None,ensuivent,None,None,ensuivît,None,None,ensuivissent,None,None,None,None
3573,poursuivre,poursuivant,poursuivi,poursuis,poursuis,poursuit,poursuivons,poursuivez,poursuivent,poursuivais,poursuivais,poursuivait,poursuivions,poursuiviez,poursuivaient,poursuivis,poursuivis,poursuivit,poursuivîmes,poursuivîtes,poursuivirent,poursuivrai,poursuivras,poursuivra,poursuivrons,poursuivrez,poursuivront,poursuivrais,poursuivrais,poursuivrait,poursuivrions,poursuivriez,poursuivraient,poursuive,poursuives,poursuive,poursuivions,poursuiviez,poursuivent,poursuivisse,poursuivisses,poursuivît,poursuivissions,poursuivissiez,poursuivissent,poursuis,poursuivons,poursuivez,None
4635,suivre,suivant,suivi,suis,suis,suit,suivons,suivez,suivent,suivais,suivais,suivait,suivions,suiviez,suivaient,suivis,suivis,suivit,suivîmes,suivîtes,suivirent,suivrai,suivras,suivra,suivrons,suivrez,suivront,suivrais,suivrais,suivrait,suivrions,suivriez,suivraient,suive,suives,suive,suivions,suiviez,suivent,suivisse,suivisses,suivît,suivissions,suivissiez,suivissent,suis,suivons,suivez,None


### -ire

#### rire

In [27]:
like_rire, remaining_data = is_like("rire", "ire", "-rire")
display(like_rire)

,infinitive,present participle,past participle,present 1sg,present 2sg,present 3sg,present 1pl,present 2pl,present 3pl,imperfect 1sg,imperfect 2sg,imperfect 3sg,imperfect 1pl,imperfect 2pl,imperfect 3pl,past historic 1sg,past historic 2sg,past historic 3sg,past historic 1pl,past historic 2pl,past historic 3pl,future 1sg,future 2sg,future 3sg,future 1pl,future 2pl,future 3pl,conditional 1sg,conditional 2sg,conditional 3sg,conditional 1pl,conditional 2pl,conditional 3pl,subjunctive present 1sg,subjunctive present 2sg,subjunctive present 3sg,subjunctive present 1pl,subjunctive present 2pl,subjunctive present 3pl,subjunctive imperfect 1sg,subjunctive imperfect 2sg,subjunctive imperfect 3sg,subjunctive imperfect 1pl,subjunctive imperfect 2pl,subjunctive imperfect 3pl,imperative 2sg,imperative 1pl,imperative 2pl,group
0,ire,iant,i,is,is,it,ions,iez,ient,iais,iais,iait,iions,iiez,iaient,is,is,it,îmes,îtes,irent,irai,iras,ira,irons,irez,iront,irais,irais,irait,irions,iriez,iraient,ie,ies,ie,iions,iiez,ient,isse,isses,ît,issions,issiez,issent,is,ions,iez,None


,infinitive,present participle,past participle,present 1sg,present 2sg,present 3sg,present 1pl,present 2pl,present 3pl,imperfect 1sg,imperfect 2sg,imperfect 3sg,imperfect 1pl,imperfect 2pl,imperfect 3pl,past historic 1sg,past historic 2sg,past historic 3sg,past historic 1pl,past historic 2pl,past historic 3pl,future 1sg,future 2sg,future 3sg,future 1pl,future 2pl,future 3pl,conditional 1sg,conditional 2sg,conditional 3sg,conditional 1pl,conditional 2pl,conditional 3pl,subjunctive present 1sg,subjunctive present 2sg,subjunctive present 3sg,subjunctive present 1pl,subjunctive present 2pl,subjunctive present 3pl,subjunctive imperfect 1sg,subjunctive imperfect 2sg,subjunctive imperfect 3sg,subjunctive imperfect 1pl,subjunctive imperfect 2pl,subjunctive imperfect 3pl,imperative 2sg,imperative 1pl,imperative 2pl,group
4175,rire,riant,ri,ris,ris,rit,rions,riez,rient,riais,riais,riait,riions,riiez,riaient,ris,ris,rit,rîmes,rîtes,rirent,rirai,riras,rira,rirons,rirez,riront,rirais,rirais,rirait,ririons,ririez,riraient,rie,ries,rie,riions,riiez,rient,risse,risses,rît,rissions,rissiez,rissent,ris,rions,riez,None
4559,sourire,souriant,souri,souris,souris,sourit,sourions,souriez,sourient,souriais,souriais,souriait,souriions,souriiez,souriaient,souris,souris,sourit,sourîmes,sourîtes,sourirent,sourirai,souriras,sourira,sourirons,sourirez,souriront,sourirais,sourirais,sourirait,souririons,souririez,souriraient,sourie,souries,sourie,souriions,souriiez,sourient,sourisse,sourisses,sourît,sourissions,sourissiez,sourissent,souris,sourions,souriez,None


#### dire

In [28]:
like_dire, remaining_data = is_like("dire", "ire", "-dire/frire")
display(like_dire)

,infinitive,present participle,past participle,present 1sg,present 2sg,present 3sg,present 1pl,present 2pl,present 3pl,imperfect 1sg,imperfect 2sg,imperfect 3sg,imperfect 1pl,imperfect 2pl,imperfect 3pl,past historic 1sg,past historic 2sg,past historic 3sg,past historic 1pl,past historic 2pl,past historic 3pl,future 1sg,future 2sg,future 3sg,future 1pl,future 2pl,future 3pl,conditional 1sg,conditional 2sg,conditional 3sg,conditional 1pl,conditional 2pl,conditional 3pl,subjunctive present 1sg,subjunctive present 2sg,subjunctive present 3sg,subjunctive present 1pl,subjunctive present 2pl,subjunctive present 3pl,subjunctive imperfect 1sg,subjunctive imperfect 2sg,subjunctive imperfect 3sg,subjunctive imperfect 1pl,subjunctive imperfect 2pl,subjunctive imperfect 3pl,imperative 2sg,imperative 1pl,imperative 2pl,group
0,ire,isant,it,is,is,it,isons,ites,isent,isais,isais,isait,isions,isiez,isaient,is,is,it,îmes,îtes,irent,irai,iras,ira,irons,irez,iront,irais,irais,irait,irions,iriez,iraient,ise,ises,ise,isions,isiez,isent,isse,isses,ît,issions,issiez,issent,is,isons,ites,None


,infinitive,present participle,past participle,present 1sg,present 2sg,present 3sg,present 1pl,present 2pl,present 3pl,imperfect 1sg,imperfect 2sg,imperfect 3sg,imperfect 1pl,imperfect 2pl,imperfect 3pl,past historic 1sg,past historic 2sg,past historic 3sg,past historic 1pl,past historic 2pl,past historic 3pl,future 1sg,future 2sg,future 3sg,future 1pl,future 2pl,future 3pl,conditional 1sg,conditional 2sg,conditional 3sg,conditional 1pl,conditional 2pl,conditional 3pl,subjunctive present 1sg,subjunctive present 2sg,subjunctive present 3sg,subjunctive present 1pl,subjunctive present 2pl,subjunctive present 3pl,subjunctive imperfect 1sg,subjunctive imperfect 2sg,subjunctive imperfect 3sg,subjunctive imperfect 1pl,subjunctive imperfect 2pl,subjunctive imperfect 3pl,imperative 2sg,imperative 1pl,imperative 2pl,group
1229,dire,disant,dit,dis,dis,dit,disons,dites,disent,disais,disais,disait,disions,disiez,disaient,dis,dis,dit,dîmes,dîtes,dirent,dirai,diras,dira,dirons,direz,diront,dirais,dirais,dirait,dirions,diriez,diraient,dise,dises,dise,disions,disiez,disent,disse,disses,dît,dissions,dissiez,dissent,dis,disons,dites,None
2408,frire,frisant,frit,fris,fris,frit,None,None,None,frisais,frisais,frisait,None,None,None,fris,fris,frit,None,None,None,frirai,friras,frira,None,None,None,frirais,frirais,frirait,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,fris,None,None,None
3906,redire,redisant,redit,redis,redis,redit,redisons,redites,redisent,redisais,redisais,redisait,redisions,redisiez,redisaient,redis,redis,redit,redîmes,redîtes,redirent,redirai,rediras,redira,redirons,redirez,rediront,redirais,redirais,redirait,redirions,rediriez,rediraient,redise,redises,redise,redisions,redisiez,redisent,redisse,redisses,redît,redissions,redissiez,redissent,redis,redisons,redites,None


In [29]:
like_interdire, remaining_data = is_like("interdire", "ire", "-dire (like interdire)")
display(like_interdire)

,infinitive,present participle,past participle,present 1sg,present 2sg,present 3sg,present 1pl,present 2pl,present 3pl,imperfect 1sg,imperfect 2sg,imperfect 3sg,imperfect 1pl,imperfect 2pl,imperfect 3pl,past historic 1sg,past historic 2sg,past historic 3sg,past historic 1pl,past historic 2pl,past historic 3pl,future 1sg,future 2sg,future 3sg,future 1pl,future 2pl,future 3pl,conditional 1sg,conditional 2sg,conditional 3sg,conditional 1pl,conditional 2pl,conditional 3pl,subjunctive present 1sg,subjunctive present 2sg,subjunctive present 3sg,subjunctive present 1pl,subjunctive present 2pl,subjunctive present 3pl,subjunctive imperfect 1sg,subjunctive imperfect 2sg,subjunctive imperfect 3sg,subjunctive imperfect 1pl,subjunctive imperfect 2pl,subjunctive imperfect 3pl,imperative 2sg,imperative 1pl,imperative 2pl,group
0,ire,isant,it,is,is,it,isons,isez,isent,isais,isais,isait,isions,isiez,isaient,is,is,it,îmes,îtes,irent,irai,iras,ira,irons,irez,iront,irais,irais,irait,irions,iriez,iraient,ise,ises,ise,isions,isiez,isent,isse,isses,ît,issions,issiez,issent,is,isons,isez,None


,infinitive,present participle,past participle,present 1sg,present 2sg,present 3sg,present 1pl,present 2pl,present 3pl,imperfect 1sg,imperfect 2sg,imperfect 3sg,imperfect 1pl,imperfect 2pl,imperfect 3pl,past historic 1sg,past historic 2sg,past historic 3sg,past historic 1pl,past historic 2pl,past historic 3pl,future 1sg,future 2sg,future 3sg,future 1pl,future 2pl,future 3pl,conditional 1sg,conditional 2sg,conditional 3sg,conditional 1pl,conditional 2pl,conditional 3pl,subjunctive present 1sg,subjunctive present 2sg,subjunctive present 3sg,subjunctive present 1pl,subjunctive present 2pl,subjunctive present 3pl,subjunctive imperfect 1sg,subjunctive imperfect 2sg,subjunctive imperfect 3sg,subjunctive imperfect 1pl,subjunctive imperfect 2pl,subjunctive imperfect 3pl,imperative 2sg,imperative 1pl,imperative 2pl,group
1025,contredire,contredisant,contredit,contredis,contredis,contredit,contredisons,contredisez,contredisent,contredisais,contredisais,contredisait,contredisions,contredisiez,contredisaient,contredis,contredis,contredit,contredîmes,contredîtes,contredirent,contredirai,contrediras,contredira,contredirons,contredirez,contrediront,contredirais,contredirais,contredirait,contredirions,contrediriez,contrediraient,contredise,contredises,contredise,contredisions,contredisiez,contredisent,contredisse,contredisses,contredît,contredissions,contredissiez,contredissent,contredis,contredisons,contredisez,None
1477,dédire,dédisant,dédit,dédis,dédis,dédit,dédisons,dédisez,dédisent,dédisais,dédisais,dédisait,dédisions,dédisiez,dédisaient,dédis,dédis,dédit,dédîmes,dédîtes,dédirent,dédirai,dédiras,dédira,dédirons,dédirez,dédiront,dédirais,dédirais,dédirait,dédirions,dédiriez,dédiraient,dédise,dédises,dédise,dédisions,dédisiez,dédisent,dédisse,dédisses,dédît,dédissions,dédissiez,dédissent,dédis,dédisons,dédisez,None
2800,interdire,interdisant,interdit,interdis,interdis,interdit,interdisons,interdisez,interdisent,interdisais,interdisais,interdisait,interdisions,interdisiez,interdisaient,interdis,interdis,interdit,interdîmes,interdîtes,interdirent,interdirai,interdiras,interdira,interdirons,interdirez,interdiront,interdirais,interdirais,interdirait,interdirions,interdiriez,interdiraient,interdise,interdises,interdise,interdisions,interdisiez,interdisent,interdisse,interdisses,interdît,interdissions,interdissiez,interdissent,interdis,interdisons,interdisez,None
3169,médire,médisant,médit,médis,médis,médit,médisons,médisez,médisent,médisais,médisais,médisait,médisions,médisiez,médisaient,médis,médis,médit,médîmes,médîtes,médirent,médirai,médiras,médira,médirons,médirez,médiront,médirais,médirais,médirait,médirions,médiriez,médiraient,médise,médises,médise,médisions,médisiez,médisent,médisse,médisses,médît,médissions,médissiez,médissent,médis,médisons,médisez,None
3637,prædire,prædisant,prædit,prædis,prædis,prædit,prædisons,prædisez,prædisent,prædisais,prædisais,prædisait,prædisions,prædisiez,prædisaient,prædis,prædis,prædit,prædîmes,prædîtes,prædirent,prædirai,prædiras,prædira,prædirons,prædirez,prædiront,prædirais,prædirais,prædirait,prædirions,prædiriez,prædiraient,prædise,prædises,prædise,prædisions,prædisiez,prædisent,prædisse,prædisses,prædît,prædissions,prædissiez,prædissent,prædis,prædisons,prædisez,None
3650,prédire,prédisant,prédit,prédis,prédis,prédit,prédisons,prédisez,prédisent,prédisais,prédisais,prédisait,prédisions,prédisiez,prédisaient,prédis,prédis,prédit,prédîmes,prédîtes,prédirent,prédirai,prédiras,prédira,prédirons,prédirez,prédiront,prédirais,prédirais,prédirait,prédirions,prédiriez,prédiraient,prédise,prédises,prédise,prédisions,prédisiez,prédisent,prédisse,prédisses,prédît,prédissions,prédissiez,prédissent,prédis,prédisons,prédisez,None


In [30]:
like_maudire, remaining_data = is_like("maudire", "ire", "-dire (like maudire)")
display(like_maudire)

,infinitive,present participle,past participle,present 1sg,present 2sg,present 3sg,present 1pl,present 2pl,present 3pl,imperfect 1sg,imperfect 2sg,imperfect 3sg,imperfect 1pl,imperfect 2pl,imperfect 3pl,past historic 1sg,past historic 2sg,past historic 3sg,past historic 1pl,past historic 2pl,past historic 3pl,future 1sg,future 2sg,future 3sg,future 1pl,future 2pl,future 3pl,conditional 1sg,conditional 2sg,conditional 3sg,conditional 1pl,conditional 2pl,conditional 3pl,subjunctive present 1sg,subjunctive present 2sg,subjunctive present 3sg,subjunctive present 1pl,subjunctive present 2pl,subjunctive present 3pl,subjunctive imperfect 1sg,subjunctive imperfect 2sg,subjunctive imperfect 3sg,subjunctive imperfect 1pl,subjunctive imperfect 2pl,subjunctive imperfect 3pl,imperative 2sg,imperative 1pl,imperative 2pl,group
0,ire,issant,it,is,is,it,issons,issez,issent,issais,issais,issait,issions,issiez,issaient,is,is,it,îmes,îtes,irent,irai,iras,ira,irons,irez,iront,irais,irais,irait,irions,iriez,iraient,isse,isses,isse,issions,issiez,issent,isse,isses,ît,issions,issiez,issent,is,issons,issez,None


,infinitive,present participle,past participle,present 1sg,present 2sg,present 3sg,present 1pl,present 2pl,present 3pl,imperfect 1sg,imperfect 2sg,imperfect 3sg,imperfect 1pl,imperfect 2pl,imperfect 3pl,past historic 1sg,past historic 2sg,past historic 3sg,past historic 1pl,past historic 2pl,past historic 3pl,future 1sg,future 2sg,future 3sg,future 1pl,future 2pl,future 3pl,conditional 1sg,conditional 2sg,conditional 3sg,conditional 1pl,conditional 2pl,conditional 3pl,subjunctive present 1sg,subjunctive present 2sg,subjunctive present 3sg,subjunctive present 1pl,subjunctive present 2pl,subjunctive present 3pl,subjunctive imperfect 1sg,subjunctive imperfect 2sg,subjunctive imperfect 3sg,subjunctive imperfect 1pl,subjunctive imperfect 2pl,subjunctive imperfect 3pl,imperative 2sg,imperative 1pl,imperative 2pl,group
3044,maudire,maudissant,maudit,maudis,maudis,maudit,maudissons,maudissez,maudissent,maudissais,maudissais,maudissait,maudissions,maudissiez,maudissaient,maudis,maudis,maudit,maudîmes,maudîtes,maudirent,maudirai,maudiras,maudira,maudirons,maudirez,maudiront,maudirais,maudirais,maudirait,maudirions,maudiriez,maudiraient,maudisse,maudisses,maudisse,maudissions,maudissiez,maudissent,maudisse,maudisses,maudît,maudissions,maudissiez,maudissent,maudis,maudissons,maudissez,None


#### confire

In [31]:
like_confire, remaining_data = is_like("confire", "ire", "-confire")
display(like_confire)

,infinitive,present participle,past participle,present 1sg,present 2sg,present 3sg,present 1pl,present 2pl,present 3pl,imperfect 1sg,imperfect 2sg,imperfect 3sg,imperfect 1pl,imperfect 2pl,imperfect 3pl,past historic 1sg,past historic 2sg,past historic 3sg,past historic 1pl,past historic 2pl,past historic 3pl,future 1sg,future 2sg,future 3sg,future 1pl,future 2pl,future 3pl,conditional 1sg,conditional 2sg,conditional 3sg,conditional 1pl,conditional 2pl,conditional 3pl,subjunctive present 1sg,subjunctive present 2sg,subjunctive present 3sg,subjunctive present 1pl,subjunctive present 2pl,subjunctive present 3pl,subjunctive imperfect 1sg,subjunctive imperfect 2sg,subjunctive imperfect 3sg,subjunctive imperfect 1pl,subjunctive imperfect 2pl,subjunctive imperfect 3pl,imperative 2sg,imperative 1pl,imperative 2pl,group
0,ire,isant,it,is,is,it,isons,isez,isent,isais,isais,isait,isions,isiez,isaient,is,is,it,îmes,îtes,irent,irai,iras,ira,irons,irez,iront,irais,irais,irait,irions,iriez,iraient,ise,ises,ise,isions,isiez,isent,isisse,isisses,isît,isissions,isissiez,isissent,is,isons,isez,None


,infinitive,present participle,past participle,present 1sg,present 2sg,present 3sg,present 1pl,present 2pl,present 3pl,imperfect 1sg,imperfect 2sg,imperfect 3sg,imperfect 1pl,imperfect 2pl,imperfect 3pl,past historic 1sg,past historic 2sg,past historic 3sg,past historic 1pl,past historic 2pl,past historic 3pl,future 1sg,future 2sg,future 3sg,future 1pl,future 2pl,future 3pl,conditional 1sg,conditional 2sg,conditional 3sg,conditional 1pl,conditional 2pl,conditional 3pl,subjunctive present 1sg,subjunctive present 2sg,subjunctive present 3sg,subjunctive present 1pl,subjunctive present 2pl,subjunctive present 3pl,subjunctive imperfect 1sg,subjunctive imperfect 2sg,subjunctive imperfect 3sg,subjunctive imperfect 1pl,subjunctive imperfect 2pl,subjunctive imperfect 3pl,imperative 2sg,imperative 1pl,imperative 2pl,group
965,confire,confisant,confit,confis,confis,confit,confisons,confisez,confisent,confisais,confisais,confisait,confisions,confisiez,confisaient,confis,confis,confit,confîmes,confîtes,confirent,confirai,confiras,confira,confirons,confirez,confiront,confirais,confirais,confirait,confirions,confiriez,confiraient,confise,confises,confise,confisions,confisiez,confisent,confisisse,confisisses,confisît,confisissions,confisissiez,confisissent,confis,confisons,confisez,None
1433,déconfire,déconfisant,déconfit,déconfis,déconfis,déconfit,déconfisons,déconfisez,déconfisent,déconfisais,déconfisais,déconfisait,déconfisions,déconfisiez,déconfisaient,déconfis,déconfis,déconfit,déconfîmes,déconfîtes,déconfirent,déconfirai,déconfiras,déconfira,déconfirons,déconfirez,déconfiront,déconfirais,déconfirais,déconfirait,déconfirions,déconfiriez,déconfiraient,déconfise,déconfises,déconfise,déconfisions,déconfisiez,déconfisent,déconfisisse,déconfisisses,déconfisît,déconfisissions,déconfisissiez,déconfisissent,déconfis,déconfisons,déconfisez,None


#### bruire

In [32]:
like_bruire, remaining_data = is_like("bruire", "ire", "bruire")
display(like_bruire)

,infinitive,present participle,past participle,present 1sg,present 2sg,present 3sg,present 1pl,present 2pl,present 3pl,imperfect 1sg,imperfect 2sg,imperfect 3sg,imperfect 1pl,imperfect 2pl,imperfect 3pl,past historic 1sg,past historic 2sg,past historic 3sg,past historic 1pl,past historic 2pl,past historic 3pl,future 1sg,future 2sg,future 3sg,future 1pl,future 2pl,future 3pl,conditional 1sg,conditional 2sg,conditional 3sg,conditional 1pl,conditional 2pl,conditional 3pl,subjunctive present 1sg,subjunctive present 2sg,subjunctive present 3sg,subjunctive present 1pl,subjunctive present 2pl,subjunctive present 3pl,subjunctive imperfect 1sg,subjunctive imperfect 2sg,subjunctive imperfect 3sg,subjunctive imperfect 1pl,subjunctive imperfect 2pl,subjunctive imperfect 3pl,imperative 2sg,imperative 1pl,imperative 2pl,group
0,ire,issant,i,is,is,it,issons,issez,issent,issais,issais,issait,issions,issiez,issaient,is,is,it,îmes,îtes,irent,irai,iras,ira,irons,irez,iront,irais,irais,irait,irions,iriez,iraient,isse,isses,isse,issions,issiez,issent,isse,isses,ît,issions,issiez,issent,is,issons,issez,None


,infinitive,present participle,past participle,present 1sg,present 2sg,present 3sg,present 1pl,present 2pl,present 3pl,imperfect 1sg,imperfect 2sg,imperfect 3sg,imperfect 1pl,imperfect 2pl,imperfect 3pl,past historic 1sg,past historic 2sg,past historic 3sg,past historic 1pl,past historic 2pl,past historic 3pl,future 1sg,future 2sg,future 3sg,future 1pl,future 2pl,future 3pl,conditional 1sg,conditional 2sg,conditional 3sg,conditional 1pl,conditional 2pl,conditional 3pl,subjunctive present 1sg,subjunctive present 2sg,subjunctive present 3sg,subjunctive present 1pl,subjunctive present 2pl,subjunctive present 3pl,subjunctive imperfect 1sg,subjunctive imperfect 2sg,subjunctive imperfect 3sg,subjunctive imperfect 1pl,subjunctive imperfect 2pl,subjunctive imperfect 3pl,imperative 2sg,imperative 1pl,imperative 2pl,group
605,bruire,bruissant,brui,bruis,bruis,bruit,bruissons,bruissez,bruissent,bruissais,bruissais,bruissait,bruissions,bruissiez,bruissaient,bruis,bruis,bruit,bruîmes,bruîtes,bruirent,bruirai,bruiras,bruira,bruirons,bruirez,bruiront,bruirais,bruirais,bruirait,bruirions,bruiriez,bruiraient,bruisse,bruisses,bruisse,bruissions,bruissiez,bruissent,bruisse,bruisses,bruît,bruissions,bruissiez,bruissent,bruis,bruissons,bruissez,None


#### luire

In [33]:
like_luire, remaining_data = is_like("luire", "ire", "-luire")
display(like_luire)

,infinitive,present participle,past participle,present 1sg,present 2sg,present 3sg,present 1pl,present 2pl,present 3pl,imperfect 1sg,imperfect 2sg,imperfect 3sg,imperfect 1pl,imperfect 2pl,imperfect 3pl,past historic 1sg,past historic 2sg,past historic 3sg,past historic 1pl,past historic 2pl,past historic 3pl,future 1sg,future 2sg,future 3sg,future 1pl,future 2pl,future 3pl,conditional 1sg,conditional 2sg,conditional 3sg,conditional 1pl,conditional 2pl,conditional 3pl,subjunctive present 1sg,subjunctive present 2sg,subjunctive present 3sg,subjunctive present 1pl,subjunctive present 2pl,subjunctive present 3pl,subjunctive imperfect 1sg,subjunctive imperfect 2sg,subjunctive imperfect 3sg,subjunctive imperfect 1pl,subjunctive imperfect 2pl,subjunctive imperfect 3pl,imperative 2sg,imperative 1pl,imperative 2pl,group
0,ire,isant,i,is,is,it,isons,isez,isent,isais,isais,isait,isions,isiez,isaient,is,is,it,îmes,îtes,irent,irai,iras,ira,irons,irez,iront,irais,irais,irait,irions,iriez,iraient,ise,ises,ise,isions,isiez,isent,isisse,isisses,isisse,isissions,isissiez,isissent,is,isons,isez,None


,infinitive,present participle,past participle,present 1sg,present 2sg,present 3sg,present 1pl,present 2pl,present 3pl,imperfect 1sg,imperfect 2sg,imperfect 3sg,imperfect 1pl,imperfect 2pl,imperfect 3pl,past historic 1sg,past historic 2sg,past historic 3sg,past historic 1pl,past historic 2pl,past historic 3pl,future 1sg,future 2sg,future 3sg,future 1pl,future 2pl,future 3pl,conditional 1sg,conditional 2sg,conditional 3sg,conditional 1pl,conditional 2pl,conditional 3pl,subjunctive present 1sg,subjunctive present 2sg,subjunctive present 3sg,subjunctive present 1pl,subjunctive present 2pl,subjunctive present 3pl,subjunctive imperfect 1sg,subjunctive imperfect 2sg,subjunctive imperfect 3sg,subjunctive imperfect 1pl,subjunctive imperfect 2pl,subjunctive imperfect 3pl,imperative 2sg,imperative 1pl,imperative 2pl,group
2962,luire,luisant,lui,luis,luis,luit,luisons,luisez,luisent,luisais,luisais,luisait,luisions,luisiez,luisaient,luis,luis,luit,luîmes,luîtes,luirent,luirai,luiras,luira,luirons,luirez,luiront,luirais,luirais,luirait,luirions,luiriez,luiraient,luise,luises,luise,luisions,luisiez,luisent,luisisse,luisisses,luisisse,luisissions,luisissiez,luisissent,luis,luisons,luisez,None
3965,reluire,reluisant,relui,reluis,reluis,reluit,reluisons,reluisez,reluisent,reluisais,reluisais,reluisait,reluisions,reluisiez,reluisaient,reluis,reluis,reluit,reluîmes,reluîtes,reluirent,reluirai,reluiras,reluira,reluirons,reluirez,reluiront,reluirais,reluirais,reluirait,reluirions,reluiriez,reluiraient,reluise,reluises,reluise,reluisions,reluisiez,reluisent,reluisisse,reluisisses,reluisisse,reluisissions,reluisissiez,reluisissent,reluis,reluisons,reluisez,None


#### suffire

In [34]:
like_suffire, remaining_data = is_like("suffire", "ire", "suffire")
display(like_suffire)

,infinitive,present participle,past participle,present 1sg,present 2sg,present 3sg,present 1pl,present 2pl,present 3pl,imperfect 1sg,imperfect 2sg,imperfect 3sg,imperfect 1pl,imperfect 2pl,imperfect 3pl,past historic 1sg,past historic 2sg,past historic 3sg,past historic 1pl,past historic 2pl,past historic 3pl,future 1sg,future 2sg,future 3sg,future 1pl,future 2pl,future 3pl,conditional 1sg,conditional 2sg,conditional 3sg,conditional 1pl,conditional 2pl,conditional 3pl,subjunctive present 1sg,subjunctive present 2sg,subjunctive present 3sg,subjunctive present 1pl,subjunctive present 2pl,subjunctive present 3pl,subjunctive imperfect 1sg,subjunctive imperfect 2sg,subjunctive imperfect 3sg,subjunctive imperfect 1pl,subjunctive imperfect 2pl,subjunctive imperfect 3pl,imperative 2sg,imperative 1pl,imperative 2pl,group
0,ire,isant,i,is,is,it,isons,isez,isent,isais,isais,isait,isions,isiez,isaient,is,is,it,îmes,îtes,irent,irai,iras,ira,irons,irez,iront,irais,irais,irait,irions,iriez,iraient,ise,ises,ise,isions,isiez,isent,isse,isses,ît,issions,issiez,issent,is,isons,isez,None


,infinitive,present participle,past participle,present 1sg,present 2sg,present 3sg,present 1pl,present 2pl,present 3pl,imperfect 1sg,imperfect 2sg,imperfect 3sg,imperfect 1pl,imperfect 2pl,imperfect 3pl,past historic 1sg,past historic 2sg,past historic 3sg,past historic 1pl,past historic 2pl,past historic 3pl,future 1sg,future 2sg,future 3sg,future 1pl,future 2pl,future 3pl,conditional 1sg,conditional 2sg,conditional 3sg,conditional 1pl,conditional 2pl,conditional 3pl,subjunctive present 1sg,subjunctive present 2sg,subjunctive present 3sg,subjunctive present 1pl,subjunctive present 2pl,subjunctive present 3pl,subjunctive imperfect 1sg,subjunctive imperfect 2sg,subjunctive imperfect 3sg,subjunctive imperfect 1pl,subjunctive imperfect 2pl,subjunctive imperfect 3pl,imperative 2sg,imperative 1pl,imperative 2pl,group
4629,suffire,suffisant,suffi,suffis,suffis,suffit,suffisons,suffisez,suffisent,suffisais,suffisais,suffisait,suffisions,suffisiez,suffisaient,suffis,suffis,suffit,suffîmes,suffîtes,suffirent,suffirai,suffiras,suffira,suffirons,suffirez,suffiront,suffirais,suffirais,suffirait,suffirions,suffiriez,suffiraient,suffise,suffises,suffise,suffisions,suffisiez,suffisent,suffisse,suffisses,suffît,suffissions,suffissiez,suffissent,suffis,suffisons,suffisez,None


#### circoncire

In [35]:
like_circoncire, remaining_data = is_like("circoncire", "ire", "circoncire")
display(like_circoncire)

,infinitive,present participle,past participle,present 1sg,present 2sg,present 3sg,present 1pl,present 2pl,present 3pl,imperfect 1sg,imperfect 2sg,imperfect 3sg,imperfect 1pl,imperfect 2pl,imperfect 3pl,past historic 1sg,past historic 2sg,past historic 3sg,past historic 1pl,past historic 2pl,past historic 3pl,future 1sg,future 2sg,future 3sg,future 1pl,future 2pl,future 3pl,conditional 1sg,conditional 2sg,conditional 3sg,conditional 1pl,conditional 2pl,conditional 3pl,subjunctive present 1sg,subjunctive present 2sg,subjunctive present 3sg,subjunctive present 1pl,subjunctive present 2pl,subjunctive present 3pl,subjunctive imperfect 1sg,subjunctive imperfect 2sg,subjunctive imperfect 3sg,subjunctive imperfect 1pl,subjunctive imperfect 2pl,subjunctive imperfect 3pl,imperative 2sg,imperative 1pl,imperative 2pl,group
0,ire,isant,is,is,is,it,isons,isez,isent,isais,isais,isait,isions,isiez,isaient,is,is,it,îmes,îtes,irent,irai,iras,ira,irons,irez,iront,irais,irais,irait,irions,iriez,iraient,ise,ises,ise,isions,isiez,isent,isisse,isisses,isît,isissions,isissiez,isissent,is,isons,isez,None


,infinitive,present participle,past participle,present 1sg,present 2sg,present 3sg,present 1pl,present 2pl,present 3pl,imperfect 1sg,imperfect 2sg,imperfect 3sg,imperfect 1pl,imperfect 2pl,imperfect 3pl,past historic 1sg,past historic 2sg,past historic 3sg,past historic 1pl,past historic 2pl,past historic 3pl,future 1sg,future 2sg,future 3sg,future 1pl,future 2pl,future 3pl,conditional 1sg,conditional 2sg,conditional 3sg,conditional 1pl,conditional 2pl,conditional 3pl,subjunctive present 1sg,subjunctive present 2sg,subjunctive present 3sg,subjunctive present 1pl,subjunctive present 2pl,subjunctive present 3pl,subjunctive imperfect 1sg,subjunctive imperfect 2sg,subjunctive imperfect 3sg,subjunctive imperfect 1pl,subjunctive imperfect 2pl,subjunctive imperfect 3pl,imperative 2sg,imperative 1pl,imperative 2pl,group
817,circoncire,circoncisant,circoncis,circoncis,circoncis,circoncit,circoncisons,circoncisez,circoncisent,circoncisais,circoncisais,circoncisait,circoncisions,circoncisiez,circoncisaient,circoncis,circoncis,circoncit,circoncîmes,circoncîtes,circoncirent,circoncirai,circonciras,circoncira,circoncirons,circoncirez,circonciront,circoncirais,circoncirais,circoncirait,circoncirions,circonciriez,circonciraient,circoncise,circoncises,circoncise,circoncisions,circoncisiez,circoncisent,circoncisisse,circoncisisses,circoncisît,circoncisissions,circoncisissiez,circoncisissent,circoncis,circoncisons,circoncisez,None


### -ore

#### enclore

In [36]:
like_enclore, remaining_data = is_like("enclore", "ore", "-clore")
display(like_enclore)

,infinitive,present participle,past participle,present 1sg,present 2sg,present 3sg,present 1pl,present 2pl,present 3pl,imperfect 1sg,imperfect 2sg,imperfect 3sg,imperfect 1pl,imperfect 2pl,imperfect 3pl,past historic 1sg,past historic 2sg,past historic 3sg,past historic 1pl,past historic 2pl,past historic 3pl,future 1sg,future 2sg,future 3sg,future 1pl,future 2pl,future 3pl,conditional 1sg,conditional 2sg,conditional 3sg,conditional 1pl,conditional 2pl,conditional 3pl,subjunctive present 1sg,subjunctive present 2sg,subjunctive present 3sg,subjunctive present 1pl,subjunctive present 2pl,subjunctive present 3pl,subjunctive imperfect 1sg,subjunctive imperfect 2sg,subjunctive imperfect 3sg,subjunctive imperfect 1pl,subjunctive imperfect 2pl,subjunctive imperfect 3pl,imperative 2sg,imperative 1pl,imperative 2pl,group
0,ore,osant,os,os,os,ôt,osons,osez,osent,None,None,None,None,None,None,None,None,None,None,None,None,orai,oras,ora,orons,orez,oront,orais,orais,orait,orions,oriez,oraient,ose,oses,ose,osions,osiez,osent,None,None,None,None,None,None,os,osons,osez,None


,infinitive,present participle,past participle,present 1sg,present 2sg,present 3sg,present 1pl,present 2pl,present 3pl,imperfect 1sg,imperfect 2sg,imperfect 3sg,imperfect 1pl,imperfect 2pl,imperfect 3pl,past historic 1sg,past historic 2sg,past historic 3sg,past historic 1pl,past historic 2pl,past historic 3pl,future 1sg,future 2sg,future 3sg,future 1pl,future 2pl,future 3pl,conditional 1sg,conditional 2sg,conditional 3sg,conditional 1pl,conditional 2pl,conditional 3pl,subjunctive present 1sg,subjunctive present 2sg,subjunctive present 3sg,subjunctive present 1pl,subjunctive present 2pl,subjunctive present 3pl,subjunctive imperfect 1sg,subjunctive imperfect 2sg,subjunctive imperfect 3sg,subjunctive imperfect 1pl,subjunctive imperfect 2pl,subjunctive imperfect 3pl,imperative 2sg,imperative 1pl,imperative 2pl,group
853,clore,closant,clos,clos,clos,clôt,None,None,closent,None,None,None,None,None,None,None,None,None,None,None,None,clorai,cloras,clora,clorons,clorez,cloront,clorais,clorais,clorait,clorions,cloriez,cloraient,close,closes,close,closions,closiez,closent,None,None,None,None,None,None,clos,None,None,None
1411,déclore,déclosant,déclos,déclos,déclos,déclôt,None,None,déclosent,None,None,None,None,None,None,None,None,None,None,None,None,déclorai,décloras,déclora,déclorons,déclorez,décloront,déclorais,déclorais,déclorait,déclorions,décloriez,décloraient,déclose,décloses,déclose,déclosions,déclosiez,déclosent,None,None,None,None,None,None,déclos,None,None,None
1963,enclore,enclosant,enclos,enclos,enclos,enclôt,enclosons,enclosez,enclosent,None,None,None,None,None,None,None,None,None,None,None,None,enclorai,encloras,enclora,enclorons,enclorez,encloront,enclorais,enclorais,enclorait,enclorions,encloriez,encloraient,enclose,encloses,enclose,enclosions,enclosiez,enclosent,None,None,None,None,None,None,enclos,enclosons,enclosez,None
2357,forclore,forclosant,forclos,forclos,forclos,forclôt,None,None,forclosent,None,None,None,None,None,None,None,None,None,None,None,None,forclorai,forcloras,forclora,forclorons,forclorez,forcloront,forclorais,forclorais,forclorait,forclorions,forcloriez,forcloraient,forclose,forcloses,forclose,forclosions,forclosiez,forclosent,None,None,None,None,None,None,forclos,None,None,None
5150,éclore,éclosant,éclos,éclos,éclos,éclôt,None,None,éclosent,None,None,None,None,None,None,None,None,None,None,None,None,éclorai,écloras,éclora,éclorons,éclorez,écloront,éclorais,éclorais,éclorait,éclorions,écloriez,écloraient,éclose,écloses,éclose,éclosions,éclosiez,éclosent,None,None,None,None,None,None,éclos,None,None,None


### -ure

#### conclure

In [37]:
like_conclure, remaining_data = is_like("conclure", "ure", "-clure (conclure/exclure)")
display(like_conclure)

,infinitive,present participle,past participle,present 1sg,present 2sg,present 3sg,present 1pl,present 2pl,present 3pl,imperfect 1sg,imperfect 2sg,imperfect 3sg,imperfect 1pl,imperfect 2pl,imperfect 3pl,past historic 1sg,past historic 2sg,past historic 3sg,past historic 1pl,past historic 2pl,past historic 3pl,future 1sg,future 2sg,future 3sg,future 1pl,future 2pl,future 3pl,conditional 1sg,conditional 2sg,conditional 3sg,conditional 1pl,conditional 2pl,conditional 3pl,subjunctive present 1sg,subjunctive present 2sg,subjunctive present 3sg,subjunctive present 1pl,subjunctive present 2pl,subjunctive present 3pl,subjunctive imperfect 1sg,subjunctive imperfect 2sg,subjunctive imperfect 3sg,subjunctive imperfect 1pl,subjunctive imperfect 2pl,subjunctive imperfect 3pl,imperative 2sg,imperative 1pl,imperative 2pl,group
0,ure,uant,u,us,us,ut,uons,uez,uent,uais,uais,uait,uions,uiez,uaient,us,us,ut,ûmes,ûtes,urent,urai,uras,ura,urons,urez,uront,urais,urais,urait,urions,uriez,uraient,ue,ues,ue,uions,uiez,uent,usse,usses,ût,ussions,ussiez,ussent,us,uons,uez,None


,infinitive,present participle,past participle,present 1sg,present 2sg,present 3sg,present 1pl,present 2pl,present 3pl,imperfect 1sg,imperfect 2sg,imperfect 3sg,imperfect 1pl,imperfect 2pl,imperfect 3pl,past historic 1sg,past historic 2sg,past historic 3sg,past historic 1pl,past historic 2pl,past historic 3pl,future 1sg,future 2sg,future 3sg,future 1pl,future 2pl,future 3pl,conditional 1sg,conditional 2sg,conditional 3sg,conditional 1pl,conditional 2pl,conditional 3pl,subjunctive present 1sg,subjunctive present 2sg,subjunctive present 3sg,subjunctive present 1pl,subjunctive present 2pl,subjunctive present 3pl,subjunctive imperfect 1sg,subjunctive imperfect 2sg,subjunctive imperfect 3sg,subjunctive imperfect 1pl,subjunctive imperfect 2pl,subjunctive imperfect 3pl,imperative 2sg,imperative 1pl,imperative 2pl,group
949,conclure,concluant,conclu,conclus,conclus,conclut,concluons,concluez,concluent,concluais,concluais,concluait,concluions,concluiez,concluaient,conclus,conclus,conclut,conclûmes,conclûtes,conclurent,conclurai,concluras,conclura,conclurons,conclurez,concluront,conclurais,conclurais,conclurait,conclurions,concluriez,concluraient,conclue,conclues,conclue,concluions,concluiez,concluent,conclusse,conclusses,conclût,conclussions,conclussiez,conclussent,conclus,concluons,concluez,None
2177,exclure,excluant,exclu,exclus,exclus,exclut,excluons,excluez,excluent,excluais,excluais,excluait,excluions,excluiez,excluaient,exclus,exclus,exclut,exclûmes,exclûtes,exclurent,exclurai,excluras,exclura,exclurons,exclurez,excluront,exclurais,exclurais,exclurait,exclurions,excluriez,excluraient,exclue,exclues,exclue,excluions,excluiez,excluent,exclusse,exclusses,exclût,exclussions,exclussiez,exclussent,exclus,excluons,excluez,None


#### inclure

In [38]:
like_inclure, remaining_data = is_like("inclure", "ure", "-clure (inclure/occlure/transclure)")
display(like_inclure)

,infinitive,present participle,past participle,present 1sg,present 2sg,present 3sg,present 1pl,present 2pl,present 3pl,imperfect 1sg,imperfect 2sg,imperfect 3sg,imperfect 1pl,imperfect 2pl,imperfect 3pl,past historic 1sg,past historic 2sg,past historic 3sg,past historic 1pl,past historic 2pl,past historic 3pl,future 1sg,future 2sg,future 3sg,future 1pl,future 2pl,future 3pl,conditional 1sg,conditional 2sg,conditional 3sg,conditional 1pl,conditional 2pl,conditional 3pl,subjunctive present 1sg,subjunctive present 2sg,subjunctive present 3sg,subjunctive present 1pl,subjunctive present 2pl,subjunctive present 3pl,subjunctive imperfect 1sg,subjunctive imperfect 2sg,subjunctive imperfect 3sg,subjunctive imperfect 1pl,subjunctive imperfect 2pl,subjunctive imperfect 3pl,imperative 2sg,imperative 1pl,imperative 2pl,group
0,ure,uant,us,us,us,ut,uons,uez,uent,uais,uais,uait,uions,uiez,uaient,us,us,ut,ûmes,ûtes,urent,urai,uras,ura,urons,urez,uront,urais,urais,urait,urions,uriez,uraient,ue,ues,ue,uions,uiez,uent,usse,usses,ût,ussions,ussiez,ussent,us,uons,uez,None


,infinitive,present participle,past participle,present 1sg,present 2sg,present 3sg,present 1pl,present 2pl,present 3pl,imperfect 1sg,imperfect 2sg,imperfect 3sg,imperfect 1pl,imperfect 2pl,imperfect 3pl,past historic 1sg,past historic 2sg,past historic 3sg,past historic 1pl,past historic 2pl,past historic 3pl,future 1sg,future 2sg,future 3sg,future 1pl,future 2pl,future 3pl,conditional 1sg,conditional 2sg,conditional 3sg,conditional 1pl,conditional 2pl,conditional 3pl,subjunctive present 1sg,subjunctive present 2sg,subjunctive present 3sg,subjunctive present 1pl,subjunctive present 2pl,subjunctive present 3pl,subjunctive imperfect 1sg,subjunctive imperfect 2sg,subjunctive imperfect 3sg,subjunctive imperfect 1pl,subjunctive imperfect 2pl,subjunctive imperfect 3pl,imperative 2sg,imperative 1pl,imperative 2pl,group
2722,inclure,incluant,inclus,inclus,inclus,inclut,incluons,incluez,incluent,incluais,incluais,incluait,incluions,incluiez,incluaient,inclus,inclus,inclut,inclûmes,inclûtes,inclurent,inclurai,incluras,inclura,inclurons,inclurez,incluront,inclurais,inclurais,inclurait,inclurions,incluriez,incluraient,inclue,inclues,inclue,incluions,incluiez,incluent,inclusse,inclusses,inclût,inclussions,inclussiez,inclussent,inclus,incluons,incluez,None
3266,occlure,occluant,occlus,occlus,occlus,occlut,occluons,occluez,occluent,occluais,occluais,occluait,occluions,occluiez,occluaient,occlus,occlus,occlut,occlûmes,occlûtes,occlurent,occlurai,occluras,occlura,occlurons,occlurez,occluront,occlurais,occlurais,occlurait,occlurions,occluriez,occluraient,occlue,occlues,occlue,occluions,occluiez,occluent,occlusse,occlusses,occlût,occlussions,occlussiez,occlussent,occlus,occluons,occluez,None
4863,transclure,transcluant,transclus,transclus,transclus,transclut,transcluons,transcluez,transcluent,transcluais,transcluais,transcluait,transcluions,transcluiez,transcluaient,transclus,transclus,transclut,transclûmes,transclûtes,transclurent,transclurai,transcluras,transclura,transclurons,transclurez,transcluront,transclurais,transclurais,transclurait,transclurions,transcluriez,transcluraient,transclue,transclues,transclue,transcluions,transcluiez,transcluent,transclusse,transclusses,transclût,transclussions,transclussiez,transclussent,transclus,transcluons,transcluez,None


### -dre

#### peindre, joindre...

In [39]:
def peindre_modify_root(root: str, ending: str, column: str):
    should_use_gn = column in ["present participle", "present 1pl", "present 2pl", "present 3pl", "present 3pl", "imperative 1pl", "imperative 2pl"] or column.startswith("imperfect") or column.startswith("past historic") or column.startswith("subjunctive present") or column.startswith("subjunctive imperfect")
    if ending is not None:
        ending_starts_with_vowel = any(ending.startswith(l) for l in "aeiouyâîû")
        assert should_use_gn == ending_starts_with_vowel
    if should_use_gn:
        return f"{root.removesuffix("n")}gn"
    return root

like_peindre, remaining_data = is_like("peindre", "dre", (peindre_modify_root, "-indre"))
display(like_peindre)

,infinitive,present participle,past participle,present 1sg,present 2sg,present 3sg,present 1pl,present 2pl,present 3pl,imperfect 1sg,imperfect 2sg,imperfect 3sg,imperfect 1pl,imperfect 2pl,imperfect 3pl,past historic 1sg,past historic 2sg,past historic 3sg,past historic 1pl,past historic 2pl,past historic 3pl,future 1sg,future 2sg,future 3sg,future 1pl,future 2pl,future 3pl,conditional 1sg,conditional 2sg,conditional 3sg,conditional 1pl,conditional 2pl,conditional 3pl,subjunctive present 1sg,subjunctive present 2sg,subjunctive present 3sg,subjunctive present 1pl,subjunctive present 2pl,subjunctive present 3pl,subjunctive imperfect 1sg,subjunctive imperfect 2sg,subjunctive imperfect 3sg,subjunctive imperfect 1pl,subjunctive imperfect 2pl,subjunctive imperfect 3pl,imperative 2sg,imperative 1pl,imperative 2pl,group
0,dre,ant,t,s,s,t,ons,ez,ent,ais,ais,ait,ions,iez,aient,is,is,it,îmes,îtes,irent,drai,dras,dra,drons,drez,dront,drais,drais,drait,drions,driez,draient,e,es,e,ions,iez,ent,isse,isses,ît,issions,issiez,issent,s,ons,ez,None


,infinitive,present participle,past participle,present 1sg,present 2sg,present 3sg,present 1pl,present 2pl,present 3pl,imperfect 1sg,imperfect 2sg,imperfect 3sg,imperfect 1pl,imperfect 2pl,imperfect 3pl,past historic 1sg,past historic 2sg,past historic 3sg,past historic 1pl,past historic 2pl,past historic 3pl,future 1sg,future 2sg,future 3sg,future 1pl,future 2pl,future 3pl,conditional 1sg,conditional 2sg,conditional 3sg,conditional 1pl,conditional 2pl,conditional 3pl,subjunctive present 1sg,subjunctive present 2sg,subjunctive present 3sg,subjunctive present 1pl,subjunctive present 2pl,subjunctive present 3pl,subjunctive imperfect 1sg,subjunctive imperfect 2sg,subjunctive imperfect 3sg,subjunctive imperfect 1pl,subjunctive imperfect 2pl,subjunctive imperfect 3pl,imperative 2sg,imperative 1pl,imperative 2pl,group
107,adjoindre,adjoignant,adjoint,adjoins,adjoins,adjoint,adjoignons,adjoignez,adjoignent,adjoignais,adjoignais,adjoignait,adjoignions,adjoigniez,adjoignaient,adjoignis,adjoignis,adjoignit,adjoignîmes,adjoignîtes,adjoignirent,adjoindrai,adjoindras,adjoindra,adjoindrons,adjoindrez,adjoindront,adjoindrais,adjoindrais,adjoindrait,adjoindrions,adjoindriez,adjoindraient,adjoigne,adjoignes,adjoigne,adjoignions,adjoigniez,adjoignent,adjoignisse,adjoignisses,adjoignît,adjoignissions,adjoignissiez,adjoignissent,adjoins,adjoignons,adjoignez,None
361,astreindre,astreignant,astreint,astreins,astreins,astreint,astreignons,astreignez,astreignent,astreignais,astreignais,astreignait,astreignions,astreigniez,astreignaient,astreignis,astreignis,astreignit,astreignîmes,astreignîtes,astreignirent,astreindrai,astreindras,astreindra,astreindrons,astreindrez,astreindront,astreindrais,astreindrais,astreindrait,astreindrions,astreindriez,astreindraient,astreigne,astreignes,astreigne,astreignions,astreigniez,astreignent,astreignisse,astreignisses,astreignît,astreignissions,astreignissiez,astreignissent,astreins,astreignons,astreignez,None
368,atteindre,atteignant,atteint,atteins,atteins,atteint,atteignons,atteignez,atteignent,atteignais,atteignais,atteignait,atteignions,atteigniez,atteignaient,atteignis,atteignis,atteignit,atteignîmes,atteignîtes,atteignirent,atteindrai,atteindras,atteindra,atteindrons,atteindrez,atteindront,atteindrais,atteindrais,atteindrait,atteindrions,atteindriez,atteindraient,atteigne,atteignes,atteigne,atteignions,atteigniez,atteignent,atteignisse,atteignisses,atteignît,atteignissions,atteignissiez,atteignissent,atteins,atteignons,atteignez,None
729,ceindre,ceignant,ceint,ceins,ceins,ceint,ceignons,ceignez,ceignent,ceignais,ceignais,ceignait,ceignions,ceigniez,ceignaient,ceignis,ceignis,ceignit,ceignîmes,ceignîtes,ceignirent,ceindrai,ceindras,ceindra,ceindrons,ceindrez,ceindront,ceindrais,ceindrais,ceindrait,ceindrions,ceindriez,ceindraient,ceigne,ceignes,ceigne,ceignions,ceigniez,ceignent,ceignisse,ceignisses,ceignît,ceignissions,ceignissiez,ceignissent,ceins,ceignons,ceignez,None
978,conjoindre,conjoignant,conjoint,conjoins,conjoins,conjoint,conjoignons,conjoignez,conjoignent,conjoignais,conjoignais,conjoignait,conjoignions,conjoigniez,conjoignaient,conjoignis,conjoignis,conjoignit,conjoignîmes,conjoignîtes,conjoignirent,conjoindrai,conjoindras,conjoindra,conjoindrons,conjoindrez,conjoindront,conjoindrais,conjoindrais,conjoindrait,conjoindrions,conjoindriez,conjoindraient,conjoigne,conjoignes,conjoigne,conjoignions,conjoigniez,conjoignent,conjoignisse,conjoignisses,conjoignît,conjoignissions,conjoignissiez,conjoignissent,conjoins,conjoignons,conjoignez,None
1019,contraindre,contraignant,contraint,contrains,contrains,contraint,contraignons,contraignez,contraignent,contraignais,contraignais,contraignait,contraignions,contraigniez,contraignaient,contraignis,contraignis,contraignit,contraignîmes,contraignîtes,contraignirent,contraindrai,contraindras,contraindra,contraindrons,contraindrez,contraindront,contraindrais,contraindrais,contraindrait,contraindrions,contraindriez,contraindraient,contraigne,contraignes,contraigne,con

#### moudre

In [40]:
like_moudre, remaining_data = is_like("moudre", "dre", "-moudre")
display(like_moudre)

,infinitive,present participle,past participle,present 1sg,present 2sg,present 3sg,present 1pl,present 2pl,present 3pl,imperfect 1sg,imperfect 2sg,imperfect 3sg,imperfect 1pl,imperfect 2pl,imperfect 3pl,past historic 1sg,past historic 2sg,past historic 3sg,past historic 1pl,past historic 2pl,past historic 3pl,future 1sg,future 2sg,future 3sg,future 1pl,future 2pl,future 3pl,conditional 1sg,conditional 2sg,conditional 3sg,conditional 1pl,conditional 2pl,conditional 3pl,subjunctive present 1sg,subjunctive present 2sg,subjunctive present 3sg,subjunctive present 1pl,subjunctive present 2pl,subjunctive present 3pl,subjunctive imperfect 1sg,subjunctive imperfect 2sg,subjunctive imperfect 3sg,subjunctive imperfect 1pl,subjunctive imperfect 2pl,subjunctive imperfect 3pl,imperative 2sg,imperative 1pl,imperative 2pl,group
0,dre,lant,lu,ds,ds,d,lons,lez,lent,lais,lais,lait,lions,liez,laient,lus,lus,lut,lûmes,lûtes,lurent,drai,dras,dra,drons,drez,dront,drais,drais,drait,drions,driez,draient,le,les,le,lions,liez,lent,lusse,lusses,lût,lussions,lussiez,lussent,ds,lons,lez,None


,infinitive,present participle,past participle,present 1sg,present 2sg,present 3sg,present 1pl,present 2pl,present 3pl,imperfect 1sg,imperfect 2sg,imperfect 3sg,imperfect 1pl,imperfect 2pl,imperfect 3pl,past historic 1sg,past historic 2sg,past historic 3sg,past historic 1pl,past historic 2pl,past historic 3pl,future 1sg,future 2sg,future 3sg,future 1pl,future 2pl,future 3pl,conditional 1sg,conditional 2sg,conditional 3sg,conditional 1pl,conditional 2pl,conditional 3pl,subjunctive present 1sg,subjunctive present 2sg,subjunctive present 3sg,subjunctive present 1pl,subjunctive present 2pl,subjunctive present 3pl,subjunctive imperfect 1sg,subjunctive imperfect 2sg,subjunctive imperfect 3sg,subjunctive imperfect 1pl,subjunctive imperfect 2pl,subjunctive imperfect 3pl,imperative 2sg,imperative 1pl,imperative 2pl,group
3124,moudre,moulant,moulu,mouds,mouds,moud,moulons,moulez,moulent,moulais,moulais,moulait,moulions,mouliez,moulaient,moulus,moulus,moulut,moulûmes,moulûtes,moulurent,moudrai,moudras,moudra,moudrons,moudrez,moudront,moudrais,moudrais,moudrait,moudrions,moudriez,moudraient,moule,moules,moule,moulions,mouliez,moulent,moulusse,moulusses,moulût,moulussions,moulussiez,moulussent,mouds,moulons,moulez,None
3993,remoudre,remoulant,remoulu,remouds,remouds,remoud,remoulons,remoulez,remoulent,remoulais,remoulais,remoulait,remoulions,remouliez,remoulaient,remoulus,remoulus,remoulut,remoulûmes,remoulûtes,remoulurent,remoudrai,remoudras,remoudra,remoudrons,remoudrez,remoudront,remoudrais,remoudrais,remoudrait,remoudrions,remoudriez,remoudraient,remoule,remoules,remoule,remoulions,remouliez,remoulent,remoulusse,remoulusses,remoulût,remoulussions,remoulussiez,remoulussent,remouds,remoulons,remoulez,None
5220,émoudre,émoulant,émoulu,émouds,émouds,émoud,émoulons,émoulez,émoulent,émoulais,émoulais,émoulait,émoulions,émouliez,émoulaient,émoulus,émoulus,émoulut,émoulûmes,émoulûtes,émoulurent,émoudrai,émoudras,émoudra,émoudrons,émoudrez,émoudront,émoudrais,émoudrais,émoudrait,émoudrions,émoudriez,émoudraient,émoule,émoules,émoule,émoulions,émouliez,émoulent,émoulusse,émoulusses,émoulût,émoulussions,émoulussiez,émoulussent,émouds,émoulons,émoulez,None


#### absoudre

In [41]:
like_absoudre, remaining_data = is_like("absoudre", "udre", "-soudre (except résoudre)")
display(like_absoudre)

,infinitive,present participle,past participle,present 1sg,present 2sg,present 3sg,present 1pl,present 2pl,present 3pl,imperfect 1sg,imperfect 2sg,imperfect 3sg,imperfect 1pl,imperfect 2pl,imperfect 3pl,past historic 1sg,past historic 2sg,past historic 3sg,past historic 1pl,past historic 2pl,past historic 3pl,future 1sg,future 2sg,future 3sg,future 1pl,future 2pl,future 3pl,conditional 1sg,conditional 2sg,conditional 3sg,conditional 1pl,conditional 2pl,conditional 3pl,subjunctive present 1sg,subjunctive present 2sg,subjunctive present 3sg,subjunctive present 1pl,subjunctive present 2pl,subjunctive present 3pl,subjunctive imperfect 1sg,subjunctive imperfect 2sg,subjunctive imperfect 3sg,subjunctive imperfect 1pl,subjunctive imperfect 2pl,subjunctive imperfect 3pl,imperative 2sg,imperative 1pl,imperative 2pl,group
0,udre,lvant,"(us, ut)",us,us,ut,lvons,lvez,lvent,lvais,lvais,lvait,lvions,lviez,lvaient,None,None,None,None,None,None,udrai,udras,udra,udrons,udrez,udront,udrais,udrais,udrait,udrions,udriez,udraient,lve,lves,lve,lvions,lviez,lvent,None,None,None,None,None,None,us,lvons,lvez,None


,infinitive,present participle,past participle,present 1sg,present 2sg,present 3sg,present 1pl,present 2pl,present 3pl,imperfect 1sg,imperfect 2sg,imperfect 3sg,imperfect 1pl,imperfect 2pl,imperfect 3pl,past historic 1sg,past historic 2sg,past historic 3sg,past historic 1pl,past historic 2pl,past historic 3pl,future 1sg,future 2sg,future 3sg,future 1pl,future 2pl,future 3pl,conditional 1sg,conditional 2sg,conditional 3sg,conditional 1pl,conditional 2pl,conditional 3pl,subjunctive present 1sg,subjunctive present 2sg,subjunctive present 3sg,subjunctive present 1pl,subjunctive present 2pl,subjunctive present 3pl,subjunctive imperfect 1sg,subjunctive imperfect 2sg,subjunctive imperfect 3sg,subjunctive imperfect 1pl,subjunctive imperfect 2pl,subjunctive imperfect 3pl,imperative 2sg,imperative 1pl,imperative 2pl,group
42,absoudre,absolvant,"(absous, absout)",absous,absous,absout,absolvons,absolvez,absolvent,absolvais,absolvais,absolvait,absolvions,absolviez,absolvaient,None,None,None,None,None,None,absoudrai,absoudras,absoudra,absoudrons,absoudrez,absoudront,absoudrais,absoudrais,absoudrait,absoudrions,absoudriez,absoudraient,absolve,absolves,absolve,absolvions,absolviez,absolvent,None,None,None,None,None,None,absous,absolvons,absolvez,None
1257,dissoudre,dissolvant,"(dissous, dissout)",dissous,dissous,dissout,dissolvons,dissolvez,dissolvent,dissolvais,dissolvais,dissolvait,dissolvions,dissolviez,dissolvaient,None,None,None,None,None,None,dissoudrai,dissoudras,dissoudra,dissoudrons,dissoudrez,dissoudront,dissoudrais,dissoudrais,dissoudrait,dissoudrions,dissoudriez,dissoudraient,dissolve,dissolves,dissolve,dissolvions,dissolviez,dissolvent,None,None,None,None,None,None,dissous,dissolvons,dissolvez,None


#### résoudre

In [42]:
like_resoudre, remaining_data = is_like("résoudre", "udre", "résoudre")
display(like_resoudre)

,infinitive,present participle,past participle,present 1sg,present 2sg,present 3sg,present 1pl,present 2pl,present 3pl,imperfect 1sg,imperfect 2sg,imperfect 3sg,imperfect 1pl,imperfect 2pl,imperfect 3pl,past historic 1sg,past historic 2sg,past historic 3sg,past historic 1pl,past historic 2pl,past historic 3pl,future 1sg,future 2sg,future 3sg,future 1pl,future 2pl,future 3pl,conditional 1sg,conditional 2sg,conditional 3sg,conditional 1pl,conditional 2pl,conditional 3pl,subjunctive present 1sg,subjunctive present 2sg,subjunctive present 3sg,subjunctive present 1pl,subjunctive present 2pl,subjunctive present 3pl,subjunctive imperfect 1sg,subjunctive imperfect 2sg,subjunctive imperfect 3sg,subjunctive imperfect 1pl,subjunctive imperfect 2pl,subjunctive imperfect 3pl,imperative 2sg,imperative 1pl,imperative 2pl,group
0,udre,lvant,lu,us,us,ut,lvons,lvez,lvent,lvais,lvais,lvait,lvions,lviez,lvaient,lus,lus,lut,lûmes,lûtes,lurent,udrai,udras,udra,udrons,udrez,udront,udrais,udrais,udrait,udrions,udriez,udraient,lve,lves,lve,lvions,lviez,lvent,lusse,lusses,lût,lussions,lussiez,lussent,us,lvons,lvez,None


,infinitive,present participle,past participle,present 1sg,present 2sg,present 3sg,present 1pl,present 2pl,present 3pl,imperfect 1sg,imperfect 2sg,imperfect 3sg,imperfect 1pl,imperfect 2pl,imperfect 3pl,past historic 1sg,past historic 2sg,past historic 3sg,past historic 1pl,past historic 2pl,past historic 3pl,future 1sg,future 2sg,future 3sg,future 1pl,future 2pl,future 3pl,conditional 1sg,conditional 2sg,conditional 3sg,conditional 1pl,conditional 2pl,conditional 3pl,subjunctive present 1sg,subjunctive present 2sg,subjunctive present 3sg,subjunctive present 1pl,subjunctive present 2pl,subjunctive present 3pl,subjunctive imperfect 1sg,subjunctive imperfect 2sg,subjunctive imperfect 3sg,subjunctive imperfect 1pl,subjunctive imperfect 2pl,subjunctive imperfect 3pl,imperative 2sg,imperative 1pl,imperative 2pl,group
4357,résoudre,résolvant,résolu,résous,résous,résout,résolvons,résolvez,résolvent,résolvais,résolvais,résolvait,résolvions,résolviez,résolvaient,résolus,résolus,résolut,résolûmes,résolûtes,résolurent,résoudrai,résoudras,résoudra,résoudrons,résoudrez,résoudront,résoudrais,résoudrais,résoudrait,résoudrions,résoudriez,résoudraient,résolve,résolves,résolve,résolvions,résolviez,résolvent,résolusse,résolusses,résolût,résolussions,résolussiez,résolussent,résous,résolvons,résolvez,None


### Highly irregular verbs

In [43]:
def get_by_root(root: str, exact: bool):
    group = data[data["infinitive"].str.endswith(root) if not exact else (data["infinitive"] == root)]
    return group, data[~data.index.isin(group.index)]

#### être

In [44]:
like_etre, remaining_data = is_like("être", "être", "être")
display(like_etre)

,infinitive,present participle,past participle,present 1sg,present 2sg,present 3sg,present 1pl,present 2pl,present 3pl,imperfect 1sg,imperfect 2sg,imperfect 3sg,imperfect 1pl,imperfect 2pl,imperfect 3pl,past historic 1sg,past historic 2sg,past historic 3sg,past historic 1pl,past historic 2pl,past historic 3pl,future 1sg,future 2sg,future 3sg,future 1pl,future 2pl,future 3pl,conditional 1sg,conditional 2sg,conditional 3sg,conditional 1pl,conditional 2pl,conditional 3pl,subjunctive present 1sg,subjunctive present 2sg,subjunctive present 3sg,subjunctive present 1pl,subjunctive present 2pl,subjunctive present 3pl,subjunctive imperfect 1sg,subjunctive imperfect 2sg,subjunctive imperfect 3sg,subjunctive imperfect 1pl,subjunctive imperfect 2pl,subjunctive imperfect 3pl,imperative 2sg,imperative 1pl,imperative 2pl,group
0,être,étant,été,suis,es,est,sommes,êtes,sont,étais,étais,était,étions,étiez,étaient,fus,fus,fut,fûmes,fûtes,furent,serai,seras,sera,serons,serez,seront,serais,serais,serait,serions,seriez,seraient,sois,sois,soit,soyons,soyez,soient,fusse,fusses,fût,fussions,fussiez,fussent,sois,soyons,soyez,None


,infinitive,present participle,past participle,present 1sg,present 2sg,present 3sg,present 1pl,present 2pl,present 3pl,imperfect 1sg,imperfect 2sg,imperfect 3sg,imperfect 1pl,imperfect 2pl,imperfect 3pl,past historic 1sg,past historic 2sg,past historic 3sg,past historic 1pl,past historic 2pl,past historic 3pl,future 1sg,future 2sg,future 3sg,future 1pl,future 2pl,future 3pl,conditional 1sg,conditional 2sg,conditional 3sg,conditional 1pl,conditional 2pl,conditional 3pl,subjunctive present 1sg,subjunctive present 2sg,subjunctive present 3sg,subjunctive present 1pl,subjunctive present 2pl,subjunctive present 3pl,subjunctive imperfect 1sg,subjunctive imperfect 2sg,subjunctive imperfect 3sg,subjunctive imperfect 1pl,subjunctive imperfect 2pl,subjunctive imperfect 3pl,imperative 2sg,imperative 1pl,imperative 2pl,group
5334,être,étant,été,suis,es,est,sommes,êtes,sont,étais,étais,était,étions,étiez,étaient,fus,fus,fut,fûmes,fûtes,furent,serai,seras,sera,serons,serez,seront,serais,serais,serait,serions,seriez,seraient,sois,sois,soit,soyons,soyez,soient,fusse,fusses,fût,fussions,fussiez,fussent,sois,soyons,soyez,None


#### avoir

In [45]:
like_avoir, remaining_data = is_like("avoir", "avoir", "avoir")
display(like_avoir)

,infinitive,present participle,past participle,present 1sg,present 2sg,present 3sg,present 1pl,present 2pl,present 3pl,imperfect 1sg,imperfect 2sg,imperfect 3sg,imperfect 1pl,imperfect 2pl,imperfect 3pl,past historic 1sg,past historic 2sg,past historic 3sg,past historic 1pl,past historic 2pl,past historic 3pl,future 1sg,future 2sg,future 3sg,future 1pl,future 2pl,future 3pl,conditional 1sg,conditional 2sg,conditional 3sg,conditional 1pl,conditional 2pl,conditional 3pl,subjunctive present 1sg,subjunctive present 2sg,subjunctive present 3sg,subjunctive present 1pl,subjunctive present 2pl,subjunctive present 3pl,subjunctive imperfect 1sg,subjunctive imperfect 2sg,subjunctive imperfect 3sg,subjunctive imperfect 1pl,subjunctive imperfect 2pl,subjunctive imperfect 3pl,imperative 2sg,imperative 1pl,imperative 2pl,group
0,avoir,ayant,eu,ai,as,a,avons,avez,ont,avais,avais,avait,avions,aviez,avaient,eus,eus,eut,eûmes,eûtes,eurent,aurai,auras,aura,aurons,aurez,auront,aurais,aurais,aurait,aurions,auriez,auraient,aie,aies,ait,ayons,ayez,aient,eusse,eusses,eût,eussions,eussiez,eussent,aie,ayons,ayez,None


,infinitive,present participle,past participle,present 1sg,present 2sg,present 3sg,present 1pl,present 2pl,present 3pl,imperfect 1sg,imperfect 2sg,imperfect 3sg,imperfect 1pl,imperfect 2pl,imperfect 3pl,past historic 1sg,past historic 2sg,past historic 3sg,past historic 1pl,past historic 2pl,past historic 3pl,future 1sg,future 2sg,future 3sg,future 1pl,future 2pl,future 3pl,conditional 1sg,conditional 2sg,conditional 3sg,conditional 1pl,conditional 2pl,conditional 3pl,subjunctive present 1sg,subjunctive present 2sg,subjunctive present 3sg,subjunctive present 1pl,subjunctive present 2pl,subjunctive present 3pl,subjunctive imperfect 1sg,subjunctive imperfect 2sg,subjunctive imperfect 3sg,subjunctive imperfect 1pl,subjunctive imperfect 2pl,subjunctive imperfect 3pl,imperative 2sg,imperative 1pl,imperative 2pl,group
414,avoir,ayant,eu,ai,as,a,avons,avez,ont,avais,avais,avait,avions,aviez,avaient,eus,eus,eut,eûmes,eûtes,eurent,aurai,auras,aura,aurons,aurez,auront,aurais,aurais,aurait,aurions,auriez,auraient,aie,aies,ait,ayons,ayez,aient,eusse,eusses,eût,eussions,eussiez,eussent,aie,ayons,ayez,None


#### aller

In [46]:
like_aller, remaining_data = is_like("aller", "aller", "aller")
display(like_aller)

,infinitive,present participle,past participle,present 1sg,present 2sg,present 3sg,present 1pl,present 2pl,present 3pl,imperfect 1sg,imperfect 2sg,imperfect 3sg,imperfect 1pl,imperfect 2pl,imperfect 3pl,past historic 1sg,past historic 2sg,past historic 3sg,past historic 1pl,past historic 2pl,past historic 3pl,future 1sg,future 2sg,future 3sg,future 1pl,future 2pl,future 3pl,conditional 1sg,conditional 2sg,conditional 3sg,conditional 1pl,conditional 2pl,conditional 3pl,subjunctive present 1sg,subjunctive present 2sg,subjunctive present 3sg,subjunctive present 1pl,subjunctive present 2pl,subjunctive present 3pl,subjunctive imperfect 1sg,subjunctive imperfect 2sg,subjunctive imperfect 3sg,subjunctive imperfect 1pl,subjunctive imperfect 2pl,subjunctive imperfect 3pl,imperative 2sg,imperative 1pl,imperative 2pl,group
0,aller,allant,allé,vais,vas,va,allons,allez,vont,allais,allais,allait,allions,alliez,allaient,allai,allas,alla,allâmes,allâtes,allèrent,irai,iras,ira,irons,irez,iront,irais,irais,irait,irions,iriez,iraient,aille,ailles,aille,allions,alliez,aillent,allasse,allasses,allât,allassions,allassiez,allassent,va,allons,allez,None


,infinitive,present participle,past participle,present 1sg,present 2sg,present 3sg,present 1pl,present 2pl,present 3pl,imperfect 1sg,imperfect 2sg,imperfect 3sg,imperfect 1pl,imperfect 2pl,imperfect 3pl,past historic 1sg,past historic 2sg,past historic 3sg,past historic 1pl,past historic 2pl,past historic 3pl,future 1sg,future 2sg,future 3sg,future 1pl,future 2pl,future 3pl,conditional 1sg,conditional 2sg,conditional 3sg,conditional 1pl,conditional 2pl,conditional 3pl,subjunctive present 1sg,subjunctive present 2sg,subjunctive present 3sg,subjunctive present 1pl,subjunctive present 2pl,subjunctive present 3pl,subjunctive imperfect 1sg,subjunctive imperfect 2sg,subjunctive imperfect 3sg,subjunctive imperfect 1pl,subjunctive imperfect 2pl,subjunctive imperfect 3pl,imperative 2sg,imperative 1pl,imperative 2pl,group
196,aller,allant,allé,vais,vas,va,allons,allez,vont,allais,allais,allait,allions,alliez,allaient,allai,allas,alla,allâmes,allâtes,allèrent,irai,iras,ira,irons,irez,iront,irais,irais,irait,irions,iriez,iraient,aille,ailles,aille,allions,alliez,aillent,allasse,allasses,allât,allassions,allassiez,allassent,va,allons,allez,None


#### devoir

In [47]:
like_devoir, remaining_data = is_like("devoir", "devoir", "devoir")
display(like_devoir)

,infinitive,present participle,past participle,present 1sg,present 2sg,present 3sg,present 1pl,present 2pl,present 3pl,imperfect 1sg,imperfect 2sg,imperfect 3sg,imperfect 1pl,imperfect 2pl,imperfect 3pl,past historic 1sg,past historic 2sg,past historic 3sg,past historic 1pl,past historic 2pl,past historic 3pl,future 1sg,future 2sg,future 3sg,future 1pl,future 2pl,future 3pl,conditional 1sg,conditional 2sg,conditional 3sg,conditional 1pl,conditional 2pl,conditional 3pl,subjunctive present 1sg,subjunctive present 2sg,subjunctive present 3sg,subjunctive present 1pl,subjunctive present 2pl,subjunctive present 3pl,subjunctive imperfect 1sg,subjunctive imperfect 2sg,subjunctive imperfect 3sg,subjunctive imperfect 1pl,subjunctive imperfect 2pl,subjunctive imperfect 3pl,imperative 2sg,imperative 1pl,imperative 2pl,group
0,devoir,devant,dû,dois,dois,doit,devons,devez,doivent,devais,devais,devait,devions,deviez,devaient,dus,dus,dut,dûmes,dûtes,durent,devrai,devras,devra,devrons,devrez,devront,devrais,devrais,devrait,devrions,devriez,devraient,doive,doives,doive,devions,deviez,doivent,dusse,dusses,dût,dussions,dussiez,dussent,dois,devons,devez,None


,infinitive,present participle,past participle,present 1sg,present 2sg,present 3sg,present 1pl,present 2pl,present 3pl,imperfect 1sg,imperfect 2sg,imperfect 3sg,imperfect 1pl,imperfect 2pl,imperfect 3pl,past historic 1sg,past historic 2sg,past historic 3sg,past historic 1pl,past historic 2pl,past historic 3pl,future 1sg,future 2sg,future 3sg,future 1pl,future 2pl,future 3pl,conditional 1sg,conditional 2sg,conditional 3sg,conditional 1pl,conditional 2pl,conditional 3pl,subjunctive present 1sg,subjunctive present 2sg,subjunctive present 3sg,subjunctive present 1pl,subjunctive present 2pl,subjunctive present 3pl,subjunctive imperfect 1sg,subjunctive imperfect 2sg,subjunctive imperfect 3sg,subjunctive imperfect 1pl,subjunctive imperfect 2pl,subjunctive imperfect 3pl,imperative 2sg,imperative 1pl,imperative 2pl,group
1205,devoir,devant,dû,dois,dois,doit,devons,devez,doivent,devais,devais,devait,devions,deviez,devaient,dus,dus,dut,dûmes,dûtes,durent,devrai,devras,devra,devrons,devrez,devront,devrais,devrais,devrait,devrions,devriez,devraient,doive,doives,doive,devions,deviez,doivent,dusse,dusses,dût,dussions,dussiez,dussent,dois,devons,devez,None
3903,redevoir,redevant,redû,redois,redois,redoit,redevons,redevez,redoivent,redevais,redevais,redevait,redevions,redeviez,redevaient,redus,redus,redut,redûmes,redûtes,redurent,redevrai,redevras,redevra,redevrons,redevrez,redevront,redevrais,redevrais,redevrait,redevrions,redevriez,redevraient,redoive,redoives,redoive,redevions,redeviez,redoivent,redusse,redusses,redût,redussions,redussiez,redussent,redois,redevons,redevez,None


#### falloir

In [48]:
like_falloir, remaining_data = is_like("falloir", "falloir", "falloir")
display(like_falloir)

,infinitive,present participle,past participle,present 1sg,present 2sg,present 3sg,present 1pl,present 2pl,present 3pl,imperfect 1sg,imperfect 2sg,imperfect 3sg,imperfect 1pl,imperfect 2pl,imperfect 3pl,past historic 1sg,past historic 2sg,past historic 3sg,past historic 1pl,past historic 2pl,past historic 3pl,future 1sg,future 2sg,future 3sg,future 1pl,future 2pl,future 3pl,conditional 1sg,conditional 2sg,conditional 3sg,conditional 1pl,conditional 2pl,conditional 3pl,subjunctive present 1sg,subjunctive present 2sg,subjunctive present 3sg,subjunctive present 1pl,subjunctive present 2pl,subjunctive present 3pl,subjunctive imperfect 1sg,subjunctive imperfect 2sg,subjunctive imperfect 3sg,subjunctive imperfect 1pl,subjunctive imperfect 2pl,subjunctive imperfect 3pl,imperative 2sg,imperative 1pl,imperative 2pl,group
0,falloir,None,fallu,None,None,faut,None,None,None,None,None,fallait,None,None,None,None,None,fallut,None,None,None,None,None,faudra,None,None,None,None,None,faudrait,None,None,None,None,None,faille,None,None,None,None,None,fallût,None,None,None,None,None,None,None


,infinitive,present participle,past participle,present 1sg,present 2sg,present 3sg,present 1pl,present 2pl,present 3pl,imperfect 1sg,imperfect 2sg,imperfect 3sg,imperfect 1pl,imperfect 2pl,imperfect 3pl,past historic 1sg,past historic 2sg,past historic 3sg,past historic 1pl,past historic 2pl,past historic 3pl,future 1sg,future 2sg,future 3sg,future 1pl,future 2pl,future 3pl,conditional 1sg,conditional 2sg,conditional 3sg,conditional 1pl,conditional 2pl,conditional 3pl,subjunctive present 1sg,subjunctive present 2sg,subjunctive present 3sg,subjunctive present 1pl,subjunctive present 2pl,subjunctive present 3pl,subjunctive imperfect 1sg,subjunctive imperfect 2sg,subjunctive imperfect 3sg,subjunctive imperfect 1pl,subjunctive imperfect 2pl,subjunctive imperfect 3pl,imperative 2sg,imperative 1pl,imperative 2pl,group
2249,falloir,None,fallu,None,None,faut,None,None,None,None,None,fallait,None,None,None,None,None,fallut,None,None,None,None,None,faudra,None,None,None,None,None,faudrait,None,None,None,None,None,faille,None,None,None,None,None,fallût,None,None,None,None,None,None,None


#### savoir

In [49]:
like_savoir, remaining_data = is_like("savoir", "savoir", "savoir")
display(like_savoir)

,infinitive,present participle,past participle,present 1sg,present 2sg,present 3sg,present 1pl,present 2pl,present 3pl,imperfect 1sg,imperfect 2sg,imperfect 3sg,imperfect 1pl,imperfect 2pl,imperfect 3pl,past historic 1sg,past historic 2sg,past historic 3sg,past historic 1pl,past historic 2pl,past historic 3pl,future 1sg,future 2sg,future 3sg,future 1pl,future 2pl,future 3pl,conditional 1sg,conditional 2sg,conditional 3sg,conditional 1pl,conditional 2pl,conditional 3pl,subjunctive present 1sg,subjunctive present 2sg,subjunctive present 3sg,subjunctive present 1pl,subjunctive present 2pl,subjunctive present 3pl,subjunctive imperfect 1sg,subjunctive imperfect 2sg,subjunctive imperfect 3sg,subjunctive imperfect 1pl,subjunctive imperfect 2pl,subjunctive imperfect 3pl,imperative 2sg,imperative 1pl,imperative 2pl,group
0,savoir,sachant,su,sais,sais,sait,savons,savez,savent,savais,savais,savait,savions,saviez,savaient,sus,sus,sut,sûmes,sûtes,surent,saurai,sauras,saura,saurons,saurez,sauront,saurais,saurais,saurait,saurions,sauriez,sauraient,sache,saches,sache,sachions,sachiez,sachent,susse,susses,sût,sussions,sussiez,sussent,sache,sachons,sachez,None


,infinitive,present participle,past participle,present 1sg,present 2sg,present 3sg,present 1pl,present 2pl,present 3pl,imperfect 1sg,imperfect 2sg,imperfect 3sg,imperfect 1pl,imperfect 2pl,imperfect 3pl,past historic 1sg,past historic 2sg,past historic 3sg,past historic 1pl,past historic 2pl,past historic 3pl,future 1sg,future 2sg,future 3sg,future 1pl,future 2pl,future 3pl,conditional 1sg,conditional 2sg,conditional 3sg,conditional 1pl,conditional 2pl,conditional 3pl,subjunctive present 1sg,subjunctive present 2sg,subjunctive present 3sg,subjunctive present 1pl,subjunctive present 2pl,subjunctive present 3pl,subjunctive imperfect 1sg,subjunctive imperfect 2sg,subjunctive imperfect 3sg,subjunctive imperfect 1pl,subjunctive imperfect 2pl,subjunctive imperfect 3pl,imperative 2sg,imperative 1pl,imperative 2pl,group
4439,savoir,sachant,su,sais,sais,sait,savons,savez,savent,savais,savais,savait,savions,saviez,savaient,sus,sus,sut,sûmes,sûtes,surent,saurai,sauras,saura,saurons,saurez,sauront,saurais,saurais,saurait,saurions,sauriez,sauraient,sache,saches,sache,sachions,sachiez,sachent,susse,susses,sût,sussions,sussiez,sussent,sache,sachons,sachez,None


#### pouvoir

In [50]:
like_pouvoir, remaining_data = is_like("pouvoir", "pouvoir", "pouvoir")
display(like_pouvoir)

,infinitive,present participle,past participle,present 1sg,present 2sg,present 3sg,present 1pl,present 2pl,present 3pl,imperfect 1sg,imperfect 2sg,imperfect 3sg,imperfect 1pl,imperfect 2pl,imperfect 3pl,past historic 1sg,past historic 2sg,past historic 3sg,past historic 1pl,past historic 2pl,past historic 3pl,future 1sg,future 2sg,future 3sg,future 1pl,future 2pl,future 3pl,conditional 1sg,conditional 2sg,conditional 3sg,conditional 1pl,conditional 2pl,conditional 3pl,subjunctive present 1sg,subjunctive present 2sg,subjunctive present 3sg,subjunctive present 1pl,subjunctive present 2pl,subjunctive present 3pl,subjunctive imperfect 1sg,subjunctive imperfect 2sg,subjunctive imperfect 3sg,subjunctive imperfect 1pl,subjunctive imperfect 2pl,subjunctive imperfect 3pl,imperative 2sg,imperative 1pl,imperative 2pl,group
0,pouvoir,pouvant,pu,peux,peux,peut,pouvons,pouvez,peuvent,pouvais,pouvais,pouvait,pouvions,pouviez,pouvaient,pus,pus,put,pûmes,pûtes,purent,pourrai,pourras,pourra,pourrons,pourrez,pourront,pourrais,pourrais,pourrait,pourrions,pourriez,pourraient,puisse,puisses,puisse,puissions,puissiez,puissent,pusse,pusses,pût,pussions,pussiez,pussent,None,None,None,None


,infinitive,present participle,past participle,present 1sg,present 2sg,present 3sg,present 1pl,present 2pl,present 3pl,imperfect 1sg,imperfect 2sg,imperfect 3sg,imperfect 1pl,imperfect 2pl,imperfect 3pl,past historic 1sg,past historic 2sg,past historic 3sg,past historic 1pl,past historic 2pl,past historic 3pl,future 1sg,future 2sg,future 3sg,future 1pl,future 2pl,future 3pl,conditional 1sg,conditional 2sg,conditional 3sg,conditional 1pl,conditional 2pl,conditional 3pl,subjunctive present 1sg,subjunctive present 2sg,subjunctive present 3sg,subjunctive present 1pl,subjunctive present 2pl,subjunctive present 3pl,subjunctive imperfect 1sg,subjunctive imperfect 2sg,subjunctive imperfect 3sg,subjunctive imperfect 1pl,subjunctive imperfect 2pl,subjunctive imperfect 3pl,imperative 2sg,imperative 1pl,imperative 2pl,group
3577,pouvoir,pouvant,pu,peux,peux,peut,pouvons,pouvez,peuvent,pouvais,pouvais,pouvait,pouvions,pouviez,pouvaient,pus,pus,put,pûmes,pûtes,purent,pourrai,pourras,pourra,pourrons,pourrez,pourront,pourrais,pourrais,pourrait,pourrions,pourriez,pourraient,puisse,puisses,puisse,puissions,puissiez,puissent,pusse,pusses,pût,pussions,pussiez,pussent,None,None,None,None
4071,repouvoir,repouvant,repu,repeux,repeux,repeut,repouvons,repouvez,repeuvent,repouvais,repouvais,repouvait,repouvions,repouviez,repouvaient,repus,repus,reput,repûmes,repûtes,repurent,repourrai,repourras,repourra,repourrons,repourrez,repourront,repourrais,repourrais,repourrait,repourrions,repourriez,repourraient,repuisse,repuisses,repuisse,repuissions,repuissiez,repuissent,repusse,repusses,repût,repussions,repussiez,repussent,None,None,None,None


#### pleuvoir

In [51]:
like_pleuvoir, remaining_data = is_like("pleuvoir", "pleuvoir", "pleuvoir")
display(like_pleuvoir)

,infinitive,present participle,past participle,present 1sg,present 2sg,present 3sg,present 1pl,present 2pl,present 3pl,imperfect 1sg,imperfect 2sg,imperfect 3sg,imperfect 1pl,imperfect 2pl,imperfect 3pl,past historic 1sg,past historic 2sg,past historic 3sg,past historic 1pl,past historic 2pl,past historic 3pl,future 1sg,future 2sg,future 3sg,future 1pl,future 2pl,future 3pl,conditional 1sg,conditional 2sg,conditional 3sg,conditional 1pl,conditional 2pl,conditional 3pl,subjunctive present 1sg,subjunctive present 2sg,subjunctive present 3sg,subjunctive present 1pl,subjunctive present 2pl,subjunctive present 3pl,subjunctive imperfect 1sg,subjunctive imperfect 2sg,subjunctive imperfect 3sg,subjunctive imperfect 1pl,subjunctive imperfect 2pl,subjunctive imperfect 3pl,imperative 2sg,imperative 1pl,imperative 2pl,group
0,pleuvoir,pleuvant,plu,None,None,pleut,None,None,pleuvent,None,None,pleuvait,None,None,pleuvaient,None,None,plut,None,None,plurent,None,None,pleuvra,None,None,pleuvront,None,None,pleuvrait,None,None,pleuvraient,None,None,pleuve,None,None,pleuvent,None,None,plût,None,None,plussent,None,None,None,None


,infinitive,present participle,past participle,present 1sg,present 2sg,present 3sg,present 1pl,present 2pl,present 3pl,imperfect 1sg,imperfect 2sg,imperfect 3sg,imperfect 1pl,imperfect 2pl,imperfect 3pl,past historic 1sg,past historic 2sg,past historic 3sg,past historic 1pl,past historic 2pl,past historic 3pl,future 1sg,future 2sg,future 3sg,future 1pl,future 2pl,future 3pl,conditional 1sg,conditional 2sg,conditional 3sg,conditional 1pl,conditional 2pl,conditional 3pl,subjunctive present 1sg,subjunctive present 2sg,subjunctive present 3sg,subjunctive present 1pl,subjunctive present 2pl,subjunctive present 3pl,subjunctive imperfect 1sg,subjunctive imperfect 2sg,subjunctive imperfect 3sg,subjunctive imperfect 1pl,subjunctive imperfect 2pl,subjunctive imperfect 3pl,imperative 2sg,imperative 1pl,imperative 2pl,group
3510,pleuvoir,pleuvant,plu,None,None,pleut,None,None,pleuvent,None,None,pleuvait,None,None,pleuvaient,None,None,plut,None,None,plurent,None,None,pleuvra,None,None,pleuvront,None,None,pleuvrait,None,None,pleuvraient,None,None,pleuve,None,None,pleuvent,None,None,plût,None,None,plussent,None,None,None,None
4062,repleuvoir,repleuvant,replu,None,None,repleut,None,None,None,None,None,repleuvait,None,None,None,None,None,replut,None,None,None,None,None,repleuvra,None,None,None,None,None,repleuvrait,None,None,None,None,None,repleuve,None,None,None,None,None,replût,None,None,None,None,None,None,None


#### vouloir

In [52]:
like_vouloir, remaining_data = is_like("vouloir", "vouloir", "vouloir")
display(like_vouloir)

,infinitive,present participle,past participle,present 1sg,present 2sg,present 3sg,present 1pl,present 2pl,present 3pl,imperfect 1sg,imperfect 2sg,imperfect 3sg,imperfect 1pl,imperfect 2pl,imperfect 3pl,past historic 1sg,past historic 2sg,past historic 3sg,past historic 1pl,past historic 2pl,past historic 3pl,future 1sg,future 2sg,future 3sg,future 1pl,future 2pl,future 3pl,conditional 1sg,conditional 2sg,conditional 3sg,conditional 1pl,conditional 2pl,conditional 3pl,subjunctive present 1sg,subjunctive present 2sg,subjunctive present 3sg,subjunctive present 1pl,subjunctive present 2pl,subjunctive present 3pl,subjunctive imperfect 1sg,subjunctive imperfect 2sg,subjunctive imperfect 3sg,subjunctive imperfect 1pl,subjunctive imperfect 2pl,subjunctive imperfect 3pl,imperative 2sg,imperative 1pl,imperative 2pl,group
0,vouloir,voulant,voulu,veux,veux,veut,voulons,voulez,veulent,voulais,voulais,voulait,voulions,vouliez,voulaient,voulus,voulus,voulut,voulûmes,voulûtes,voulurent,voudrai,voudras,voudra,voudrons,voudrez,voudront,voudrais,voudrais,voudrait,voudrions,voudriez,voudraient,veuille,veuilles,veuille,voulions,vouliez,veuillent,voulusse,voulusses,voulût,voulussions,voulussiez,voulussent,veux,voulons,voulez,None


,infinitive,present participle,past participle,present 1sg,present 2sg,present 3sg,present 1pl,present 2pl,present 3pl,imperfect 1sg,imperfect 2sg,imperfect 3sg,imperfect 1pl,imperfect 2pl,imperfect 3pl,past historic 1sg,past historic 2sg,past historic 3sg,past historic 1pl,past historic 2pl,past historic 3pl,future 1sg,future 2sg,future 3sg,future 1pl,future 2pl,future 3pl,conditional 1sg,conditional 2sg,conditional 3sg,conditional 1pl,conditional 2pl,conditional 3pl,subjunctive present 1sg,subjunctive present 2sg,subjunctive present 3sg,subjunctive present 1pl,subjunctive present 2pl,subjunctive present 3pl,subjunctive imperfect 1sg,subjunctive imperfect 2sg,subjunctive imperfect 3sg,subjunctive imperfect 1pl,subjunctive imperfect 2pl,subjunctive imperfect 3pl,imperative 2sg,imperative 1pl,imperative 2pl,group
4158,revouloir,revoulant,revoulu,reveux,reveux,reveut,revoulons,revoulez,reveulent,revoulais,revoulais,revoulait,revoulions,revouliez,revoulaient,revoulus,revoulus,revoulut,revoulûmes,revoulûtes,revoulurent,revoudrai,revoudras,revoudra,revoudrons,revoudrez,revoudront,revoudrais,revoudrais,revoudrait,revoudrions,revoudriez,revoudraient,reveuille,reveuilles,reveuille,revoulions,revouliez,reveuillent,revoulusse,revoulusses,revoulût,revoulussions,revoulussiez,revoulussent,reveux,revoulons,revoulez,None
5068,vouloir,voulant,voulu,veux,veux,veut,voulons,voulez,veulent,voulais,voulais,voulait,voulions,vouliez,voulaient,voulus,voulus,voulut,voulûmes,voulûtes,voulurent,voudrai,voudras,voudra,voudrons,voudrez,voudront,voudrais,voudrais,voudrait,voudrions,voudriez,voudraient,veuille,veuilles,veuille,voulions,vouliez,veuillent,voulusse,voulusses,voulût,voulussions,voulussiez,voulussent,veux,voulons,voulez,None


#### valoir

In [53]:
like_valoir, remaining_data = is_like("valoir", "valoir", "valoir")
display(like_valoir)

,infinitive,present participle,past participle,present 1sg,present 2sg,present 3sg,present 1pl,present 2pl,present 3pl,imperfect 1sg,imperfect 2sg,imperfect 3sg,imperfect 1pl,imperfect 2pl,imperfect 3pl,past historic 1sg,past historic 2sg,past historic 3sg,past historic 1pl,past historic 2pl,past historic 3pl,future 1sg,future 2sg,future 3sg,future 1pl,future 2pl,future 3pl,conditional 1sg,conditional 2sg,conditional 3sg,conditional 1pl,conditional 2pl,conditional 3pl,subjunctive present 1sg,subjunctive present 2sg,subjunctive present 3sg,subjunctive present 1pl,subjunctive present 2pl,subjunctive present 3pl,subjunctive imperfect 1sg,subjunctive imperfect 2sg,subjunctive imperfect 3sg,subjunctive imperfect 1pl,subjunctive imperfect 2pl,subjunctive imperfect 3pl,imperative 2sg,imperative 1pl,imperative 2pl,group
0,valoir,valant,valu,vaux,vaux,vaut,valons,valez,valent,valais,valais,valait,valions,valiez,valaient,valus,valus,valut,valûmes,valûtes,valurent,vaudrai,vaudras,vaudra,vaudrons,vaudrez,vaudront,vaudrais,vaudrais,vaudrait,vaudrions,vaudriez,vaudraient,vaille,vailles,vaille,valions,valiez,vaillent,valusse,valusses,valût,valussions,valussiez,valussent,None,None,None,None


,infinitive,present participle,past participle,present 1sg,present 2sg,present 3sg,present 1pl,present 2pl,present 3pl,imperfect 1sg,imperfect 2sg,imperfect 3sg,imperfect 1pl,imperfect 2pl,imperfect 3pl,past historic 1sg,past historic 2sg,past historic 3sg,past historic 1pl,past historic 2pl,past historic 3pl,future 1sg,future 2sg,future 3sg,future 1pl,future 2pl,future 3pl,conditional 1sg,conditional 2sg,conditional 3sg,conditional 1pl,conditional 2pl,conditional 3pl,subjunctive present 1sg,subjunctive present 2sg,subjunctive present 3sg,subjunctive present 1pl,subjunctive present 2pl,subjunctive present 3pl,subjunctive imperfect 1sg,subjunctive imperfect 2sg,subjunctive imperfect 3sg,subjunctive imperfect 1pl,subjunctive imperfect 2pl,subjunctive imperfect 3pl,imperative 2sg,imperative 1pl,imperative 2pl,group
3681,prévaloir,prévalant,prévalu,prévaux,prévaux,prévaut,prévalons,prévalez,prévalent,prévalais,prévalais,prévalait,prévalions,prévaliez,prévalaient,prévalus,prévalus,prévalut,prévalûmes,prévalûtes,prévalurent,prévaudrai,prévaudras,prévaudra,prévaudrons,prévaudrez,prévaudront,prévaudrais,prévaudrais,prévaudrait,prévaudrions,prévaudriez,prévaudraient,prévaille,prévailles,prévaille,prévalions,prévaliez,prévaillent,prévalusse,prévalusses,prévalût,prévalussions,prévalussiez,prévalussent,None,None,None,None
4991,valoir,valant,valu,vaux,vaux,vaut,valons,valez,valent,valais,valais,valait,valions,valiez,valaient,valus,valus,valut,valûmes,valûtes,valurent,vaudrai,vaudras,vaudra,vaudrons,vaudrez,vaudront,vaudrais,vaudrais,vaudrait,vaudrions,vaudriez,vaudraient,vaille,vailles,vaille,valions,valiez,vaillent,valusse,valusses,valût,valussions,valussiez,valussent,None,None,None,None
5271,équivaloir,équivalant,équivalu,équivaux,équivaux,équivaut,équivalons,équivalez,équivalent,équivalais,équivalais,équivalait,équivalions,équivaliez,équivalaient,équivalus,équivalus,équivalut,équivalûmes,équivalûtes,équivalurent,équivaudrai,équivaudras,équivaudra,équivaudrons,équivaudrez,équivaudront,équivaudrais,équivaudrais,équivaudrait,équivaudrions,équivaudriez,équivaudraient,équivaille,équivailles,équivaille,équivalions,équivaliez,équivaillent,équivalusse,équivalusses,équivalût,équivalussions,équivalussiez,équivalussent,None,None,None,None


#### voir

In [54]:
like_voir, remaining_data = is_like("voir", "oir", "voir")
display(like_voir)

,infinitive,present participle,past participle,present 1sg,present 2sg,present 3sg,present 1pl,present 2pl,present 3pl,imperfect 1sg,imperfect 2sg,imperfect 3sg,imperfect 1pl,imperfect 2pl,imperfect 3pl,past historic 1sg,past historic 2sg,past historic 3sg,past historic 1pl,past historic 2pl,past historic 3pl,future 1sg,future 2sg,future 3sg,future 1pl,future 2pl,future 3pl,conditional 1sg,conditional 2sg,conditional 3sg,conditional 1pl,conditional 2pl,conditional 3pl,subjunctive present 1sg,subjunctive present 2sg,subjunctive present 3sg,subjunctive present 1pl,subjunctive present 2pl,subjunctive present 3pl,subjunctive imperfect 1sg,subjunctive imperfect 2sg,subjunctive imperfect 3sg,subjunctive imperfect 1pl,subjunctive imperfect 2pl,subjunctive imperfect 3pl,imperative 2sg,imperative 1pl,imperative 2pl,group
0,oir,oyant,u,ois,ois,oit,oyons,oyez,oient,oyais,oyais,oyait,oyions,oyiez,oyaient,is,is,it,îmes,îtes,irent,errai,erras,erra,errons,errez,erront,errais,errais,errait,errions,erriez,erraient,oie,oies,oie,oyions,oyiez,oient,isse,isses,ît,issions,issiez,issent,ois,oyons,oyez,None


,infinitive,present participle,past participle,present 1sg,present 2sg,present 3sg,present 1pl,present 2pl,present 3pl,imperfect 1sg,imperfect 2sg,imperfect 3sg,imperfect 1pl,imperfect 2pl,imperfect 3pl,past historic 1sg,past historic 2sg,past historic 3sg,past historic 1pl,past historic 2pl,past historic 3pl,future 1sg,future 2sg,future 3sg,future 1pl,future 2pl,future 3pl,conditional 1sg,conditional 2sg,conditional 3sg,conditional 1pl,conditional 2pl,conditional 3pl,subjunctive present 1sg,subjunctive present 2sg,subjunctive present 3sg,subjunctive present 1pl,subjunctive present 2pl,subjunctive present 3pl,subjunctive imperfect 1sg,subjunctive imperfect 2sg,subjunctive imperfect 3sg,subjunctive imperfect 1pl,subjunctive imperfect 2pl,subjunctive imperfect 3pl,imperative 2sg,imperative 1pl,imperative 2pl,group
2107,entrevoir,entrevoyant,entrevu,entrevois,entrevois,entrevoit,entrevoyons,entrevoyez,entrevoient,entrevoyais,entrevoyais,entrevoyait,entrevoyions,entrevoyiez,entrevoyaient,entrevis,entrevis,entrevit,entrevîmes,entrevîtes,entrevirent,entreverrai,entreverras,entreverra,entreverrons,entreverrez,entreverront,entreverrais,entreverrais,entreverrait,entreverrions,entreverriez,entreverraient,entrevoie,entrevoies,entrevoie,entrevoyions,entrevoyiez,entrevoient,entrevisse,entrevisses,entrevît,entrevissions,entrevissiez,entrevissent,entrevois,entrevoyons,entrevoyez,None
4157,revoir,revoyant,revu,revois,revois,revoit,revoyons,revoyez,revoient,revoyais,revoyais,revoyait,revoyions,revoyiez,revoyaient,revis,revis,revit,revîmes,revîtes,revirent,reverrai,reverras,reverra,reverrons,reverrez,reverront,reverrais,reverrais,reverrait,reverrions,reverriez,reverraient,revoie,revoies,revoie,revoyions,revoyiez,revoient,revisse,revisses,revît,revissions,revissiez,revissent,revois,revoyons,revoyez,None
5057,voir,voyant,vu,vois,vois,voit,voyons,voyez,voient,voyais,voyais,voyait,voyions,voyiez,voyaient,vis,vis,vit,vîmes,vîtes,virent,verrai,verras,verra,verrons,verrez,verront,verrais,verrais,verrait,verrions,verriez,verraient,voie,voies,voie,voyions,voyiez,voient,visse,visses,vît,vissions,vissiez,vissent,vois,voyons,voyez,None


#### pourvoir

In [55]:
like_pourvoir, remaining_data = is_like("pourvoir", "oir", "pourvoir")
display(like_pourvoir)

,infinitive,present participle,past participle,present 1sg,present 2sg,present 3sg,present 1pl,present 2pl,present 3pl,imperfect 1sg,imperfect 2sg,imperfect 3sg,imperfect 1pl,imperfect 2pl,imperfect 3pl,past historic 1sg,past historic 2sg,past historic 3sg,past historic 1pl,past historic 2pl,past historic 3pl,future 1sg,future 2sg,future 3sg,future 1pl,future 2pl,future 3pl,conditional 1sg,conditional 2sg,conditional 3sg,conditional 1pl,conditional 2pl,conditional 3pl,subjunctive present 1sg,subjunctive present 2sg,subjunctive present 3sg,subjunctive present 1pl,subjunctive present 2pl,subjunctive present 3pl,subjunctive imperfect 1sg,subjunctive imperfect 2sg,subjunctive imperfect 3sg,subjunctive imperfect 1pl,subjunctive imperfect 2pl,subjunctive imperfect 3pl,imperative 2sg,imperative 1pl,imperative 2pl,group
0,oir,oyant,u,ois,ois,oit,oyons,oyez,oient,oyais,oyais,oyait,oyions,oyiez,oyaient,us,us,ut,ûmes,ûtes,urent,oirai,oiras,oira,oirons,oirez,oiront,oirais,oirais,oirait,oirions,oiriez,oiraient,oie,oies,oie,oyions,oyiez,oient,usse,usses,ût,ussions,ussiez,ussent,ois,oyons,oyez,None


,infinitive,present participle,past participle,present 1sg,present 2sg,present 3sg,present 1pl,present 2pl,present 3pl,imperfect 1sg,imperfect 2sg,imperfect 3sg,imperfect 1pl,imperfect 2pl,imperfect 3pl,past historic 1sg,past historic 2sg,past historic 3sg,past historic 1pl,past historic 2pl,past historic 3pl,future 1sg,future 2sg,future 3sg,future 1pl,future 2pl,future 3pl,conditional 1sg,conditional 2sg,conditional 3sg,conditional 1pl,conditional 2pl,conditional 3pl,subjunctive present 1sg,subjunctive present 2sg,subjunctive present 3sg,subjunctive present 1pl,subjunctive present 2pl,subjunctive present 3pl,subjunctive imperfect 1sg,subjunctive imperfect 2sg,subjunctive imperfect 3sg,subjunctive imperfect 1pl,subjunctive imperfect 2pl,subjunctive imperfect 3pl,imperative 2sg,imperative 1pl,imperative 2pl,group
1680,dépourvoir,dépourvoyant,dépourvu,dépourvois,dépourvois,dépourvoit,dépourvoyons,dépourvoyez,dépourvoient,dépourvoyais,dépourvoyais,dépourvoyait,dépourvoyions,dépourvoyiez,dépourvoyaient,dépourvus,dépourvus,dépourvut,dépourvûmes,dépourvûtes,dépourvurent,dépourvoirai,dépourvoiras,dépourvoira,dépourvoirons,dépourvoirez,dépourvoiront,dépourvoirais,dépourvoirais,dépourvoirait,dépourvoirions,dépourvoiriez,dépourvoiraient,dépourvoie,dépourvoies,dépourvoie,dépourvoyions,dépourvoyiez,dépourvoient,dépourvusse,dépourvusses,dépourvût,dépourvussions,dépourvussiez,dépourvussent,dépourvois,dépourvoyons,dépourvoyez,None
3574,pourvoir,pourvoyant,pourvu,pourvois,pourvois,pourvoit,pourvoyons,pourvoyez,pourvoient,pourvoyais,pourvoyais,pourvoyait,pourvoyions,pourvoyiez,pourvoyaient,pourvus,pourvus,pourvut,pourvûmes,pourvûtes,pourvurent,pourvoirai,pourvoiras,pourvoira,pourvoirons,pourvoirez,pourvoiront,pourvoirais,pourvoirais,pourvoirait,pourvoirions,pourvoiriez,pourvoiraient,pourvoie,pourvoies,pourvoie,pourvoyions,pourvoyiez,pourvoient,pourvusse,pourvusses,pourvût,pourvussions,pourvussiez,pourvussent,pourvois,pourvoyons,pourvoyez,None


#### prévoir

In [56]:
like_prevoir, remaining_data = is_like("prévoir", "oir", "prévoir")
display(like_prevoir)

,infinitive,present participle,past participle,present 1sg,present 2sg,present 3sg,present 1pl,present 2pl,present 3pl,imperfect 1sg,imperfect 2sg,imperfect 3sg,imperfect 1pl,imperfect 2pl,imperfect 3pl,past historic 1sg,past historic 2sg,past historic 3sg,past historic 1pl,past historic 2pl,past historic 3pl,future 1sg,future 2sg,future 3sg,future 1pl,future 2pl,future 3pl,conditional 1sg,conditional 2sg,conditional 3sg,conditional 1pl,conditional 2pl,conditional 3pl,subjunctive present 1sg,subjunctive present 2sg,subjunctive present 3sg,subjunctive present 1pl,subjunctive present 2pl,subjunctive present 3pl,subjunctive imperfect 1sg,subjunctive imperfect 2sg,subjunctive imperfect 3sg,subjunctive imperfect 1pl,subjunctive imperfect 2pl,subjunctive imperfect 3pl,imperative 2sg,imperative 1pl,imperative 2pl,group
0,oir,oyant,u,ois,ois,oit,oyons,oyez,oient,oyais,oyais,oyait,oyions,oyiez,oyaient,is,is,it,îmes,îtes,irent,oirai,oiras,oira,oirons,oirez,oiront,oirais,oirais,oirait,oirions,oiriez,oiraient,oie,oies,oie,oyions,oyiez,oient,isse,isses,ît,issions,issiez,issent,ois,oyons,oyez,None


,infinitive,present participle,past participle,present 1sg,present 2sg,present 3sg,present 1pl,present 2pl,present 3pl,imperfect 1sg,imperfect 2sg,imperfect 3sg,imperfect 1pl,imperfect 2pl,imperfect 3pl,past historic 1sg,past historic 2sg,past historic 3sg,past historic 1pl,past historic 2pl,past historic 3pl,future 1sg,future 2sg,future 3sg,future 1pl,future 2pl,future 3pl,conditional 1sg,conditional 2sg,conditional 3sg,conditional 1pl,conditional 2pl,conditional 3pl,subjunctive present 1sg,subjunctive present 2sg,subjunctive present 3sg,subjunctive present 1pl,subjunctive present 2pl,subjunctive present 3pl,subjunctive imperfect 1sg,subjunctive imperfect 2sg,subjunctive imperfect 3sg,subjunctive imperfect 1pl,subjunctive imperfect 2pl,subjunctive imperfect 3pl,imperative 2sg,imperative 1pl,imperative 2pl,group
3684,prévoir,prévoyant,prévu,prévois,prévois,prévoit,prévoyons,prévoyez,prévoient,prévoyais,prévoyais,prévoyait,prévoyions,prévoyiez,prévoyaient,prévis,prévis,prévit,prévîmes,prévîtes,prévirent,prévoirai,prévoiras,prévoira,prévoirons,prévoirez,prévoiront,prévoirais,prévoirais,prévoirait,prévoirions,prévoiriez,prévoiraient,prévoie,prévoies,prévoie,prévoyions,prévoyiez,prévoient,prévisse,prévisses,prévît,prévissions,prévissiez,prévissent,prévois,prévoyons,prévoyez,None


#### mouvoir

In [57]:
like_mouvoir, remaining_data = is_like("mouvoir", "ouvoir", "mouvoir")
display(like_mouvoir)

,infinitive,present participle,past participle,present 1sg,present 2sg,present 3sg,present 1pl,present 2pl,present 3pl,imperfect 1sg,imperfect 2sg,imperfect 3sg,imperfect 1pl,imperfect 2pl,imperfect 3pl,past historic 1sg,past historic 2sg,past historic 3sg,past historic 1pl,past historic 2pl,past historic 3pl,future 1sg,future 2sg,future 3sg,future 1pl,future 2pl,future 3pl,conditional 1sg,conditional 2sg,conditional 3sg,conditional 1pl,conditional 2pl,conditional 3pl,subjunctive present 1sg,subjunctive present 2sg,subjunctive present 3sg,subjunctive present 1pl,subjunctive present 2pl,subjunctive present 3pl,subjunctive imperfect 1sg,subjunctive imperfect 2sg,subjunctive imperfect 3sg,subjunctive imperfect 1pl,subjunctive imperfect 2pl,subjunctive imperfect 3pl,imperative 2sg,imperative 1pl,imperative 2pl,group
0,ouvoir,ouvant,û,eus,eus,eut,ouvons,ouvez,euvent,ouvais,ouvais,ouvait,ouvions,ouviez,ouvaient,us,us,ut,ûmes,ûtes,urent,ouvrai,ouvras,ouvra,ouvrons,ouvrez,ouvront,ouvrais,ouvrais,ouvrait,ouvrions,ouvriez,ouvraient,euve,euves,euve,ouvions,ouviez,euvent,usse,usses,ût,ussions,ussiez,ussent,eus,ouvons,ouvez,None


,infinitive,present participle,past participle,present 1sg,present 2sg,present 3sg,present 1pl,present 2pl,present 3pl,imperfect 1sg,imperfect 2sg,imperfect 3sg,imperfect 1pl,imperfect 2pl,imperfect 3pl,past historic 1sg,past historic 2sg,past historic 3sg,past historic 1pl,past historic 2pl,past historic 3pl,future 1sg,future 2sg,future 3sg,future 1pl,future 2pl,future 3pl,conditional 1sg,conditional 2sg,conditional 3sg,conditional 1pl,conditional 2pl,conditional 3pl,subjunctive present 1sg,subjunctive present 2sg,subjunctive present 3sg,subjunctive present 1pl,subjunctive present 2pl,subjunctive present 3pl,subjunctive imperfect 1sg,subjunctive imperfect 2sg,subjunctive imperfect 3sg,subjunctive imperfect 1pl,subjunctive imperfect 2pl,subjunctive imperfect 3pl,imperative 2sg,imperative 1pl,imperative 2pl,group
3133,mouvoir,mouvant,mû,meus,meus,meut,mouvons,mouvez,meuvent,mouvais,mouvais,mouvait,mouvions,mouviez,mouvaient,mus,mus,mut,mûmes,mûtes,murent,mouvrai,mouvras,mouvra,mouvrons,mouvrez,mouvront,mouvrais,mouvrais,mouvrait,mouvrions,mouvriez,mouvraient,meuve,meuves,meuve,mouvions,mouviez,meuvent,musse,musses,mût,mussions,mussiez,mussent,meus,mouvons,mouvez,None


#### promouvoir

In [58]:
like_promouvoir, remaining_data = is_like("promouvoir", "ouvoir", "-mouvoir (except mouvoir)")
display(like_promouvoir)

,infinitive,present participle,past participle,present 1sg,present 2sg,present 3sg,present 1pl,present 2pl,present 3pl,imperfect 1sg,imperfect 2sg,imperfect 3sg,imperfect 1pl,imperfect 2pl,imperfect 3pl,past historic 1sg,past historic 2sg,past historic 3sg,past historic 1pl,past historic 2pl,past historic 3pl,future 1sg,future 2sg,future 3sg,future 1pl,future 2pl,future 3pl,conditional 1sg,conditional 2sg,conditional 3sg,conditional 1pl,conditional 2pl,conditional 3pl,subjunctive present 1sg,subjunctive present 2sg,subjunctive present 3sg,subjunctive present 1pl,subjunctive present 2pl,subjunctive present 3pl,subjunctive imperfect 1sg,subjunctive imperfect 2sg,subjunctive imperfect 3sg,subjunctive imperfect 1pl,subjunctive imperfect 2pl,subjunctive imperfect 3pl,imperative 2sg,imperative 1pl,imperative 2pl,group
0,ouvoir,ouvant,u,eus,eus,eut,ouvons,ouvez,euvent,ouvais,ouvais,ouvait,ouvions,ouviez,ouvaient,us,us,ut,ûmes,ûtes,urent,ouvrai,ouvras,ouvra,ouvrons,ouvrez,ouvront,ouvrais,ouvrais,ouvrait,ouvrions,ouvriez,ouvraient,euve,euves,euve,ouvions,ouviez,euvent,usse,usses,ût,ussions,ussiez,ussent,eus,ouvons,ouvez,None


,infinitive,present participle,past participle,present 1sg,present 2sg,present 3sg,present 1pl,present 2pl,present 3pl,imperfect 1sg,imperfect 2sg,imperfect 3sg,imperfect 1pl,imperfect 2pl,imperfect 3pl,past historic 1sg,past historic 2sg,past historic 3sg,past historic 1pl,past historic 2pl,past historic 3pl,future 1sg,future 2sg,future 3sg,future 1pl,future 2pl,future 3pl,conditional 1sg,conditional 2sg,conditional 3sg,conditional 1pl,conditional 2pl,conditional 3pl,subjunctive present 1sg,subjunctive present 2sg,subjunctive present 3sg,subjunctive present 1pl,subjunctive present 2pl,subjunctive present 3pl,subjunctive imperfect 1sg,subjunctive imperfect 2sg,subjunctive imperfect 3sg,subjunctive imperfect 1pl,subjunctive imperfect 2pl,subjunctive imperfect 3pl,imperative 2sg,imperative 1pl,imperative 2pl,group
3614,promouvoir,promouvant,promu,promeus,promeus,promeut,promouvons,promouvez,promeuvent,promouvais,promouvais,promouvait,promouvions,promouviez,promouvaient,promus,promus,promut,promûmes,promûtes,promurent,promouvrai,promouvras,promouvra,promouvrons,promouvrez,promouvront,promouvrais,promouvrais,promouvrait,promouvrions,promouvriez,promouvraient,promeuve,promeuves,promeuve,promouvions,promouviez,promeuvent,promusse,promusses,promût,promussions,promussiez,promussent,promeus,promouvons,promouvez,None
5223,émouvoir,émouvant,ému,émeus,émeus,émeut,émouvons,émouvez,émeuvent,émouvais,émouvais,émouvait,émouvions,émouviez,émouvaient,émus,émus,émut,émûmes,émûtes,émurent,émouvrai,émouvras,émouvra,émouvrons,émouvrez,émouvront,émouvrais,émouvrais,émouvrait,émouvrions,émouvriez,émouvraient,émeuve,émeuves,émeuve,émouvions,émouviez,émeuvent,émusse,émusses,émût,émussions,émussiez,émussent,émeus,émouvons,émouvez,None


#### asseoir

In [59]:
like_asseoir, remaining_data = is_like("asseoir", "eoir", "asseoir")
display(like_asseoir)

,infinitive,present participle,past participle,present 1sg,present 2sg,present 3sg,present 1pl,present 2pl,present 3pl,imperfect 1sg,imperfect 2sg,imperfect 3sg,imperfect 1pl,imperfect 2pl,imperfect 3pl,past historic 1sg,past historic 2sg,past historic 3sg,past historic 1pl,past historic 2pl,past historic 3pl,future 1sg,future 2sg,future 3sg,future 1pl,future 2pl,future 3pl,conditional 1sg,conditional 2sg,conditional 3sg,conditional 1pl,conditional 2pl,conditional 3pl,subjunctive present 1sg,subjunctive present 2sg,subjunctive present 3sg,subjunctive present 1pl,subjunctive present 2pl,subjunctive present 3pl,subjunctive imperfect 1sg,subjunctive imperfect 2sg,subjunctive imperfect 3sg,subjunctive imperfect 1pl,subjunctive imperfect 2pl,subjunctive imperfect 3pl,imperative 2sg,imperative 1pl,imperative 2pl,group
0,eoir,"(oyant, eyant)",is,"(ois, ieds)","(ois, ieds)","(oit, ied)","(oyons, eyons)","(oyez, eyez)","(oient, eyent)","(oyais, eyais)","(oyais, eyais)","(oyait, eyait)","(oyions, eyions)","(oyiez, eyiez)","(oyaient, eyaient)",is,is,it,îmes,îtes,irent,"(oirai, iérai)","(oiras, iéras)","(oira, iéra)","(oirons, iérons)","(oirez, iérez)","(oiront, iéront)","(oirais, iérais)","(oirais, iérais)","(oirait, iérait)","(oirions, iérions)","(oiriez, iériez)","(oiraient, iéraient)","(oie, eye)","(oies, eyes)","(oie, eye)","(oyions, eyions)","(oyiez, eyiez)","(oyient, eyent)",isse,isses,ît,issions,issiez,issent,"(ois, ieds)","(oyons, eyons)","(oyez, eyez)",None


,infinitive,present participle,past participle,present 1sg,present 2sg,present 3sg,present 1pl,present 2pl,present 3pl,imperfect 1sg,imperfect 2sg,imperfect 3sg,imperfect 1pl,imperfect 2pl,imperfect 3pl,past historic 1sg,past historic 2sg,past historic 3sg,past historic 1pl,past historic 2pl,past historic 3pl,future 1sg,future 2sg,future 3sg,future 1pl,future 2pl,future 3pl,conditional 1sg,conditional 2sg,conditional 3sg,conditional 1pl,conditional 2pl,conditional 3pl,subjunctive present 1sg,subjunctive present 2sg,subjunctive present 3sg,subjunctive present 1pl,subjunctive present 2pl,subjunctive present 3pl,subjunctive imperfect 1sg,subjunctive imperfect 2sg,subjunctive imperfect 3sg,subjunctive imperfect 1pl,subjunctive imperfect 2pl,subjunctive imperfect 3pl,imperative 2sg,imperative 1pl,imperative 2pl,group
340,asseoir,"(assoyant, asseyant)",assis,"(assois, assieds)","(assois, assieds)","(assoit, assied)","(assoyons, asseyons)","(assoyez, asseyez)","(assoient, asseyent)","(assoyais, asseyais)","(assoyais, asseyais)","(assoyait, asseyait)","(assoyions, asseyions)","(assoyiez, asseyiez)","(assoyaient, asseyaient)",assis,assis,assit,assîmes,assîtes,assirent,"(assoirai, assiérai)","(assoiras, assiéras)","(assoira, assiéra)","(assoirons, assiérons)","(assoirez, assiérez)","(assoiront, assiéront)","(assoirais, assiérais)","(assoirais, assiérais)","(assoirait, assiérait)","(assoirions, assiérions)","(assoiriez, assiériez)","(assoiraient, assiéraient)","(assoie, asseye)","(assoies, asseyes)","(assoie, asseye)","(assoyions, asseyions)","(assoyiez, asseyiez)","(assoyient, asseyent)",assisse,assisses,assît,assissions,assissiez,assissent,"(assois, assieds)","(assoyons, asseyons)","(assoyez, asseyez)",None
3809,rasseoir,"(rassoyant, rasseyant)",rassis,"(rassois, rassieds)","(rassois, rassieds)","(rassoit, rassied)","(rassoyons, rasseyons)","(rassoyez, rasseyez)","(rassoient, rasseyent)","(rassoyais, rasseyais)","(rassoyais, rasseyais)","(rassoyait, rasseyait)","(rassoyions, rasseyions)","(rassoyiez, rasseyiez)","(rassoyaient, rasseyaient)",rassis,rassis,rassit,rassîmes,rassîtes,rassirent,"(rassoirai, rassiérai)","(rassoiras, rassiéras)","(rassoira, rassiéra)","(rassoirons, rassiérons)","(rassoirez, rassiérez)","(rassoiront, rassiéront)","(rassoirais, rassiérais)","(rassoirais, rassiérais)","(rassoirait, rassiérait)","(rassoirions, rassiérions)","(rassoiriez, rassiériez)","(rassoiraient, rassiéraient)","(rassoie, rasseye)","(rassoies, rasseyes)","(rassoie, rasseye)","(rassoyions, rasseyions)","(rassoyiez, rasseyiez)","(rassoyient, rasseyent)",rassisse,rassisses,rassît,rassissions,rassissiez,rassissent,"(rassois, rassieds)","(rassoyons, rasseyons)","(rassoyez, rasseyez)",None


#### seoir

In [60]:
like_seoir, remaining_data = is_like("seoir", "eoir", "seoir")
display(like_seoir)

,infinitive,present participle,past participle,present 1sg,present 2sg,present 3sg,present 1pl,present 2pl,present 3pl,imperfect 1sg,imperfect 2sg,imperfect 3sg,imperfect 1pl,imperfect 2pl,imperfect 3pl,past historic 1sg,past historic 2sg,past historic 3sg,past historic 1pl,past historic 2pl,past historic 3pl,future 1sg,future 2sg,future 3sg,future 1pl,future 2pl,future 3pl,conditional 1sg,conditional 2sg,conditional 3sg,conditional 1pl,conditional 2pl,conditional 3pl,subjunctive present 1sg,subjunctive present 2sg,subjunctive present 3sg,subjunctive present 1pl,subjunctive present 2pl,subjunctive present 3pl,subjunctive imperfect 1sg,subjunctive imperfect 2sg,subjunctive imperfect 3sg,subjunctive imperfect 1pl,subjunctive imperfect 2pl,subjunctive imperfect 3pl,imperative 2sg,imperative 1pl,imperative 2pl,group
0,eoir,éant,is,ieds,ieds,ied,eyons,eyez,iéent,eyais,eyais,eyait,eyions,eyiez,eyaient,is,is,it,îmes,îtes,irent,iérai,iéras,iéra,iérons,iérez,iéront,iérais,iérais,iérait,iérions,iériez,iéraient,iée,iées,iée,None,None,iéent,None,None,None,None,None,None,ieds,eyons,eyez,None


,infinitive,present participle,past participle,present 1sg,present 2sg,present 3sg,present 1pl,present 2pl,present 3pl,imperfect 1sg,imperfect 2sg,imperfect 3sg,imperfect 1pl,imperfect 2pl,imperfect 3pl,past historic 1sg,past historic 2sg,past historic 3sg,past historic 1pl,past historic 2pl,past historic 3pl,future 1sg,future 2sg,future 3sg,future 1pl,future 2pl,future 3pl,conditional 1sg,conditional 2sg,conditional 3sg,conditional 1pl,conditional 2pl,conditional 3pl,subjunctive present 1sg,subjunctive present 2sg,subjunctive present 3sg,subjunctive present 1pl,subjunctive present 2pl,subjunctive present 3pl,subjunctive imperfect 1sg,subjunctive imperfect 2sg,subjunctive imperfect 3sg,subjunctive imperfect 1pl,subjunctive imperfect 2pl,subjunctive imperfect 3pl,imperative 2sg,imperative 1pl,imperative 2pl,group
4475,seoir,séant,sis,sieds,sieds,sied,seyons,seyez,siéent,seyais,seyais,seyait,seyions,seyiez,seyaient,sis,sis,sit,sîmes,sîtes,sirent,siérai,siéras,siéra,siérons,siérez,siéront,siérais,siérais,siérait,siérions,siériez,siéraient,siée,siées,siée,None,None,siéent,None,None,None,None,None,None,sieds,seyons,seyez,None


#### surseoir

In [61]:
like_surseoir, remaining_data = is_like("surseoir", "eoir", "surseoir")
display(like_surseoir)

,infinitive,present participle,past participle,present 1sg,present 2sg,present 3sg,present 1pl,present 2pl,present 3pl,imperfect 1sg,imperfect 2sg,imperfect 3sg,imperfect 1pl,imperfect 2pl,imperfect 3pl,past historic 1sg,past historic 2sg,past historic 3sg,past historic 1pl,past historic 2pl,past historic 3pl,future 1sg,future 2sg,future 3sg,future 1pl,future 2pl,future 3pl,conditional 1sg,conditional 2sg,conditional 3sg,conditional 1pl,conditional 2pl,conditional 3pl,subjunctive present 1sg,subjunctive present 2sg,subjunctive present 3sg,subjunctive present 1pl,subjunctive present 2pl,subjunctive present 3pl,subjunctive imperfect 1sg,subjunctive imperfect 2sg,subjunctive imperfect 3sg,subjunctive imperfect 1pl,subjunctive imperfect 2pl,subjunctive imperfect 3pl,imperative 2sg,imperative 1pl,imperative 2pl,group
0,eoir,oyant,is,ois,ois,oit,oyons,oyez,oient,oyais,oyais,oyait,oyions,oyiez,oyaient,is,is,it,îmes,îtes,irent,eoirai,eoiras,eoira,eoirons,eoirez,eoiront,eoirais,eoirais,eoirait,eoirions,eoiriez,eoiraient,oie,oies,oie,oyions,oyiez,oient,isse,isses,ît,issions,issiez,issent,ois,oyons,oyez,None


,infinitive,present participle,past participle,present 1sg,present 2sg,present 3sg,present 1pl,present 2pl,present 3pl,imperfect 1sg,imperfect 2sg,imperfect 3sg,imperfect 1pl,imperfect 2pl,imperfect 3pl,past historic 1sg,past historic 2sg,past historic 3sg,past historic 1pl,past historic 2pl,past historic 3pl,future 1sg,future 2sg,future 3sg,future 1pl,future 2pl,future 3pl,conditional 1sg,conditional 2sg,conditional 3sg,conditional 1pl,conditional 2pl,conditional 3pl,subjunctive present 1sg,subjunctive present 2sg,subjunctive present 3sg,subjunctive present 1pl,subjunctive present 2pl,subjunctive present 3pl,subjunctive imperfect 1sg,subjunctive imperfect 2sg,subjunctive imperfect 3sg,subjunctive imperfect 1pl,subjunctive imperfect 2pl,subjunctive imperfect 3pl,imperative 2sg,imperative 1pl,imperative 2pl,group
4691,surseoir,sursoyant,sursis,sursois,sursois,sursoit,sursoyons,sursoyez,sursoient,sursoyais,sursoyais,sursoyait,sursoyions,sursoyiez,sursoyaient,sursis,sursis,sursit,sursîmes,sursîtes,sursirent,surseoirai,surseoiras,surseoira,surseoirons,surseoirez,surseoiront,surseoirais,surseoirais,surseoirait,surseoirions,surseoiriez,surseoiraient,sursoie,sursoies,sursoie,sursoyions,sursoyiez,sursoient,sursisse,sursisses,sursît,sursissions,sursissiez,sursissent,sursois,sursoyons,sursoyez,None


#### boire

In [62]:
like_boire, remaining_data = is_like("boire", "boire", "boire")
display(like_boire)

,infinitive,present participle,past participle,present 1sg,present 2sg,present 3sg,present 1pl,present 2pl,present 3pl,imperfect 1sg,imperfect 2sg,imperfect 3sg,imperfect 1pl,imperfect 2pl,imperfect 3pl,past historic 1sg,past historic 2sg,past historic 3sg,past historic 1pl,past historic 2pl,past historic 3pl,future 1sg,future 2sg,future 3sg,future 1pl,future 2pl,future 3pl,conditional 1sg,conditional 2sg,conditional 3sg,conditional 1pl,conditional 2pl,conditional 3pl,subjunctive present 1sg,subjunctive present 2sg,subjunctive present 3sg,subjunctive present 1pl,subjunctive present 2pl,subjunctive present 3pl,subjunctive imperfect 1sg,subjunctive imperfect 2sg,subjunctive imperfect 3sg,subjunctive imperfect 1pl,subjunctive imperfect 2pl,subjunctive imperfect 3pl,imperative 2sg,imperative 1pl,imperative 2pl,group
0,boire,buvant,bu,bois,bois,boit,buvons,buvez,boivent,buvais,buvais,buvait,buvions,buviez,buvaient,bus,bus,but,bûmes,bûtes,burent,boirai,boiras,boira,boirons,boirez,boiront,boirais,boirais,boirait,boirions,boiriez,boiraient,boive,boives,boive,buvions,buviez,boivent,busse,busses,bût,bussions,bussiez,bussent,bois,buvons,buvez,None


,infinitive,present participle,past participle,present 1sg,present 2sg,present 3sg,present 1pl,present 2pl,present 3pl,imperfect 1sg,imperfect 2sg,imperfect 3sg,imperfect 1pl,imperfect 2pl,imperfect 3pl,past historic 1sg,past historic 2sg,past historic 3sg,past historic 1pl,past historic 2pl,past historic 3pl,future 1sg,future 2sg,future 3sg,future 1pl,future 2pl,future 3pl,conditional 1sg,conditional 2sg,conditional 3sg,conditional 1pl,conditional 2pl,conditional 3pl,subjunctive present 1sg,subjunctive present 2sg,subjunctive present 3sg,subjunctive present 1pl,subjunctive present 2pl,subjunctive present 3pl,subjunctive imperfect 1sg,subjunctive imperfect 2sg,subjunctive imperfect 3sg,subjunctive imperfect 1pl,subjunctive imperfect 2pl,subjunctive imperfect 3pl,imperative 2sg,imperative 1pl,imperative 2pl,group
522,boire,buvant,bu,bois,bois,boit,buvons,buvez,boivent,buvais,buvais,buvait,buvions,buviez,buvaient,bus,bus,but,bûmes,bûtes,burent,boirai,boiras,boira,boirons,boirez,boiront,boirais,boirais,boirait,boirions,boiriez,boiraient,boive,boives,boive,buvions,buviez,boivent,busse,busses,bût,bussions,bussiez,bussent,bois,buvons,buvez,None
3838,reboire,rebuvant,rebu,rebois,rebois,reboit,rebuvons,rebuvez,reboivent,rebuvais,rebuvais,rebuvait,rebuvions,rebuviez,rebuvaient,rebus,rebus,rebut,rebûmes,rebûtes,reburent,reboirai,reboiras,reboira,reboirons,reboirez,reboiront,reboirais,reboirais,reboirait,reboirions,reboiriez,reboiraient,reboive,reboives,reboive,rebuvions,rebuviez,reboivent,rebusse,rebusses,rebût,rebussions,rebussiez,rebussent,rebois,rebuvons,rebuvez,None


#### croire

In [63]:
like_croire, remaining_data = is_like("croire", "croire", "croire")
display(like_croire)

,infinitive,present participle,past participle,present 1sg,present 2sg,present 3sg,present 1pl,present 2pl,present 3pl,imperfect 1sg,imperfect 2sg,imperfect 3sg,imperfect 1pl,imperfect 2pl,imperfect 3pl,past historic 1sg,past historic 2sg,past historic 3sg,past historic 1pl,past historic 2pl,past historic 3pl,future 1sg,future 2sg,future 3sg,future 1pl,future 2pl,future 3pl,conditional 1sg,conditional 2sg,conditional 3sg,conditional 1pl,conditional 2pl,conditional 3pl,subjunctive present 1sg,subjunctive present 2sg,subjunctive present 3sg,subjunctive present 1pl,subjunctive present 2pl,subjunctive present 3pl,subjunctive imperfect 1sg,subjunctive imperfect 2sg,subjunctive imperfect 3sg,subjunctive imperfect 1pl,subjunctive imperfect 2pl,subjunctive imperfect 3pl,imperative 2sg,imperative 1pl,imperative 2pl,group
0,croire,croyant,cru,crois,crois,croit,croyons,croyez,croient,croyais,croyais,croyait,croyions,croyiez,croyaient,crus,crus,crut,crûmes,crûtes,crurent,croirai,croiras,croira,croirons,croirez,croiront,croirais,croirais,croirait,croirions,croiriez,croiraient,croie,croies,croie,croyions,croyiez,croient,crusse,crusses,crût,crussions,crussiez,crussent,crois,croyons,croyez,None


,infinitive,present participle,past participle,present 1sg,present 2sg,present 3sg,present 1pl,present 2pl,present 3pl,imperfect 1sg,imperfect 2sg,imperfect 3sg,imperfect 1pl,imperfect 2pl,imperfect 3pl,past historic 1sg,past historic 2sg,past historic 3sg,past historic 1pl,past historic 2pl,past historic 3pl,future 1sg,future 2sg,future 3sg,future 1pl,future 2pl,future 3pl,conditional 1sg,conditional 2sg,conditional 3sg,conditional 1pl,conditional 2pl,conditional 3pl,subjunctive present 1sg,subjunctive present 2sg,subjunctive present 3sg,subjunctive present 1pl,subjunctive present 2pl,subjunctive present 3pl,subjunctive imperfect 1sg,subjunctive imperfect 2sg,subjunctive imperfect 3sg,subjunctive imperfect 1pl,subjunctive imperfect 2pl,subjunctive imperfect 3pl,imperative 2sg,imperative 1pl,imperative 2pl,group
1123,croire,croyant,cru,crois,crois,croit,croyons,croyez,croient,croyais,croyais,croyait,croyions,croyiez,croyaient,crus,crus,crut,crûmes,crûtes,crurent,croirai,croiras,croira,croirons,croirez,croiront,croirais,croirais,croirait,croirions,croiriez,croiraient,croie,croies,croie,croyions,croyiez,croient,crusse,crusses,crût,crussions,crussiez,crussent,crois,croyons,croyez,None
3888,recroire,recroyant,recru,recrois,recrois,recroit,recroyons,recroyez,recroient,recroyais,recroyais,recroyait,recroyions,recroyiez,recroyaient,recrus,recrus,recrut,recrûmes,recrûtes,recrurent,recroirai,recroiras,recroira,recroirons,recroirez,recroiront,recroirais,recroirais,recroirait,recroirions,recroiriez,recroiraient,recroie,recroies,recroie,recroyions,recroyiez,recroient,recrusse,recrusses,recrût,recrussions,recrussiez,recrussent,recrois,recroyons,recroyez,None


#### prendre

In [64]:
like_prendre, remaining_data = is_like("prendre", "prendre", "prendre")
display(like_prendre)

,infinitive,present participle,past participle,present 1sg,present 2sg,present 3sg,present 1pl,present 2pl,present 3pl,imperfect 1sg,imperfect 2sg,imperfect 3sg,imperfect 1pl,imperfect 2pl,imperfect 3pl,past historic 1sg,past historic 2sg,past historic 3sg,past historic 1pl,past historic 2pl,past historic 3pl,future 1sg,future 2sg,future 3sg,future 1pl,future 2pl,future 3pl,conditional 1sg,conditional 2sg,conditional 3sg,conditional 1pl,conditional 2pl,conditional 3pl,subjunctive present 1sg,subjunctive present 2sg,subjunctive present 3sg,subjunctive present 1pl,subjunctive present 2pl,subjunctive present 3pl,subjunctive imperfect 1sg,subjunctive imperfect 2sg,subjunctive imperfect 3sg,subjunctive imperfect 1pl,subjunctive imperfect 2pl,subjunctive imperfect 3pl,imperative 2sg,imperative 1pl,imperative 2pl,group
0,prendre,prenant,pris,prends,prends,prend,prenons,prenez,prennent,prenais,prenais,prenait,prenions,preniez,prenaient,pris,pris,prit,prîmes,prîtes,prirent,prendrai,prendras,prendra,prendrons,prendrez,prendront,prendrais,prendrais,prendrait,prendrions,prendriez,prendraient,prenne,prennes,prenne,prenions,preniez,prennent,prisse,prisses,prît,prissions,prissiez,prissent,prends,prenons,prenez,None


,infinitive,present participle,past participle,present 1sg,present 2sg,present 3sg,present 1pl,present 2pl,present 3pl,imperfect 1sg,imperfect 2sg,imperfect 3sg,imperfect 1pl,imperfect 2pl,imperfect 3pl,past historic 1sg,past historic 2sg,past historic 3sg,past historic 1pl,past historic 2pl,past historic 3pl,future 1sg,future 2sg,future 3sg,future 1pl,future 2pl,future 3pl,conditional 1sg,conditional 2sg,conditional 3sg,conditional 1pl,conditional 2pl,conditional 3pl,subjunctive present 1sg,subjunctive present 2sg,subjunctive present 3sg,subjunctive present 1pl,subjunctive present 2pl,subjunctive present 3pl,subjunctive imperfect 1sg,subjunctive imperfect 2sg,subjunctive imperfect 3sg,subjunctive imperfect 1pl,subjunctive imperfect 2pl,subjunctive imperfect 3pl,imperative 2sg,imperative 1pl,imperative 2pl,group
287,apprendre,apprenant,appris,apprends,apprends,apprend,apprenons,apprenez,apprennent,apprenais,apprenais,apprenait,apprenions,appreniez,apprenaient,appris,appris,apprit,apprîmes,apprîtes,apprirent,apprendrai,apprendras,apprendra,apprendrons,apprendrez,apprendront,apprendrais,apprendrais,apprendrait,apprendrions,apprendriez,apprendraient,apprenne,apprennes,apprenne,apprenions,appreniez,apprennent,apprisse,apprisses,apprît,apprissions,apprissiez,apprissent,apprends,apprenons,apprenez,None
933,comprendre,comprenant,compris,comprends,comprends,comprend,comprenons,comprenez,comprennent,comprenais,comprenais,comprenait,comprenions,compreniez,comprenaient,compris,compris,comprit,comprîmes,comprîtes,comprirent,comprendrai,comprendras,comprendra,comprendrons,comprendrez,comprendront,comprendrais,comprendrais,comprendrait,comprendrions,comprendriez,comprendraient,comprenne,comprennes,comprenne,comprenions,compreniez,comprennent,comprisse,comprisses,comprît,comprissions,comprissiez,comprissent,comprends,comprenons,comprenez,None
1730,désapprendre,désapprenant,désappris,désapprends,désapprends,désapprend,désapprenons,désapprenez,désapprennent,désapprenais,désapprenais,désapprenait,désapprenions,désappreniez,désapprenaient,désappris,désappris,désapprit,désapprîmes,désapprîtes,désapprirent,désapprendrai,désapprendras,désapprendra,désapprendrons,désapprendrez,désapprendront,désapprendrais,désapprendrais,désapprendrait,désapprendrions,désapprendriez,désapprendraient,désapprenne,désapprennes,désapprenne,désapprenions,désappreniez,désapprennent,désapprisse,désapprisses,désapprît,désapprissions,désapprissiez,désapprissent,désapprends,désapprenons,désapprenez,None
2103,entreprendre,entreprenant,entrepris,entreprends,entreprends,entreprend,entreprenons,entreprenez,entreprennent,entreprenais,entreprenais,entreprenait,entreprenions,entrepreniez,entreprenaient,entrepris,entrepris,entreprit,entreprîmes,entreprîtes,entreprirent,entreprendrai,entreprendras,entreprendra,entreprendrons,entreprendrez,entreprendront,entreprendrais,entreprendrais,entreprendrait,entreprendrions,entreprendriez,entreprendraient,entreprenne,entreprennes,entreprenne,entreprenions,entrepreniez,entreprennent,entreprisse,entreprisses,entreprît,entreprissions,entreprissiez,entreprissent,entreprends,entreprenons,entreprenez,None
3180,méprendre,méprenant,mépris,méprends,méprends,méprend,méprenons,méprenez,méprennent,méprenais,méprenais,méprenait,méprenions,mépreniez,méprenaient,mépris,mépris,méprit,méprîmes,méprîtes,méprirent,méprendrai,méprendras,méprendra,méprendrons,méprendrez,méprendront,méprendrais,méprendrais,méprendrait,méprendrions,méprendriez,méprendraient,méprenne,méprennes,méprenne,méprenions,mépreniez,méprennent,méprisse,méprisses,méprît,méprissions,méprissiez,méprissent,méprends,méprenons,méprenez,None
3582,prendre,prenant,pris,prends,prends,prend,prenons,prenez,prennent,prenais,prenais,prenait,prenions,preniez,prenaient,pris,pris,prit,prîmes,prîtes,prirent,prendrai,prendras,prendra,prendrons,prendrez,prendront,prendrais,prendrais,prendrait,prendrions,prendriez,prendraient,prenne,prennes,prenne,prenions,preniez,prennent,prisse,prisses,prît,prissions,prissiez,pris

#### faire

In [65]:
like_faire, remaining_data = is_like("faire", "aire", "faire")
display(like_faire)

,infinitive,present participle,past participle,present 1sg,present 2sg,present 3sg,present 1pl,present 2pl,present 3pl,imperfect 1sg,imperfect 2sg,imperfect 3sg,imperfect 1pl,imperfect 2pl,imperfect 3pl,past historic 1sg,past historic 2sg,past historic 3sg,past historic 1pl,past historic 2pl,past historic 3pl,future 1sg,future 2sg,future 3sg,future 1pl,future 2pl,future 3pl,conditional 1sg,conditional 2sg,conditional 3sg,conditional 1pl,conditional 2pl,conditional 3pl,subjunctive present 1sg,subjunctive present 2sg,subjunctive present 3sg,subjunctive present 1pl,subjunctive present 2pl,subjunctive present 3pl,subjunctive imperfect 1sg,subjunctive imperfect 2sg,subjunctive imperfect 3sg,subjunctive imperfect 1pl,subjunctive imperfect 2pl,subjunctive imperfect 3pl,imperative 2sg,imperative 1pl,imperative 2pl,group
0,aire,aisant,ait,ais,ais,ait,aisons,aites,ont,aisais,aisais,aisait,aisions,aisiez,aisaient,is,is,it,îmes,îtes,irent,erai,eras,era,erons,erez,eront,erais,erais,erait,erions,eriez,eraient,asse,asses,asse,assions,assiez,assent,isse,isses,ît,issions,issiez,issent,ais,aisons,aites,None


,infinitive,present participle,past participle,present 1sg,present 2sg,present 3sg,present 1pl,present 2pl,present 3pl,imperfect 1sg,imperfect 2sg,imperfect 3sg,imperfect 1pl,imperfect 2pl,imperfect 3pl,past historic 1sg,past historic 2sg,past historic 3sg,past historic 1pl,past historic 2pl,past historic 3pl,future 1sg,future 2sg,future 3sg,future 1pl,future 2pl,future 3pl,conditional 1sg,conditional 2sg,conditional 3sg,conditional 1pl,conditional 2pl,conditional 3pl,subjunctive present 1sg,subjunctive present 2sg,subjunctive present 3sg,subjunctive present 1pl,subjunctive present 2pl,subjunctive present 3pl,subjunctive imperfect 1sg,subjunctive imperfect 2sg,subjunctive imperfect 3sg,subjunctive imperfect 1pl,subjunctive imperfect 2pl,subjunctive imperfect 3pl,imperative 2sg,imperative 1pl,imperative 2pl,group
1026,contrefaire,contrefaisant,contrefait,contrefais,contrefais,contrefait,contrefaisons,contrefaites,contrefont,contrefaisais,contrefaisais,contrefaisait,contrefaisions,contrefaisiez,contrefaisaient,contrefis,contrefis,contrefit,contrefîmes,contrefîtes,contrefirent,contreferai,contreferas,contrefera,contreferons,contreferez,contreferont,contreferais,contreferais,contreferait,contreferions,contreferiez,contreferaient,contrefasse,contrefasses,contrefasse,contrefassions,contrefassiez,contrefassent,contrefisse,contrefisses,contrefît,contrefissions,contrefissiez,contrefissent,contrefais,contrefaisons,contrefaites,None
1485,défaire,défaisant,défait,défais,défais,défait,défaisons,défaites,défont,défaisais,défaisais,défaisait,défaisions,défaisiez,défaisaient,défis,défis,défit,défîmes,défîtes,défirent,déferai,déferas,défera,déferons,déferez,déferont,déferais,déferais,déferait,déferions,déferiez,déferaient,défasse,défasses,défasse,défassions,défassiez,défassent,défisse,défisses,défît,défissions,défissiez,défissent,défais,défaisons,défaites,None
2248,faire,faisant,fait,fais,fais,fait,faisons,faites,font,faisais,faisais,faisait,faisions,faisiez,faisaient,fis,fis,fit,fîmes,fîtes,firent,ferai,feras,fera,ferons,ferez,feront,ferais,ferais,ferait,ferions,feriez,feraient,fasse,fasses,fasse,fassions,fassiez,fassent,fisse,fisses,fît,fissions,fissiez,fissent,fais,faisons,faites,None
2997,malfaire,malfaisant,malfait,malfais,malfais,malfait,malfaisons,malfaites,malfont,malfaisais,malfaisais,malfaisait,malfaisions,malfaisiez,malfaisaient,malfis,malfis,malfit,malfîmes,malfîtes,malfirent,malferai,malferas,malfera,malferons,malferez,malferont,malferais,malferais,malferait,malferions,malferiez,malferaient,malfasse,malfasses,malfasse,malfassions,malfassiez,malfassent,malfisse,malfisses,malfît,malfissions,malfissiez,malfissent,malfais,malfaisons,malfaites,None
3172,méfaire,méfaisant,méfait,méfais,méfais,méfait,méfaisons,méfaites,méfont,méfaisais,méfaisais,méfaisait,méfaisions,méfaisiez,méfaisaient,méfis,méfis,méfit,méfîmes,méfîtes,méfirent,méferai,méferas,méfera,méferons,méferez,méferont,méferais,méferais,méferait,méferions,méferiez,méferaient,méfasse,méfasses,méfasse,méfassions,méfassiez,méfassent,méfisse,méfisses,méfît,méfissions,méfissiez,méfissent,méfais,méfaisons,méfaites,None
3362,parfaire,parfaisant,parfait,parfais,parfais,parfait,parfaisons,parfaites,parfont,parfaisais,parfaisais,parfaisait,parfaisions,parfaisiez,parfaisaient,parfis,parfis,parfit,parfîmes,parfîtes,parfirent,parferai,parferas,parfera,parferons,parferez,parferont,parferais,parferais,parferait,parferions,parferiez,parferaient,parfasse,parfasses,parfasse,parfassions,parfassiez,parfassent,parfisse,parfisses,parfît,parfissions,parfissiez,parfissent,parfais,parfaisons,parfaites,None
3923,refaire,refaisant,refait,refais,refais,refait,refaisons,refaites,refont,refaisais,refaisais,refaisait,refaisions,refaisiez,refaisaient,refis,refis,refit,refîmes,refîtes,refirent,referai,referas,refera,referons,referez,referont,referais,referais,referait,referions,referiez,referaient,refasse,refasses,refasse,refassions,refassiez,refassent,refisse,refisses,refît,refissions,refissiez,refissent,refai

#### taire

In [66]:
like_taire, remaining_data = is_like("taire", "aire", "taire")
display(like_taire)

,infinitive,present participle,past participle,present 1sg,present 2sg,present 3sg,present 1pl,present 2pl,present 3pl,imperfect 1sg,imperfect 2sg,imperfect 3sg,imperfect 1pl,imperfect 2pl,imperfect 3pl,past historic 1sg,past historic 2sg,past historic 3sg,past historic 1pl,past historic 2pl,past historic 3pl,future 1sg,future 2sg,future 3sg,future 1pl,future 2pl,future 3pl,conditional 1sg,conditional 2sg,conditional 3sg,conditional 1pl,conditional 2pl,conditional 3pl,subjunctive present 1sg,subjunctive present 2sg,subjunctive present 3sg,subjunctive present 1pl,subjunctive present 2pl,subjunctive present 3pl,subjunctive imperfect 1sg,subjunctive imperfect 2sg,subjunctive imperfect 3sg,subjunctive imperfect 1pl,subjunctive imperfect 2pl,subjunctive imperfect 3pl,imperative 2sg,imperative 1pl,imperative 2pl,group
0,aire,aisant,u,ais,ais,ait,aisons,aisez,aisent,aisais,aisais,aisait,aisions,aisiez,aisaient,us,us,ut,ûmes,ûtes,urent,airai,airas,aira,airons,airez,airont,airais,airais,airait,airions,airiez,airaient,aise,aises,aise,aisions,aisiez,aisent,usse,usses,ût,ussions,ussiez,ussent,ais,aisons,aisez,None


,infinitive,present participle,past participle,present 1sg,present 2sg,present 3sg,present 1pl,present 2pl,present 3pl,imperfect 1sg,imperfect 2sg,imperfect 3sg,imperfect 1pl,imperfect 2pl,imperfect 3pl,past historic 1sg,past historic 2sg,past historic 3sg,past historic 1pl,past historic 2pl,past historic 3pl,future 1sg,future 2sg,future 3sg,future 1pl,future 2pl,future 3pl,conditional 1sg,conditional 2sg,conditional 3sg,conditional 1pl,conditional 2pl,conditional 3pl,subjunctive present 1sg,subjunctive present 2sg,subjunctive present 3sg,subjunctive present 1pl,subjunctive present 2pl,subjunctive present 3pl,subjunctive imperfect 1sg,subjunctive imperfect 2sg,subjunctive imperfect 3sg,subjunctive imperfect 1pl,subjunctive imperfect 2pl,subjunctive imperfect 3pl,imperative 2sg,imperative 1pl,imperative 2pl,group
4744,taire,taisant,tu,tais,tais,tait,taisons,taisez,taisent,taisais,taisais,taisait,taisions,taisiez,taisaient,tus,tus,tut,tûmes,tûtes,turent,tairai,tairas,taira,tairons,tairez,tairont,tairais,tairais,tairait,tairions,tairiez,tairaient,taise,taises,taise,taisions,taisiez,taisent,tusse,tusses,tût,tussions,tussiez,tussent,tais,taisons,taisez,None


#### plaire

In [67]:
like_plaire, remaining_data = is_like("plaire", "aire", "plaire")
display(like_plaire)

,infinitive,present participle,past participle,present 1sg,present 2sg,present 3sg,present 1pl,present 2pl,present 3pl,imperfect 1sg,imperfect 2sg,imperfect 3sg,imperfect 1pl,imperfect 2pl,imperfect 3pl,past historic 1sg,past historic 2sg,past historic 3sg,past historic 1pl,past historic 2pl,past historic 3pl,future 1sg,future 2sg,future 3sg,future 1pl,future 2pl,future 3pl,conditional 1sg,conditional 2sg,conditional 3sg,conditional 1pl,conditional 2pl,conditional 3pl,subjunctive present 1sg,subjunctive present 2sg,subjunctive present 3sg,subjunctive present 1pl,subjunctive present 2pl,subjunctive present 3pl,subjunctive imperfect 1sg,subjunctive imperfect 2sg,subjunctive imperfect 3sg,subjunctive imperfect 1pl,subjunctive imperfect 2pl,subjunctive imperfect 3pl,imperative 2sg,imperative 1pl,imperative 2pl,group
0,aire,aisant,u,ais,ais,"(aît, ait)",aisons,aisez,aisent,aisais,aisais,aisait,aisions,aisiez,aisaient,us,us,ut,ûmes,ûtes,urent,airai,airas,aira,airons,airez,airont,airais,airais,airait,airions,airiez,airaient,aise,aises,aise,aisions,aisiez,aisent,usse,usses,ût,ussions,ussiez,ussent,ais,aisons,aisez,None


,infinitive,present participle,past participle,present 1sg,present 2sg,present 3sg,present 1pl,present 2pl,present 3pl,imperfect 1sg,imperfect 2sg,imperfect 3sg,imperfect 1pl,imperfect 2pl,imperfect 3pl,past historic 1sg,past historic 2sg,past historic 3sg,past historic 1pl,past historic 2pl,past historic 3pl,future 1sg,future 2sg,future 3sg,future 1pl,future 2pl,future 3pl,conditional 1sg,conditional 2sg,conditional 3sg,conditional 1pl,conditional 2pl,conditional 3pl,subjunctive present 1sg,subjunctive present 2sg,subjunctive present 3sg,subjunctive present 1pl,subjunctive present 2pl,subjunctive present 3pl,subjunctive imperfect 1sg,subjunctive imperfect 2sg,subjunctive imperfect 3sg,subjunctive imperfect 1pl,subjunctive imperfect 2pl,subjunctive imperfect 3pl,imperative 2sg,imperative 1pl,imperative 2pl,group
922,complaire,complaisant,complu,complais,complais,"(complaît, complait)",complaisons,complaisez,complaisent,complaisais,complaisais,complaisait,complaisions,complaisiez,complaisaient,complus,complus,complut,complûmes,complûtes,complurent,complairai,complairas,complaira,complairons,complairez,complairont,complairais,complairais,complairait,complairions,complairiez,complairaient,complaise,complaises,complaise,complaisions,complaisiez,complaisent,complusse,complusses,complût,complussions,complussiez,complussent,complais,complaisons,complaisez,None
1669,déplaire,déplaisant,déplu,déplais,déplais,"(déplaît, déplait)",déplaisons,déplaisez,déplaisent,déplaisais,déplaisais,déplaisait,déplaisions,déplaisiez,déplaisaient,déplus,déplus,déplut,déplûmes,déplûtes,déplurent,déplairai,déplairas,déplaira,déplairons,déplairez,déplairont,déplairais,déplairais,déplairait,déplairions,déplairiez,déplairaient,déplaise,déplaises,déplaise,déplaisions,déplaisiez,déplaisent,déplusse,déplusses,déplût,déplussions,déplussiez,déplussent,déplais,déplaisons,déplaisez,None
3494,plaire,plaisant,plu,plais,plais,"(plaît, plait)",plaisons,plaisez,plaisent,plaisais,plaisais,plaisait,plaisions,plaisiez,plaisaient,plus,plus,plut,plûmes,plûtes,plurent,plairai,plairas,plaira,plairons,plairez,plairont,plairais,plairais,plairait,plairions,plairiez,plairaient,plaise,plaises,plaise,plaisions,plaisiez,plaisent,plusse,plusses,plût,plussions,plussiez,plussent,plais,plaisons,plaisez,None


#### lire

In [68]:
like_lire, remaining_data = is_like("lire", "ire", "lire")
display(like_lire)

,infinitive,present participle,past participle,present 1sg,present 2sg,present 3sg,present 1pl,present 2pl,present 3pl,imperfect 1sg,imperfect 2sg,imperfect 3sg,imperfect 1pl,imperfect 2pl,imperfect 3pl,past historic 1sg,past historic 2sg,past historic 3sg,past historic 1pl,past historic 2pl,past historic 3pl,future 1sg,future 2sg,future 3sg,future 1pl,future 2pl,future 3pl,conditional 1sg,conditional 2sg,conditional 3sg,conditional 1pl,conditional 2pl,conditional 3pl,subjunctive present 1sg,subjunctive present 2sg,subjunctive present 3sg,subjunctive present 1pl,subjunctive present 2pl,subjunctive present 3pl,subjunctive imperfect 1sg,subjunctive imperfect 2sg,subjunctive imperfect 3sg,subjunctive imperfect 1pl,subjunctive imperfect 2pl,subjunctive imperfect 3pl,imperative 2sg,imperative 1pl,imperative 2pl,group
0,ire,isant,u,is,is,it,isons,isez,isent,isais,isais,isait,isions,isiez,isaient,us,us,ut,ûmes,ûtes,urent,irai,iras,ira,irons,irez,iront,irais,irais,irait,irions,iriez,iraient,ise,ises,ise,isions,isiez,isent,usse,usses,ût,ussions,ussiez,ussent,is,isons,isez,None


,infinitive,present participle,past participle,present 1sg,present 2sg,present 3sg,present 1pl,present 2pl,present 3pl,imperfect 1sg,imperfect 2sg,imperfect 3sg,imperfect 1pl,imperfect 2pl,imperfect 3pl,past historic 1sg,past historic 2sg,past historic 3sg,past historic 1pl,past historic 2pl,past historic 3pl,future 1sg,future 2sg,future 3sg,future 1pl,future 2pl,future 3pl,conditional 1sg,conditional 2sg,conditional 3sg,conditional 1pl,conditional 2pl,conditional 3pl,subjunctive present 1sg,subjunctive present 2sg,subjunctive present 3sg,subjunctive present 1pl,subjunctive present 2pl,subjunctive present 3pl,subjunctive imperfect 1sg,subjunctive imperfect 2sg,subjunctive imperfect 3sg,subjunctive imperfect 1pl,subjunctive imperfect 2pl,subjunctive imperfect 3pl,imperative 2sg,imperative 1pl,imperative 2pl,group
2939,lire,lisant,lu,lis,lis,lit,lisons,lisez,lisent,lisais,lisais,lisait,lisions,lisiez,lisaient,lus,lus,lut,lûmes,lûtes,lurent,lirai,liras,lira,lirons,lirez,liront,lirais,lirais,lirait,lirions,liriez,liraient,lise,lises,lise,lisions,lisiez,lisent,lusse,lusses,lût,lussions,lussiez,lussent,lis,lisons,lisez,None
3962,relire,relisant,relu,relis,relis,relit,relisons,relisez,relisent,relisais,relisais,relisait,relisions,relisiez,relisaient,relus,relus,relut,relûmes,relûtes,relurent,relirai,reliras,relira,relirons,relirez,reliront,relirais,relirais,relirait,relirions,reliriez,reliraient,relise,relises,relise,relisions,relisiez,relisent,relusse,relusses,relût,relussions,relussiez,relussent,relis,relisons,relisez,None
4387,réélire,réélisant,réélu,réélis,réélis,réélit,réélisons,réélisez,réélisent,réélisais,réélisais,réélisait,réélisions,réélisiez,réélisaient,réélus,réélus,réélut,réélûmes,réélûtes,réélurent,réélirai,rééliras,réélira,réélirons,réélirez,rééliront,réélirais,réélirais,réélirait,réélirions,rééliriez,rééliraient,réélise,réélises,réélise,réélisions,réélisiez,réélisent,réélusse,réélusses,réélût,réélussions,réélussiez,réélussent,réélis,réélisons,réélisez,None
5203,élire,élisant,élu,élis,élis,élit,élisons,élisez,élisent,élisais,élisais,élisait,élisions,élisiez,élisaient,élus,élus,élut,élûmes,élûtes,élurent,élirai,éliras,élira,élirons,élirez,éliront,élirais,élirais,élirait,élirions,éliriez,éliraient,élise,élises,élise,élisions,élisiez,élisent,élusse,élusses,élût,élussions,élussiez,élussent,élis,élisons,élisez,None


#### vivre

In [69]:
like_vivre, remaining_data = is_like("vivre", "ivre", "vivre")
display(like_vivre)

,infinitive,present participle,past participle,present 1sg,present 2sg,present 3sg,present 1pl,present 2pl,present 3pl,imperfect 1sg,imperfect 2sg,imperfect 3sg,imperfect 1pl,imperfect 2pl,imperfect 3pl,past historic 1sg,past historic 2sg,past historic 3sg,past historic 1pl,past historic 2pl,past historic 3pl,future 1sg,future 2sg,future 3sg,future 1pl,future 2pl,future 3pl,conditional 1sg,conditional 2sg,conditional 3sg,conditional 1pl,conditional 2pl,conditional 3pl,subjunctive present 1sg,subjunctive present 2sg,subjunctive present 3sg,subjunctive present 1pl,subjunctive present 2pl,subjunctive present 3pl,subjunctive imperfect 1sg,subjunctive imperfect 2sg,subjunctive imperfect 3sg,subjunctive imperfect 1pl,subjunctive imperfect 2pl,subjunctive imperfect 3pl,imperative 2sg,imperative 1pl,imperative 2pl,group
0,ivre,ivant,écu,is,is,it,ivons,ivez,ivent,ivais,ivais,ivait,ivions,iviez,ivaient,écus,écus,écut,écûmes,écûtes,écurent,ivrai,ivras,ivra,ivrons,ivrez,ivront,ivrais,ivrais,ivrait,ivrions,ivriez,ivraient,ive,ives,ive,ivions,iviez,ivent,écusse,écusses,écût,écussions,écussiez,écussent,is,ivons,ivez,None


,infinitive,present participle,past participle,present 1sg,present 2sg,present 3sg,present 1pl,present 2pl,present 3pl,imperfect 1sg,imperfect 2sg,imperfect 3sg,imperfect 1pl,imperfect 2pl,imperfect 3pl,past historic 1sg,past historic 2sg,past historic 3sg,past historic 1pl,past historic 2pl,past historic 3pl,future 1sg,future 2sg,future 3sg,future 1pl,future 2pl,future 3pl,conditional 1sg,conditional 2sg,conditional 3sg,conditional 1pl,conditional 2pl,conditional 3pl,subjunctive present 1sg,subjunctive present 2sg,subjunctive present 3sg,subjunctive present 1pl,subjunctive present 2pl,subjunctive present 3pl,subjunctive imperfect 1sg,subjunctive imperfect 2sg,subjunctive imperfect 3sg,subjunctive imperfect 1pl,subjunctive imperfect 2pl,subjunctive imperfect 3pl,imperative 2sg,imperative 1pl,imperative 2pl,group
4156,revivre,revivant,revécu,revis,revis,revit,revivons,revivez,revivent,revivais,revivais,revivait,revivions,reviviez,revivaient,revécus,revécus,revécut,revécûmes,revécûtes,revécurent,revivrai,revivras,revivra,revivrons,revivrez,revivront,revivrais,revivrais,revivrait,revivrions,revivriez,revivraient,revive,revives,revive,revivions,reviviez,revivent,revécusse,revécusses,revécût,revécussions,revécussiez,revécussent,revis,revivons,revivez,None
4697,survivre,survivant,survécu,survis,survis,survit,survivons,survivez,survivent,survivais,survivais,survivait,survivions,surviviez,survivaient,survécus,survécus,survécut,survécûmes,survécûtes,survécurent,survivrai,survivras,survivra,survivrons,survivrez,survivront,survivrais,survivrais,survivrait,survivrions,survivriez,survivraient,survive,survives,survive,survivions,surviviez,survivent,survécusse,survécusses,survécût,survécussions,survécussiez,survécussent,survis,survivons,survivez,None
5052,vivre,vivant,vécu,vis,vis,vit,vivons,vivez,vivent,vivais,vivais,vivait,vivions,viviez,vivaient,vécus,vécus,vécut,vécûmes,vécûtes,vécurent,vivrai,vivras,vivra,vivrons,vivrez,vivront,vivrais,vivrais,vivrait,vivrions,vivriez,vivraient,vive,vives,vive,vivions,viviez,vivent,vécusse,vécusses,vécût,vécussions,vécussiez,vécussent,vis,vivons,vivez,None


#### connaître/connaitre...

In [70]:
like_connaitre, remaining_data = is_like("connaître", "aître", "connaître")
like_connaitre2, remaining_data = is_like("connaitre", "aitre", "connaitre")
display(pd.concat([like_connaitre, like_connaitre2]).sort_values("infinitive"))

,infinitive,present participle,past participle,present 1sg,present 2sg,present 3sg,present 1pl,present 2pl,present 3pl,imperfect 1sg,imperfect 2sg,imperfect 3sg,imperfect 1pl,imperfect 2pl,imperfect 3pl,past historic 1sg,past historic 2sg,past historic 3sg,past historic 1pl,past historic 2pl,past historic 3pl,future 1sg,future 2sg,future 3sg,future 1pl,future 2pl,future 3pl,conditional 1sg,conditional 2sg,conditional 3sg,conditional 1pl,conditional 2pl,conditional 3pl,subjunctive present 1sg,subjunctive present 2sg,subjunctive present 3sg,subjunctive present 1pl,subjunctive present 2pl,subjunctive present 3pl,subjunctive imperfect 1sg,subjunctive imperfect 2sg,subjunctive imperfect 3sg,subjunctive imperfect 1pl,subjunctive imperfect 2pl,subjunctive imperfect 3pl,imperative 2sg,imperative 1pl,imperative 2pl,group
0,aître,aissant,u,ais,ais,aît,aissons,aissez,aissent,aissais,aissais,aissait,aissions,aissiez,aissaient,us,us,ut,ûmes,ûtes,urent,aîtrai,aîtras,aîtra,aîtrons,aîtrez,aîtront,aîtrais,aîtrais,aîtrait,aîtrions,aîtriez,aîtraient,aisse,aisses,aisse,aissions,aissiez,aissent,usse,usses,ût,ussions,ussiez,ussent,ais,aissons,aissez,None


,infinitive,present participle,past participle,present 1sg,present 2sg,present 3sg,present 1pl,present 2pl,present 3pl,imperfect 1sg,imperfect 2sg,imperfect 3sg,imperfect 1pl,imperfect 2pl,imperfect 3pl,past historic 1sg,past historic 2sg,past historic 3sg,past historic 1pl,past historic 2pl,past historic 3pl,future 1sg,future 2sg,future 3sg,future 1pl,future 2pl,future 3pl,conditional 1sg,conditional 2sg,conditional 3sg,conditional 1pl,conditional 2pl,conditional 3pl,subjunctive present 1sg,subjunctive present 2sg,subjunctive present 3sg,subjunctive present 1pl,subjunctive present 2pl,subjunctive present 3pl,subjunctive imperfect 1sg,subjunctive imperfect 2sg,subjunctive imperfect 3sg,subjunctive imperfect 1pl,subjunctive imperfect 2pl,subjunctive imperfect 3pl,imperative 2sg,imperative 1pl,imperative 2pl,group
0,aitre,aissant,u,ais,ais,ait,aissons,aissez,aissent,aissais,aissais,aissait,aissions,aissiez,aissaient,us,us,ut,ûmes,ûtes,urent,aitrai,aitras,aitra,aitrons,aitrez,aitront,aitrais,aitrais,aitrait,aitrions,aitriez,aitraient,aisse,aisses,aisse,aissions,aissiez,aissent,usse,usses,ût,ussions,ussiez,ussent,ais,aissons,aissez,None


,infinitive,present participle,past participle,present 1sg,present 2sg,present 3sg,present 1pl,present 2pl,present 3pl,imperfect 1sg,imperfect 2sg,imperfect 3sg,imperfect 1pl,imperfect 2pl,imperfect 3pl,past historic 1sg,past historic 2sg,past historic 3sg,past historic 1pl,past historic 2pl,past historic 3pl,future 1sg,future 2sg,future 3sg,future 1pl,future 2pl,future 3pl,conditional 1sg,conditional 2sg,conditional 3sg,conditional 1pl,conditional 2pl,conditional 3pl,subjunctive present 1sg,subjunctive present 2sg,subjunctive present 3sg,subjunctive present 1pl,subjunctive present 2pl,subjunctive present 3pl,subjunctive imperfect 1sg,subjunctive imperfect 2sg,subjunctive imperfect 3sg,subjunctive imperfect 1pl,subjunctive imperfect 2pl,subjunctive imperfect 3pl,imperative 2sg,imperative 1pl,imperative 2pl,group
272,apparaitre,apparaissant,apparu,apparais,apparais,apparait,apparaissons,apparaissez,apparaissent,apparaissais,apparaissais,apparaissait,apparaissions,apparaissiez,apparaissaient,apparus,apparus,apparut,apparûmes,apparûtes,apparurent,apparaitrai,apparaitras,apparaitra,apparaitrons,apparaitrez,apparaitront,apparaitrais,apparaitrais,apparaitrait,apparaitrions,apparaitriez,apparaitraient,apparaisse,apparaisses,apparaisse,apparaissions,apparaissiez,apparaissent,apparusse,apparusses,apparût,apparussions,apparussiez,apparussent,apparais,apparaissons,apparaissez,None
273,apparaître,apparaissant,apparu,apparais,apparais,apparaît,apparaissons,apparaissez,apparaissent,apparaissais,apparaissais,apparaissait,apparaissions,apparaissiez,apparaissaient,apparus,apparus,apparut,apparûmes,apparûtes,apparurent,apparaîtrai,apparaîtras,apparaîtra,apparaîtrons,apparaîtrez,apparaîtront,apparaîtrais,apparaîtrais,apparaîtrait,apparaîtrions,apparaîtriez,apparaîtraient,apparaisse,apparaisses,apparaisse,apparaissions,apparaissiez,apparaissent,apparusse,apparusses,apparût,apparussions,apparussiez,apparussent,apparais,apparaissons,apparaissez,None
913,comparaitre,comparaissant,comparu,comparais,comparais,comparait,comparaissons,comparaissez,comparaissent,comparaissais,comparaissais,comparaissait,comparaissions,comparaissiez,comparaissaient,comparus,comparus,comparut,comparûmes,comparûtes,comparurent,comparaitrai,comparaitras,comparaitra,comparaitrons,comparaitrez,comparaitront,comparaitrais,comparaitrais,comparaitrait,comparaitrions,comparaitriez,comparaitraient,comparaisse,comparaisses,comparaisse,comparaissions,comparaissiez,comparaissent,comparusse,comparusses,comparût,comparussions,comparussiez,comparussent,comparais,comparaissons,comparaissez,None
914,comparaître,comparaissant,comparu,comparais,comparais,comparaît,comparaissons,comparaissez,comparaissent,comparaissais,comparaissais,comparaissait,comparaissions,comparaissiez,comparaissaient,comparus,comparus,comparut,comparûmes,comparûtes,comparurent,comparaîtrai,comparaîtras,comparaîtra,comparaîtrons,comparaîtrez,comparaîtront,comparaîtrais,comparaîtrais,comparaîtrait,comparaîtrions,comparaîtriez,comparaîtraient,comparaisse,comparaisses,comparaisse,comparaissions,comparaissiez,comparaissent,comparusse,comparusses,comparût,comparussions,comparussiez,comparussent,comparais,comparaissons,comparaissez,None
981,connaitre,connaissant,connu,connais,connais,connait,connaissons,connaissez,connaissent,connaissais,connaissais,connaissait,connaissions,connaissiez,connaissaient,connus,connus,connut,connûmes,connûtes,connurent,connaitrai,connaitras,connaitra,connaitrons,connaitrez,connaitront,connaitrais,connaitrais,connaitrait,connaitrions,connaitriez,connaitraient,connaisse,connaisses,connaisse,connaissions,connaissiez,connaissent,connusse,connusses,connût,connussions,connussiez,connussent,connais,connaissons,connaissez,None
982,connaître,connaissant,connu,connais,connais,connaît,connaissons,connaissez,connaissent,connaissais,connaissais,connaissait,connaissions,connaissiez,connaissaient,connus,connus,connut,connûmes,connûtes,connurent,connaîtrai,connaîtras,connaîtra,connaîtrons,connaîtrez,connaîtront,c

#### naitre/naître

In [71]:
like_naitre, remaining_data = is_like("naître", "aître", "naître")
like_naitre2, remaining_data = is_like("naitre", "aitre", "naitre")
display(pd.concat([like_naitre, like_naitre2]).sort_values("infinitive"))

,infinitive,present participle,past participle,present 1sg,present 2sg,present 3sg,present 1pl,present 2pl,present 3pl,imperfect 1sg,imperfect 2sg,imperfect 3sg,imperfect 1pl,imperfect 2pl,imperfect 3pl,past historic 1sg,past historic 2sg,past historic 3sg,past historic 1pl,past historic 2pl,past historic 3pl,future 1sg,future 2sg,future 3sg,future 1pl,future 2pl,future 3pl,conditional 1sg,conditional 2sg,conditional 3sg,conditional 1pl,conditional 2pl,conditional 3pl,subjunctive present 1sg,subjunctive present 2sg,subjunctive present 3sg,subjunctive present 1pl,subjunctive present 2pl,subjunctive present 3pl,subjunctive imperfect 1sg,subjunctive imperfect 2sg,subjunctive imperfect 3sg,subjunctive imperfect 1pl,subjunctive imperfect 2pl,subjunctive imperfect 3pl,imperative 2sg,imperative 1pl,imperative 2pl,group
0,aître,aissant,é,ais,ais,aît,aissons,aissez,aissent,aissais,aissais,aissait,aissions,aissiez,aissaient,aquis,aquis,aquit,aquîmes,aquîtes,aquirent,aîtrai,aîtras,aîtra,aîtrons,aîtrez,aîtront,aîtrais,aîtrais,aîtrait,aîtrions,aîtriez,aîtraient,aisse,aisses,aisse,aissions,aissiez,aissent,aquisse,aquisses,aquît,aquissions,aquissiez,aquissent,ais,aissons,aissez,None


,infinitive,present participle,past participle,present 1sg,present 2sg,present 3sg,present 1pl,present 2pl,present 3pl,imperfect 1sg,imperfect 2sg,imperfect 3sg,imperfect 1pl,imperfect 2pl,imperfect 3pl,past historic 1sg,past historic 2sg,past historic 3sg,past historic 1pl,past historic 2pl,past historic 3pl,future 1sg,future 2sg,future 3sg,future 1pl,future 2pl,future 3pl,conditional 1sg,conditional 2sg,conditional 3sg,conditional 1pl,conditional 2pl,conditional 3pl,subjunctive present 1sg,subjunctive present 2sg,subjunctive present 3sg,subjunctive present 1pl,subjunctive present 2pl,subjunctive present 3pl,subjunctive imperfect 1sg,subjunctive imperfect 2sg,subjunctive imperfect 3sg,subjunctive imperfect 1pl,subjunctive imperfect 2pl,subjunctive imperfect 3pl,imperative 2sg,imperative 1pl,imperative 2pl,group
0,aitre,aissant,é,ais,ais,ait,aissons,aissez,aissent,aissais,aissais,aissait,aissions,aissiez,aissaient,aquis,aquis,aquit,aquîmes,aquîtes,aquirent,aitrai,aitras,aitra,aitrons,aitrez,aitront,aitrais,aitrais,aitrait,aitrions,aitriez,aitraient,aisse,aisses,aisse,aissions,aissiez,aissent,aquisse,aquisses,aquît,aquissions,aquissiez,aquissent,ais,aissons,aissez,None


,infinitive,present participle,past participle,present 1sg,present 2sg,present 3sg,present 1pl,present 2pl,present 3pl,imperfect 1sg,imperfect 2sg,imperfect 3sg,imperfect 1pl,imperfect 2pl,imperfect 3pl,past historic 1sg,past historic 2sg,past historic 3sg,past historic 1pl,past historic 2pl,past historic 3pl,future 1sg,future 2sg,future 3sg,future 1pl,future 2pl,future 3pl,conditional 1sg,conditional 2sg,conditional 3sg,conditional 1pl,conditional 2pl,conditional 3pl,subjunctive present 1sg,subjunctive present 2sg,subjunctive present 3sg,subjunctive present 1pl,subjunctive present 2pl,subjunctive present 3pl,subjunctive imperfect 1sg,subjunctive imperfect 2sg,subjunctive imperfect 3sg,subjunctive imperfect 1pl,subjunctive imperfect 2pl,subjunctive imperfect 3pl,imperative 2sg,imperative 1pl,imperative 2pl,group
3194,naitre,naissant,né,nais,nais,nait,naissons,naissez,naissent,naissais,naissais,naissait,naissions,naissiez,naissaient,naquis,naquis,naquit,naquîmes,naquîtes,naquirent,naitrai,naitras,naitra,naitrons,naitrez,naitront,naitrais,naitrais,naitrait,naitrions,naitriez,naitraient,naisse,naisses,naisse,naissions,naissiez,naissent,naquisse,naquisses,naquît,naquissions,naquissiez,naquissent,nais,naissons,naissez,None
3209,naître,naissant,né,nais,nais,naît,naissons,naissez,naissent,naissais,naissais,naissait,naissions,naissiez,naissaient,naquis,naquis,naquit,naquîmes,naquîtes,naquirent,naîtrai,naîtras,naîtra,naîtrons,naîtrez,naîtront,naîtrais,naîtrais,naîtrait,naîtrions,naîtriez,naîtraient,naisse,naisses,naisse,naissions,naissiez,naissent,naquisse,naquisses,naquît,naquissions,naquissiez,naquissent,nais,naissons,naissez,None
4007,renaitre,renaissant,rené,renais,renais,renait,renaissons,renaissez,renaissent,renaissais,renaissais,renaissait,renaissions,renaissiez,renaissaient,renaquis,renaquis,renaquit,renaquîmes,renaquîtes,renaquirent,renaitrai,renaitras,renaitra,renaitrons,renaitrez,renaitront,renaitrais,renaitrais,renaitrait,renaitrions,renaitriez,renaitraient,renaisse,renaisses,renaisse,renaissions,renaissiez,renaissent,renaquisse,renaquisses,renaquît,renaquissions,renaquissiez,renaquissent,renais,renaissons,renaissez,None
4008,renaître,renaissant,rené,renais,renais,renaît,renaissons,renaissez,renaissent,renaissais,renaissais,renaissait,renaissions,renaissiez,renaissaient,renaquis,renaquis,renaquit,renaquîmes,renaquîtes,renaquirent,renaîtrai,renaîtras,renaîtra,renaîtrons,renaîtrez,renaîtront,renaîtrais,renaîtrais,renaîtrait,renaîtrions,renaîtriez,renaîtraient,renaisse,renaisses,renaisse,renaissions,renaissiez,renaissent,renaquisse,renaquisses,renaquît,renaquissions,renaquissiez,renaquissent,renais,renaissons,renaissez,None


#### croître

In [72]:
like_croitre, remaining_data = is_like("croître", "oître", "croître")
display(like_croitre)

,infinitive,present participle,past participle,present 1sg,present 2sg,present 3sg,present 1pl,present 2pl,present 3pl,imperfect 1sg,imperfect 2sg,imperfect 3sg,imperfect 1pl,imperfect 2pl,imperfect 3pl,past historic 1sg,past historic 2sg,past historic 3sg,past historic 1pl,past historic 2pl,past historic 3pl,future 1sg,future 2sg,future 3sg,future 1pl,future 2pl,future 3pl,conditional 1sg,conditional 2sg,conditional 3sg,conditional 1pl,conditional 2pl,conditional 3pl,subjunctive present 1sg,subjunctive present 2sg,subjunctive present 3sg,subjunctive present 1pl,subjunctive present 2pl,subjunctive present 3pl,subjunctive imperfect 1sg,subjunctive imperfect 2sg,subjunctive imperfect 3sg,subjunctive imperfect 1pl,subjunctive imperfect 2pl,subjunctive imperfect 3pl,imperative 2sg,imperative 1pl,imperative 2pl,group
0,oître,oissant,û,oîs,oîs,oît,oissons,oissez,oissent,oissais,oissais,oissait,oissions,oissiez,oissaient,ûs,ûs,ût,ûmes,ûtes,urent,oîtrai,oîtras,oîtra,oîtrons,oîtrez,oîtront,oîtrais,oîtrais,oîtrait,oîtrions,oîtriez,oîtraient,oisse,oisses,oisse,oissions,oissiez,oissent,ûsse,ûsses,ût,ussions,ussiez,ussent,oîs,oissons,oissez,None


,infinitive,present participle,past participle,present 1sg,present 2sg,present 3sg,present 1pl,present 2pl,present 3pl,imperfect 1sg,imperfect 2sg,imperfect 3sg,imperfect 1pl,imperfect 2pl,imperfect 3pl,past historic 1sg,past historic 2sg,past historic 3sg,past historic 1pl,past historic 2pl,past historic 3pl,future 1sg,future 2sg,future 3sg,future 1pl,future 2pl,future 3pl,conditional 1sg,conditional 2sg,conditional 3sg,conditional 1pl,conditional 2pl,conditional 3pl,subjunctive present 1sg,subjunctive present 2sg,subjunctive present 3sg,subjunctive present 1pl,subjunctive present 2pl,subjunctive present 3pl,subjunctive imperfect 1sg,subjunctive imperfect 2sg,subjunctive imperfect 3sg,subjunctive imperfect 1pl,subjunctive imperfect 2pl,subjunctive imperfect 3pl,imperative 2sg,imperative 1pl,imperative 2pl,group
1131,croître,croissant,crû,croîs,croîs,croît,croissons,croissez,croissent,croissais,croissais,croissait,croissions,croissiez,croissaient,crûs,crûs,crût,crûmes,crûtes,crurent,croîtrai,croîtras,croîtra,croîtrons,croîtrez,croîtront,croîtrais,croîtrais,croîtrait,croîtrions,croîtriez,croîtraient,croisse,croisses,croisse,croissions,croissiez,croissent,crûsse,crûsses,crût,crussions,crussiez,crussent,croîs,croissons,croissez,None
3890,recroître,recroissant,recrû,recroîs,recroîs,recroît,recroissons,recroissez,recroissent,recroissais,recroissais,recroissait,recroissions,recroissiez,recroissaient,recrûs,recrûs,recrût,recrûmes,recrûtes,recrurent,recroîtrai,recroîtras,recroîtra,recroîtrons,recroîtrez,recroîtront,recroîtrais,recroîtrais,recroîtrait,recroîtrions,recroîtriez,recroîtraient,recroisse,recroisses,recroisse,recroissions,recroissiez,recroissent,recrûsse,recrûsses,recrût,recrussions,recrussiez,recrussent,recroîs,recroissons,recroissez,None


#### connoître

In [73]:
like_connoitre, remaining_data = is_like("connoître", "oître", "connoître")
display(like_connoitre)

,infinitive,present participle,past participle,present 1sg,present 2sg,present 3sg,present 1pl,present 2pl,present 3pl,imperfect 1sg,imperfect 2sg,imperfect 3sg,imperfect 1pl,imperfect 2pl,imperfect 3pl,past historic 1sg,past historic 2sg,past historic 3sg,past historic 1pl,past historic 2pl,past historic 3pl,future 1sg,future 2sg,future 3sg,future 1pl,future 2pl,future 3pl,conditional 1sg,conditional 2sg,conditional 3sg,conditional 1pl,conditional 2pl,conditional 3pl,subjunctive present 1sg,subjunctive present 2sg,subjunctive present 3sg,subjunctive present 1pl,subjunctive present 2pl,subjunctive present 3pl,subjunctive imperfect 1sg,subjunctive imperfect 2sg,subjunctive imperfect 3sg,subjunctive imperfect 1pl,subjunctive imperfect 2pl,subjunctive imperfect 3pl,imperative 2sg,imperative 1pl,imperative 2pl,group
0,oître,oissant,u,ois,ois,oît,oissons,oissez,oissent,oissais,oissais,oissait,oissions,oissiez,oissaient,us,us,ut,ûmes,ûtes,urent,oîtrai,oîtras,oîtra,oîtrons,oîtrez,oîtront,oîtrais,oîtrais,oîtrait,oîtrions,oîtriez,oîtraient,oisse,oisses,oisse,oissions,oissiez,oissent,usse,usses,ût,ussions,ussiez,ussent,ois,oissons,oissez,None


,infinitive,present participle,past participle,present 1sg,present 2sg,present 3sg,present 1pl,present 2pl,present 3pl,imperfect 1sg,imperfect 2sg,imperfect 3sg,imperfect 1pl,imperfect 2pl,imperfect 3pl,past historic 1sg,past historic 2sg,past historic 3sg,past historic 1pl,past historic 2pl,past historic 3pl,future 1sg,future 2sg,future 3sg,future 1pl,future 2pl,future 3pl,conditional 1sg,conditional 2sg,conditional 3sg,conditional 1pl,conditional 2pl,conditional 3pl,subjunctive present 1sg,subjunctive present 2sg,subjunctive present 3sg,subjunctive present 1pl,subjunctive present 2pl,subjunctive present 3pl,subjunctive imperfect 1sg,subjunctive imperfect 2sg,subjunctive imperfect 3sg,subjunctive imperfect 1pl,subjunctive imperfect 2pl,subjunctive imperfect 3pl,imperative 2sg,imperative 1pl,imperative 2pl,group
75,accroître,accroissant,accru,accrois,accrois,accroît,accroissons,accroissez,accroissent,accroissais,accroissais,accroissait,accroissions,accroissiez,accroissaient,accrus,accrus,accrut,accrûmes,accrûtes,accrurent,accroîtrai,accroîtras,accroîtra,accroîtrons,accroîtrez,accroîtront,accroîtrais,accroîtrais,accroîtrait,accroîtrions,accroîtriez,accroîtraient,accroisse,accroisses,accroisse,accroissions,accroissiez,accroissent,accrusse,accrusses,accrût,accrussions,accrussiez,accrussent,accrois,accroissons,accroissez,None
985,connoître,connoissant,connu,connois,connois,connoît,connoissons,connoissez,connoissent,connoissais,connoissais,connoissait,connoissions,connoissiez,connoissaient,connus,connus,connut,connûmes,connûtes,connurent,connoîtrai,connoîtras,connoîtra,connoîtrons,connoîtrez,connoîtront,connoîtrais,connoîtrais,connoîtrait,connoîtrions,connoîtriez,connoîtraient,connoisse,connoisses,connoisse,connoissions,connoissiez,connoissent,connusse,connusses,connût,connussions,connussiez,connussent,connois,connoissons,connoissez,None
1462,décroître,décroissant,décru,décrois,décrois,décroît,décroissons,décroissez,décroissent,décroissais,décroissais,décroissait,décroissions,décroissiez,décroissaient,décrus,décrus,décrut,décrûmes,décrûtes,décrurent,décroîtrai,décroîtras,décroîtra,décroîtrons,décroîtrez,décroîtront,décroîtrais,décroîtrais,décroîtrait,décroîtrions,décroîtriez,décroîtraient,décroisse,décroisses,décroisse,décroissions,décroissiez,décroissent,décrusse,décrusses,décrût,décrussions,décrussiez,décrussent,décrois,décroissons,décroissez,None
3870,reconnoître,reconnoissant,reconnu,reconnois,reconnois,reconnoît,reconnoissons,reconnoissez,reconnoissent,reconnoissais,reconnoissais,reconnoissait,reconnoissions,reconnoissiez,reconnoissaient,reconnus,reconnus,reconnut,reconnûmes,reconnûtes,reconnurent,reconnoîtrai,reconnoîtras,reconnoîtra,reconnoîtrons,reconnoîtrez,reconnoîtront,reconnoîtrais,reconnoîtrais,reconnoîtrait,reconnoîtrions,reconnoîtriez,reconnoîtraient,reconnoisse,reconnoisses,reconnoisse,reconnoissions,reconnoissiez,reconnoissent,reconnusse,reconnusses,reconnût,reconnussions,reconnussiez,reconnussent,reconnois,reconnoissons,reconnoissez,None


#### venir, tenir

In [74]:
like_venir, remaining_data = is_like("venir", "enir", "venir/tenir")
display(like_venir)

,infinitive,present participle,past participle,present 1sg,present 2sg,present 3sg,present 1pl,present 2pl,present 3pl,imperfect 1sg,imperfect 2sg,imperfect 3sg,imperfect 1pl,imperfect 2pl,imperfect 3pl,past historic 1sg,past historic 2sg,past historic 3sg,past historic 1pl,past historic 2pl,past historic 3pl,future 1sg,future 2sg,future 3sg,future 1pl,future 2pl,future 3pl,conditional 1sg,conditional 2sg,conditional 3sg,conditional 1pl,conditional 2pl,conditional 3pl,subjunctive present 1sg,subjunctive present 2sg,subjunctive present 3sg,subjunctive present 1pl,subjunctive present 2pl,subjunctive present 3pl,subjunctive imperfect 1sg,subjunctive imperfect 2sg,subjunctive imperfect 3sg,subjunctive imperfect 1pl,subjunctive imperfect 2pl,subjunctive imperfect 3pl,imperative 2sg,imperative 1pl,imperative 2pl,group
0,enir,enant,enu,iens,iens,ient,enons,enez,iennent,enais,enais,enait,enions,eniez,enaient,ins,ins,int,înmes,întes,inrent,iendrai,iendras,iendra,iendrons,iendrez,iendront,iendrais,iendrais,iendrait,iendrions,iendriez,iendraient,ienne,iennes,ienne,enions,eniez,iennent,insse,insses,înt,inssions,inssiez,inssent,iens,enons,enez,None


,infinitive,present participle,past participle,present 1sg,present 2sg,present 3sg,present 1pl,present 2pl,present 3pl,imperfect 1sg,imperfect 2sg,imperfect 3sg,imperfect 1pl,imperfect 2pl,imperfect 3pl,past historic 1sg,past historic 2sg,past historic 3sg,past historic 1pl,past historic 2pl,past historic 3pl,future 1sg,future 2sg,future 3sg,future 1pl,future 2pl,future 3pl,conditional 1sg,conditional 2sg,conditional 3sg,conditional 1pl,conditional 2pl,conditional 3pl,subjunctive present 1sg,subjunctive present 2sg,subjunctive present 3sg,subjunctive present 1pl,subjunctive present 2pl,subjunctive present 3pl,subjunctive imperfect 1sg,subjunctive imperfect 2sg,subjunctive imperfect 3sg,subjunctive imperfect 1pl,subjunctive imperfect 2pl,subjunctive imperfect 3pl,imperative 2sg,imperative 1pl,imperative 2pl,group
43,abstenir,abstenant,abstenu,abstiens,abstiens,abstient,abstenons,abstenez,abstiennent,abstenais,abstenais,abstenait,abstenions,absteniez,abstenaient,abstins,abstins,abstint,abstînmes,abstîntes,abstinrent,abstiendrai,abstiendras,abstiendra,abstiendrons,abstiendrez,abstiendront,abstiendrais,abstiendrais,abstiendrait,abstiendrions,abstiendriez,abstiendraient,abstienne,abstiennes,abstienne,abstenions,absteniez,abstiennent,abstinsse,abstinsses,abstînt,abstinssions,abstinssiez,abstinssent,abstiens,abstenons,abstenez,None
123,advenir,advenant,advenu,adviens,adviens,advient,advenons,advenez,adviennent,advenais,advenais,advenait,advenions,adveniez,advenaient,advins,advins,advint,advînmes,advîntes,advinrent,adviendrai,adviendras,adviendra,adviendrons,adviendrez,adviendront,adviendrais,adviendrais,adviendrait,adviendrions,adviendriez,adviendraient,advienne,adviennes,advienne,advenions,adveniez,adviennent,advinsse,advinsses,advînt,advinssions,advinssiez,advinssent,adviens,advenons,advenez,None
277,appartenir,appartenant,appartenu,appartiens,appartiens,appartient,appartenons,appartenez,appartiennent,appartenais,appartenais,appartenait,appartenions,apparteniez,appartenaient,appartins,appartins,appartint,appartînmes,appartîntes,appartinrent,appartiendrai,appartiendras,appartiendra,appartiendrons,appartiendrez,appartiendront,appartiendrais,appartiendrais,appartiendrait,appartiendrions,appartiendriez,appartiendraient,appartienne,appartiennes,appartienne,appartenions,apparteniez,appartiennent,appartinsse,appartinsses,appartînt,appartinssions,appartinssiez,appartinssent,appartiens,appartenons,appartenez,None
372,attenir,attenant,attenu,attiens,attiens,attient,attenons,attenez,attiennent,attenais,attenais,attenait,attenions,atteniez,attenaient,attins,attins,attint,attînmes,attîntes,attinrent,attiendrai,attiendras,attiendra,attiendrons,attiendrez,attiendront,attiendrais,attiendrais,attiendrait,attiendrions,attiendriez,attiendraient,attienne,attiennes,attienne,attenions,atteniez,attiennent,attinsse,attinsses,attînt,attinssions,attinssiez,attinssent,attiens,attenons,attenez,None
819,circonvenir,circonvenant,circonvenu,circonviens,circonviens,circonvient,circonvenons,circonvenez,circonviennent,circonvenais,circonvenais,circonvenait,circonvenions,circonveniez,circonvenaient,circonvins,circonvins,circonvint,circonvînmes,circonvîntes,circonvinrent,circonviendrai,circonviendras,circonviendra,circonviendrons,circonviendrez,circonviendront,circonviendrais,circonviendrais,circonviendrait,circonviendrions,circonviendriez,circonviendraient,circonvienne,circonviennes,circonvienne,circonvenions,circonveniez,circonviennent,circonvinsse,circonvinsses,circonvînt,circonvinssions,circonvinssiez,circonvinssent,circonviens,circonvenons,circonvenez,None
1010,contenir,contenant,contenu,contiens,contiens,contient,contenons,contenez,contiennent,contenais,contenais,contenait,contenions,conteniez,contenaient,contins,contins,contint,contînmes,contîntes,continrent,contiendrai,contiendras,contiendra,contiendrons,contiendrez,contiendront,contiendrais,contiendrais,contiendrait,contiendrions,contiendriez,contiendraient,contienne,contiennes,contienne,contenions,conteniez

#### mourir

In [75]:
like_mourir, remaining_data = is_like("mourir", "mourir", "mourir")
display(like_mourir)

,infinitive,present participle,past participle,present 1sg,present 2sg,present 3sg,present 1pl,present 2pl,present 3pl,imperfect 1sg,imperfect 2sg,imperfect 3sg,imperfect 1pl,imperfect 2pl,imperfect 3pl,past historic 1sg,past historic 2sg,past historic 3sg,past historic 1pl,past historic 2pl,past historic 3pl,future 1sg,future 2sg,future 3sg,future 1pl,future 2pl,future 3pl,conditional 1sg,conditional 2sg,conditional 3sg,conditional 1pl,conditional 2pl,conditional 3pl,subjunctive present 1sg,subjunctive present 2sg,subjunctive present 3sg,subjunctive present 1pl,subjunctive present 2pl,subjunctive present 3pl,subjunctive imperfect 1sg,subjunctive imperfect 2sg,subjunctive imperfect 3sg,subjunctive imperfect 1pl,subjunctive imperfect 2pl,subjunctive imperfect 3pl,imperative 2sg,imperative 1pl,imperative 2pl,group
0,mourir,mourant,mort,meurs,meurs,meurt,mourons,mourez,meurent,mourais,mourais,mourait,mourions,mouriez,mouraient,mourus,mourus,mourut,mourûmes,mourûtes,moururent,mourrai,mourras,mourra,mourrons,mourrez,mourront,mourrais,mourrais,mourrait,mourrions,mourriez,mourraient,meure,meures,meure,mourions,mouriez,meurent,mourusse,mourusses,mourût,mourussions,mourussiez,mourussent,meurs,mourons,mourez,None


,infinitive,present participle,past participle,present 1sg,present 2sg,present 3sg,present 1pl,present 2pl,present 3pl,imperfect 1sg,imperfect 2sg,imperfect 3sg,imperfect 1pl,imperfect 2pl,imperfect 3pl,past historic 1sg,past historic 2sg,past historic 3sg,past historic 1pl,past historic 2pl,past historic 3pl,future 1sg,future 2sg,future 3sg,future 1pl,future 2pl,future 3pl,conditional 1sg,conditional 2sg,conditional 3sg,conditional 1pl,conditional 2pl,conditional 3pl,subjunctive present 1sg,subjunctive present 2sg,subjunctive present 3sg,subjunctive present 1pl,subjunctive present 2pl,subjunctive present 3pl,subjunctive imperfect 1sg,subjunctive imperfect 2sg,subjunctive imperfect 3sg,subjunctive imperfect 1pl,subjunctive imperfect 2pl,subjunctive imperfect 3pl,imperative 2sg,imperative 1pl,imperative 2pl,group
3129,mourir,mourant,mort,meurs,meurs,meurt,mourons,mourez,meurent,mourais,mourais,mourait,mourions,mouriez,mouraient,mourus,mourus,mourut,mourûmes,mourûtes,moururent,mourrai,mourras,mourra,mourrons,mourrez,mourront,mourrais,mourrais,mourrait,mourrions,mourriez,mourraient,meure,meures,meure,mourions,mouriez,meurent,mourusse,mourusses,mourût,mourussions,mourussiez,mourussent,meurs,mourons,mourez,None


#### faillir

In [76]:
like_faillir, remaining_data = is_like("faillir", "faillir", "faillir")
display(like_faillir)

,infinitive,present participle,past participle,present 1sg,present 2sg,present 3sg,present 1pl,present 2pl,present 3pl,imperfect 1sg,imperfect 2sg,imperfect 3sg,imperfect 1pl,imperfect 2pl,imperfect 3pl,past historic 1sg,past historic 2sg,past historic 3sg,past historic 1pl,past historic 2pl,past historic 3pl,future 1sg,future 2sg,future 3sg,future 1pl,future 2pl,future 3pl,conditional 1sg,conditional 2sg,conditional 3sg,conditional 1pl,conditional 2pl,conditional 3pl,subjunctive present 1sg,subjunctive present 2sg,subjunctive present 3sg,subjunctive present 1pl,subjunctive present 2pl,subjunctive present 3pl,subjunctive imperfect 1sg,subjunctive imperfect 2sg,subjunctive imperfect 3sg,subjunctive imperfect 1pl,subjunctive imperfect 2pl,subjunctive imperfect 3pl,imperative 2sg,imperative 1pl,imperative 2pl,group
0,faillir,faillant,failli,faux,faux,faut,faillons,faillez,faillent,faillais,faillais,faillait,faillions,failliez,faillaient,faillis,faillis,faillit,faillîmes,faillîtes,faillirent,faudrai,faudras,faudra,faudrons,faudrez,faudront,faudrais,faudrais,faudrait,faudrions,faudriez,faudraient,faille,failles,faille,faillions,failliez,faillent,faillisse,faillisses,faillît,faillissions,faillissiez,faillissent,None,None,None,None


,infinitive,present participle,past participle,present 1sg,present 2sg,present 3sg,present 1pl,present 2pl,present 3pl,imperfect 1sg,imperfect 2sg,imperfect 3sg,imperfect 1pl,imperfect 2pl,imperfect 3pl,past historic 1sg,past historic 2sg,past historic 3sg,past historic 1pl,past historic 2pl,past historic 3pl,future 1sg,future 2sg,future 3sg,future 1pl,future 2pl,future 3pl,conditional 1sg,conditional 2sg,conditional 3sg,conditional 1pl,conditional 2pl,conditional 3pl,subjunctive present 1sg,subjunctive present 2sg,subjunctive present 3sg,subjunctive present 1pl,subjunctive present 2pl,subjunctive present 3pl,subjunctive imperfect 1sg,subjunctive imperfect 2sg,subjunctive imperfect 3sg,subjunctive imperfect 1pl,subjunctive imperfect 2pl,subjunctive imperfect 3pl,imperative 2sg,imperative 1pl,imperative 2pl,group
2246,faillir,faillant,failli,faux,faux,faut,faillons,faillez,faillent,faillais,faillais,faillait,faillions,failliez,faillaient,faillis,faillis,faillit,faillîmes,faillîtes,faillirent,faudrai,faudras,faudra,faudrons,faudrez,faudront,faudrais,faudrais,faudrait,faudrions,faudriez,faudraient,faille,failles,faille,faillions,failliez,faillent,faillisse,faillisses,faillît,faillissions,faillissiez,faillissent,None,None,None,None


#### quérir

In [77]:
like_querir, remaining_data = is_like("quérir", "érir", "quérir")
display(like_querir)

,infinitive,present participle,past participle,present 1sg,present 2sg,present 3sg,present 1pl,present 2pl,present 3pl,imperfect 1sg,imperfect 2sg,imperfect 3sg,imperfect 1pl,imperfect 2pl,imperfect 3pl,past historic 1sg,past historic 2sg,past historic 3sg,past historic 1pl,past historic 2pl,past historic 3pl,future 1sg,future 2sg,future 3sg,future 1pl,future 2pl,future 3pl,conditional 1sg,conditional 2sg,conditional 3sg,conditional 1pl,conditional 2pl,conditional 3pl,subjunctive present 1sg,subjunctive present 2sg,subjunctive present 3sg,subjunctive present 1pl,subjunctive present 2pl,subjunctive present 3pl,subjunctive imperfect 1sg,subjunctive imperfect 2sg,subjunctive imperfect 3sg,subjunctive imperfect 1pl,subjunctive imperfect 2pl,subjunctive imperfect 3pl,imperative 2sg,imperative 1pl,imperative 2pl,group
0,érir,érant,is,iers,iers,iert,érons,érez,ièrent,érais,érais,érait,érions,ériez,éraient,is,is,it,îmes,îtes,irent,errai,erras,erra,errons,errez,erront,errais,errais,errait,errions,erriez,erraient,ière,ières,ière,érions,ériez,ièrent,isse,isses,ît,issions,issiez,issent,iers,érons,érez,None


,infinitive,present participle,past participle,present 1sg,present 2sg,present 3sg,present 1pl,present 2pl,present 3pl,imperfect 1sg,imperfect 2sg,imperfect 3sg,imperfect 1pl,imperfect 2pl,imperfect 3pl,past historic 1sg,past historic 2sg,past historic 3sg,past historic 1pl,past historic 2pl,past historic 3pl,future 1sg,future 2sg,future 3sg,future 1pl,future 2pl,future 3pl,conditional 1sg,conditional 2sg,conditional 3sg,conditional 1pl,conditional 2pl,conditional 3pl,subjunctive present 1sg,subjunctive present 2sg,subjunctive present 3sg,subjunctive present 1pl,subjunctive present 2pl,subjunctive present 3pl,subjunctive imperfect 1sg,subjunctive imperfect 2sg,subjunctive imperfect 3sg,subjunctive imperfect 1pl,subjunctive imperfect 2pl,subjunctive imperfect 3pl,imperative 2sg,imperative 1pl,imperative 2pl,group
97,acquérir,acquérant,acquis,acquiers,acquiers,acquiert,acquérons,acquérez,acquièrent,acquérais,acquérais,acquérait,acquérions,acquériez,acquéraient,acquis,acquis,acquit,acquîmes,acquîtes,acquirent,acquerrai,acquerras,acquerra,acquerrons,acquerrez,acquerront,acquerrais,acquerrais,acquerrait,acquerrions,acquerriez,acquerraient,acquière,acquières,acquière,acquérions,acquériez,acquièrent,acquisse,acquisses,acquît,acquissions,acquissiez,acquissent,acquiers,acquérons,acquérez,None
986,conquérir,conquérant,conquis,conquiers,conquiers,conquiert,conquérons,conquérez,conquièrent,conquérais,conquérais,conquérait,conquérions,conquériez,conquéraient,conquis,conquis,conquit,conquîmes,conquîtes,conquirent,conquerrai,conquerras,conquerra,conquerrons,conquerrez,conquerront,conquerrais,conquerrais,conquerrait,conquerrions,conquerriez,conquerraient,conquière,conquières,conquière,conquérions,conquériez,conquièrent,conquisse,conquisses,conquît,conquissions,conquissiez,conquissent,conquiers,conquérons,conquérez,None
2047,enquérir,enquérant,enquis,enquiers,enquiers,enquiert,enquérons,enquérez,enquièrent,enquérais,enquérais,enquérait,enquérions,enquériez,enquéraient,enquis,enquis,enquit,enquîmes,enquîtes,enquirent,enquerrai,enquerras,enquerra,enquerrons,enquerrez,enquerront,enquerrais,enquerrais,enquerrait,enquerrions,enquerriez,enquerraient,enquière,enquières,enquière,enquérions,enquériez,enquièrent,enquisse,enquisses,enquît,enquissions,enquissiez,enquissent,enquiers,enquérons,enquérez,None
3734,quérir,quérant,quis,quiers,quiers,quiert,quérons,quérez,quièrent,quérais,quérais,quérait,quérions,quériez,quéraient,quis,quis,quit,quîmes,quîtes,quirent,querrai,querras,querra,querrons,querrez,querront,querrais,querrais,querrait,querrions,querriez,querraient,quière,quières,quière,quérions,quériez,quièrent,quisse,quisses,quît,quissions,quissiez,quissent,quiers,quérons,quérez,None
3871,reconquérir,reconquérant,reconquis,reconquiers,reconquiers,reconquiert,reconquérons,reconquérez,reconquièrent,reconquérais,reconquérais,reconquérait,reconquérions,reconquériez,reconquéraient,reconquis,reconquis,reconquit,reconquîmes,reconquîtes,reconquirent,reconquerrai,reconquerras,reconquerra,reconquerrons,reconquerrez,reconquerront,reconquerrais,reconquerrais,reconquerrait,reconquerrions,reconquerriez,reconquerraient,reconquière,reconquières,reconquière,reconquérions,reconquériez,reconquièrent,reconquisse,reconquisses,reconquît,reconquissions,reconquissiez,reconquissent,reconquiers,reconquérons,reconquérez,None
4086,requérir,requérant,requis,requiers,requiers,requiert,requérons,requérez,requièrent,requérais,requérais,requérait,requérions,requériez,requéraient,requis,requis,requit,requîmes,requîtes,requirent,requerrai,requerras,requerra,requerrons,requerrez,requerront,requerrais,requerrais,requerrait,requerrions,requerriez,requerraient,requière,requières,requière,requérions,requériez,requièrent,requisse,requisses,requît,requissions,requissiez,requissent,requiers,requérons,requérez,None


In [78]:
data.to_csv("rest.csv")

In [79]:
all_ending_patterns = sorted(all_ending_patterns, key=lambda x: (len(x["infinitive"]), x["infinitive"]))
all_ending_patterns = pd.DataFrame(all_ending_patterns)
all_ending_patterns

,infinitive,present participle,past participle,present 1sg,present 2sg,present 3sg,present 1pl,present 2pl,present 3pl,imperfect 1sg,imperfect 2sg,imperfect 3sg,imperfect 1pl,imperfect 2pl,imperfect 3pl,past historic 1sg,past historic 2sg,past historic 3sg,past historic 1pl,past historic 2pl,past historic 3pl,future 1sg,future 2sg,future 3sg,future 1pl,future 2pl,future 3pl,conditional 1sg,conditional 2sg,conditional 3sg,conditional 1pl,conditional 2pl,conditional 3pl,subjunctive present 1sg,subjunctive present 2sg,subjunctive present 3sg,subjunctive present 1pl,subjunctive present 2pl,subjunctive present 3pl,subjunctive imperfect 1sg,subjunctive imperfect 2sg,subjunctive imperfect 3sg,subjunctive imperfect 1pl,subjunctive imperfect 2pl,subjunctive imperfect 3pl,imperative 2sg,imperative 1pl,imperative 2pl,group,groups
0,er,ant,é,e,es,e,ons,ez,ent,ais,ais,ait,ions,iez,aient,ai,as,a,âmes,âtes,èrent,erai,eras,era,erons,erez,eront,erais,erais,erait,erions,eriez,eraient,e,es,e,ions,iez,ent,asse,asses,ât,assions,assiez,assent,e,ons,ez,None,"{-éCer, -e[tl]er, -e[wxy]er, -ayer, -[ou]yer, ..."
1,er,ant,é,e,es,e,ons,ez,ent,ais,ais,ait,ions,iez,aient,ai,as,a,âmes,âtes,èrent,ai,as,a,ons,ez,ont,ais,ais,ait,ions,iez,aient,e,es,e,ions,iez,ent,asse,asses,ât,assions,assiez,assent,e,ons,ez,None,{-envoyer}
2,ir,issant,i,is,is,it,issons,issez,issent,issais,issais,issait,issions,issiez,issaient,is,is,it,îmes,îtes,irent,irai,iras,ira,irons,irez,iront,irais,irais,irait,irions,iriez,iraient,isse,isses,isse,issions,issiez,issent,isse,isses,ît,issions,issiez,issent,is,issons,issez,None,{-ir}
3,ir,ant,i,s,s,t,ons,ez,ent,ais,ais,ait,ions,iez,aient,is,is,it,îmes,îtes,irent,irai,iras,ira,irons,irez,iront,irais,irais,irait,irions,iriez,iraient,e,es,e,ions,iez,ent,isse,isses,ît,issions,issiez,issent,s,ons,ez,None,{-ir like partir}
4,ir,ant,u,s,s,t,ons,ez,ent,ais,ais,ait,ions,iez,aient,us,us,ut,ûmes,ûtes,urent,rai,ras,ra,rons,rez,ront,rais,rais,rait,rions,riez,raient,e,es,e,ions,iez,ent,usse,usses,ût,ussions,ussiez,ussent,s,ons,ez,None,{-courir}
5,ir,ant,u,s,s,,ons,ez,ent,ais,ais,ait,ions,iez,aient,is,is,it,îmes,îtes,irent,irai,iras,ira,irons,irez,iront,irais,irais,irait,irions,iriez,iraient,e,es,e,ions,iez,ent,isse,isses,ît,issions,issiez,issent,s,ons,ez,None,{-vêtir}
6,ir,ant,i,e,es,e,ons,ez,ent,ais,ais,ait,ions,iez,aient,is,is,it,îmes,îtes,irent,"(irai, erai)","(iras, eras)","(ira, era)","(irons, erons)","(irez, erez)","(iront, eront)","(irais, erais)","(irais, erais)","(irait, erait)","(irions, erions)","(iriez, eriez)","(iraient, eraient)",e,es,e,ions,iez,ent,isse,isses,ît,issions,issiez,issent,e,ons,ez,None,{-cueillir/-saillir}
7,ir,ant,i,e,es,e,ons,ez,ent,ais,ais,ait,ions,iez,aient,is,is,it,îmes,îtes,irent,irai,iras,ira,irons,irez,iront,irais,irais,irait,irions,iriez,iraient,e,es,e,ions,iez,ent,isse,isses,ît,issions,issiez,issent,e,ons,ez,None,{défaillir}
8,ir,ant,ert,e,es,e,ons,ez,ent,ais,ais,ait,ions,iez,aient,is,is,it,îmes,îtes,irent,irai,iras,ira,irons,irez,iront,irais,irais,irait,irions,iriez,iraient,e,es,e,ions,iez,ent,isse,isses,ît,issions,issiez,issent,e,ons,ez,None,{-ouvrir/-ouffrir}
9,re,ant,u,s,s,,ons,ez,ent,ais,ais,ait,ions,iez,aient,is,is,it,îmes,îtes,irent,rai,ras,ra,rons,rez,ront,rais,rais,rait,rions,riez,raient,e,es,e,ions,iez,ent,isse,isses,ît,issions,issiez,issent,s,ons,ez,None,"{-mettre, -re, -coudre, -cre, -battre}"


In [80]:
all_ending_patterns[all_ending_patterns.duplicated(subset=all_ending_patterns.columns[:-1], keep=False)]

,infinitive,present participle,past participle,present 1sg,present 2sg,present 3sg,present 1pl,present 2pl,present 3pl,imperfect 1sg,imperfect 2sg,imperfect 3sg,imperfect 1pl,imperfect 2pl,imperfect 3pl,past historic 1sg,past historic 2sg,past historic 3sg,past historic 1pl,past historic 2pl,past historic 3pl,future 1sg,future 2sg,future 3sg,future 1pl,future 2pl,future 3pl,conditional 1sg,conditional 2sg,conditional 3sg,conditional 1pl,conditional 2pl,conditional 3pl,subjunctive present 1sg,subjunctive present 2sg,subjunctive present 3sg,subjunctive present 1pl,subjunctive present 2pl,subjunctive present 3pl,subjunctive imperfect 1sg,subjunctive imperfect 2sg,subjunctive imperfect 3sg,subjunctive imperfect 1pl,subjunctive imperfect 2pl,subjunctive imperfect 3pl,imperative 2sg,imperative 1pl,imperative 2pl,group,groups


In [81]:
def longest_common_prefix(strings: pd.Series):
    non_null_strings = strings.dropna()
    prefix = non_null_strings[0]
    for s in non_null_strings[1:]:
        while prefix and not s.startswith(prefix):
            prefix = prefix[:-1]

    suffixes = tuple((s[len(prefix):] if s is not None else None) for s in strings)

    return prefix, suffixes

In [82]:
def analyze_group(label: str, col_names: list[str]):
    patterns = data[col_names].apply(longest_common_prefix, axis=1)
    pattern_groups = {}
    for i, (prefix, suffixes) in patterns.items():
        if suffixes not in pattern_groups:
            pattern_groups[suffixes] = []
        pattern_groups[suffixes].append((i, prefix))

    # pattern_groups = coalesce_pattern_groups(pattern_groups)

    pattern_groups = sorted(
        pattern_groups.items(), key=lambda x: len(x[1]), reverse=True
    )
    pattern_groups = [(s, tuple(zip(*v))) for s, v in pattern_groups]
    if label not in data:
        data.insert(data.shape[1], label, 0)
    for i, (suffixes, (indices, prefix)) in enumerate(pattern_groups):
        data.loc[indices, label] = i
    return {
        "label": label,
        "col_names": col_names,
        "data": [(suffixes, prefix) for (suffixes, (_, prefix)) in pattern_groups],
    }

In [83]:
def split_data(group, aggregate=None, subdata=data, last_n=-1):
    if aggregate is None:
        aggregate = {}
    if (
        subdata[group["label"]].nunique() == 1
        or last_n == -subdata["infinitive"].str.len().max()
    ):
        pattern = group["data"][subdata[group["label"]].iloc[0]][0]
        if pattern not in aggregate:
            aggregate[pattern] = []
        aggregate[pattern].append(subdata["infinitive"].str[last_n + 1 :].iloc[0])
    else:
        subdata.groupby(by=subdata["infinitive"].str[last_n:], as_index=False).apply(
            lambda x: split_data(group, aggregate, x, last_n - 1)
        )
    return aggregate

## Present participle pattern


| infinitive | present 1pl | present participle |
| ---------- | ----------- | ------------------ |
| avoir      | avons       | ayant              |
| être       | sommes      | étant              |
| savoir     | savons      | sachant            |
| ...        | -ons        | -ant               |


In [84]:
present_participle_group = analyze_group(
    "present_participle_group", ["present 1pl", "present participle"]
)
split_data(present_participle_group)

AttributeError: 'tuple' object has no attribute 'startswith'

## Subjunctive pattern


In [85]:
subjunctive_group1 = analyze_group(
    "subjunctive_group1",
    [
        "present 3pl",
        "subjunctive present 1sg",
        "subjunctive present 2sg",
        "subjunctive present 3sg",
        "subjunctive present 3pl",
    ],
)
split_data(subjunctive_group1)

{('nt', '', 's', '', 'nt'): ['e', 'er', 'uir', 'ïr'],
 ('', None, None, None, None): ['sir']}

| infinitive | present 3pl | subjunctive 1sg | subjunctive 2sg | subjunctive 3sg | subjunctive 3pl |
| ---------- | ----------- | --------------- | --------------- | --------------- | --------------- |
| aller      | vont        | aille           | ailles          | aille           | aillent         |
| avoir      | ont         | aie             | aies            | ait             | aient           |
| être       | sont        | sois            | sois            | soit            | soient          |
| savoir     | savent      | sache           | saches          | sache           | sachent         |
| -faire     | -font       | -fasse          | -fasses         | -fasse          | -fassent        |
| -pouvoir   | -peuvent    | -puisse         | -puisses        | -puisse         | -puissent       |
| ...        | -ent        | -e              | -es             | -e              | -ent            |


In [86]:
subjunctive_group2 = analyze_group(
    "subjunctive_group2",
    ["present 1pl", "subjunctive present 1pl", "subjunctive present 2pl"],
)
split_data(subjunctive_group2)

{('ons', 'ions', 'iez'): ['e', 'uir', 'ïr'],
 ('çons', 'cions', 'ciez'): ['er'],
 ('', None, None): ['sir']}

| infinitive | present 1pl | subjunctive 1pl | subjunctive 2pl |
| ---------- | ----------- | --------------- | --------------- |
| avoir      | avons       | ayons           | ayez            |
| être       | sommes      | soyons          | soyez           |
| savoir     | savons      | sachions        | sachiez         |
| -faire     | -faisons    | -fassions       | -fassiez        |
| -pouvoir   | -pouvons    | -puissions      | -puissiez       |
| -cer       | -çons       | -cions          | -ciez           |
| -ger       | -geons      | -gions          | -giez           |
| ...        | -ons        | -ions           | -iez            |


## Future pattern


In [ ]:
future_group = analyze_group(
    "future_group",
    [
        "infinitive",
        "future 1sg",
        "future 2sg",
        "future 3sg",
        "future 1pl",
        "future 2pl",
        "future 3pl",
    ],
)
split_data(future_group)

{('e', 'ai', 'as', 'a', 'ons', 'ez', 'ont'): ['e'],
 ('', 'ai', 'as', 'a', 'ons', 'ez', 'ont'): ['er', 'uir', 'ïr'],
 ('', None, None, None, None, None, None): ['sir']}

## Past participle pattern


In [88]:
past_participle_group = analyze_group(
    "past_participle_group", ["infinitive", "past participle"]
)
split_data(past_participle_group)

{('ire', 'yant'): ['aire'],
 ('re', ''): ['uire'],
 ('er', 'é'): ['er'],
 ('', None): ['sir'],
 ('r', ''): ['uir', 'ïr']}

## Imperfect pattern


In [89]:
imperfect_group = analyze_group(
    "imperfect_group",
    [
        "present 1pl",
        "imperfect 1sg",
        "imperfect 2sg",
        "imperfect 3sg",
        "imperfect 1pl",
        "imperfect 2pl",
        "imperfect 3pl",
    ],
)
split_data(imperfect_group)

{('ons', 'ais', 'ais', 'ait', 'ions', 'iez', 'aient'): ['e', 'ir', 'ïr'],
 ('çons', 'çais', 'çais', 'çait', 'cions', 'ciez', 'çaient'): ['er']}

| infinitive | present 1pl | imperfect 1sg | imperfect 2sg | imperfect 3sg | imperfect 1pl | imperfect 2pl | imperfect 3pl |
| ---------- | ----------- | ------------- | ------------- | ------------- | ------------- | ------------- | ------------- |
| être       | sommes      | étais         | étais         | était         | étions        | étiez         | étaient       |
| -cer       | -çons       | -çais         | -çais         | -çait         | -cions        | -ciez         | -çaient       |
| -ger       | -geons      | -geais        | -geais        | -geait        | -gions        | -giez         | -geaient      |
| ...        | -ons        | -ais          | -ais          | -ait          | -ions         | -iez          | -aient        |
